In [ ]:
#修改表格_增加欄位
#cursor=conn.cursor()
#row=cursor.execute("ALTER TABLE `edm`.`edm_log` ADD COLUMN `Username` VARCHAR(9) NULL AFTER `CleanCode`;")
#conn.commit()
#cursor.close()

#修改表格_刪減欄位
#cursor=conn.cursor()
#row=cursor.execute("ALTER TABLE `simulate`.`simulate_log2` DROP COLUMN `new_columns`;")
#conn.commit()
#cursor.close()

#選擇資料
# cursor=conn.cursor()
# cursor.execute("select COLUMN_NAME from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME = 'edm_log';")
# columns=cursor.fetchall()
# cursor.close()

#修改資料
#cursor=conn.cursor()
#row=cursor.execute("UPDATE `simulate`.`simulate_log2` SET `simulate_log2col` = '234' WHERE (`MyUnknownColumn` = '1');")
#conn.commit()
#cursor.close()

# #清空表格
# cursor=conn.cursor()
# row=cursor.execute("TRUNCATE TABLE edm.implementation;")
# conn.commit()
# cursor.close()

# #刪除資料
# cursor=conn.cursor()
# row=cursor.execute("DELETE FROM `edm`.`implementation` WHERE (`no` = '4623');")
# conn.commit()
# cursor.close()

In [ ]:
# !pip install PyMySQL

# Import & reconnect

In [ ]:
import time
import datetime
import pymysql
import numpy as np
import pandas as pd
import re
import os
import copy
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [ ]:
# conn=pymysql.connect(host="", port=3306, 
#                      user="", passwd="", 
#                      db="",
#                      charset="")

In [ ]:
#  conn.close()

# edm_log

## renew_on_time(v1)

In [ ]:
import time
import datetime
import pymysql
import numpy as np
import pandas as pd
import re
import os
import copy
from datetime import datetime, timedelta
# from IPython import display
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

conn=pymysql.connect(host="", port=3306, 
                     user="", passwd="", 
                     db="",
                     charset="")

while True:
    print(str(datetime.now()))
    
    ##### userlist #####
    cursor=conn.cursor()
    cursor.execute("select `Username` from `edm`.`member` ;") #where `Role`='學生';")
    u=cursor.fetchall()
    cursor.close()
    userlist=[]
    for i in u:
        userlist.append(i[0]) 

    ##### new data_index #####
    if os.path.isfile("./lastindex.txt") ==False:
        x= open("lastindex.txt","w",encoding="utf-8")
        x.close()
        ind=0
        print("now index:",ind)
        cursor=conn.cursor()
        cursor.execute("select * from edm_log")
        data=cursor.fetchall()
        cursor.close()
        len_data=len(data)
        print("新資料數：", len_data)

    else:
        with open("lastindex.txt","r",encoding="utf-8") as x: 
            ind=x.read()
        print("now index:",ind)
        cursor=conn.cursor()
        cursor.execute("select * from edm_log where log_id>{}".format(int(ind)))
        data=cursor.fetchall()
        cursor.close()
        len_data=len(data)
        print("新資料數：", len_data)
        
        
    if len_data!=0:
        ind2=data[-1][0]
        print("target index:", ind2)

        ##### remark, cleancode, username #####
        for i in range(len(data)):
            remark=""
            code= data[i][2]
            if pd.notnull(code):
                if "#" in code:
                    for r in re.findall("#.*[\n]?", code):
                        remark+=r
                        code="\n".join(code.split(r))
                    code=code.strip().replace("\'","\"")
                    remark=remark.strip().replace("\'","\"")
                else:
                    code=code.replace("\'","\"")
                    remark=remark.replace("\'","\"")
            username=data[i][1].split("/")[2]
            print(username)
            print("==code==")
            print(code)
            print("==remark==")
            print(remark)
            print("="*50)
            if len(remark)>1000:
                remark=remark[:1000]

            try:
                cursor=conn.cursor()
                row=cursor.execute("UPDATE `edm`.`edm_log` SET `CleanCode` = '{}', `Remark` = '{}', `Username` = '{}'  WHERE (`log_id` = '{}');".format(code, remark, username, data[i][0]))
                conn.commit()
                cursor.close()
            except:
                print("ERROR!!:", data[i][0],"\n===code===\n",code,"\n===remark===\n", remark,"\n===username===\n", username)
        print("=====finish: remark, cleancode, username=====")
        
        ##### new_data_df #####
        cursor=conn.cursor()
        cursor.execute("select * from edm_log where log_id>{}".format(int(ind)))
        data=cursor.fetchall()
        cursor.close()
        
        title=['log_id', 'user_id', 'code', 'event', 'isError', 'kernelId', 'notebookName', 'notebookPath', 'cellId', 'timestamp', 'text', 'codeLength', 'errorName', 'errorValue','Remark','Cleancode','Username']
        df=pd.DataFrame()
        for i in range(len(title)):
            li=[]
            for j in data:
                li.append(j[i])
            df[title[i]]=li
        print("本次資料：",len(df))

        ##### notebook now #####
        notebook_now="Untitled.ipynb"
        cursor=conn.cursor()
        cursor.execute("select `NotebookName` from `edm`.`section` where `SectionTimeStart` < '{}';".format(str(datetime.now())))
        sec=cursor.fetchall()
        cursor.close()
        if sec!=():
            notebook_now=sec[-1][0]
        print("notebook now:", notebook_now)

        ##### implementation #####
        redata=pd.read_excel("python_re.xlsx", sheet_name="全")
        redata=redata[redata["NotebookName"]==notebook_now]
        redata=redata.reset_index(drop=True)
        df2=df[df["notebookName"]==notebook_now]

        for user in userlist:
            print("==========renewing implementation:", user)
            df3=df2[df2["Username"]==user]
            df3=df3[pd.notnull(df3["code"])]
            code_mix="\n".join(list(df3["code"]))
            #print("user資料:", len(df3))
            #print("===codemix===")
            #print(code_mix)
            #print("============")
            for i in range(len(redata)):
                a=redata.loc[i,"KeypointNo"]
                cursor=conn.cursor()
                cursor.execute('''
                select `KeypointCount`,`Catched`, `KeypointDone` 
                from `edm`.`implementation` 
                where `NotebookName` = '{}' and `Username` = '{}' and `KeypointNo`={};
                '''.format(notebook_now, user, a))
                y=cursor.fetchall()
                cursor.close()
                
                # done, catched_plus, count_plus
                count=y[0][0]
                if pd.isnull(count): count=0
                catched=y[0][1]
                if pd.isnull(catched): catched=""
                done=y[0][2]
                if pd.isnull(done): done=0
                catched_plus=""
                x=re.findall(redata.loc[i,"KeypointRE"], code_mix)
                count_plus=len(x)
                if x !=[]:
                    done=1
                    catched_plus="\n".join(x)
                if pd.isnull(count):
                    count=count_plus
                else:
                    count+=count_plus
                if pd.isnull(catched):
                    catched=catched_plus
                else:
                    catched+=catched_plus    
                if catched_plus!="":
                    print("user:",user,"    target:",redata.loc[i,"KeypointRE"], "\n    本次完成:", done, "    計次:",count, "    結果:",catched_plus)
                    print("="*50)
                cursor=conn.cursor()
                catched=catched.replace("\'","\"")
                row=cursor.execute('''
                UPDATE `edm`.`implementation` 
                SET `KeypointDone` = {} , `KeypointCount` = {}, `Catched` = '{}'
                WHERE `NotebookName`='{}' and `Username`='{}' and `KeypointNo`={};
                '''.format(done, count, catched, notebook_now, user, a))
                conn.commit()
                cursor.close()
        print("finish: implementation renew" )
        print("="*50)

        with open("lastindex.txt","w",encoding="utf-8") as x:
            x.write(str(ind2))
            print("reset index:", ind2, "\n")
    
    time.sleep(10)
    if datetime.now() > datetime(2021,1,13,23,59,59,0):
        conn.close()
        break


## renew_on_time(v2)_modify

In [ ]:
import time
import datetime
import pymysql
import numpy as np
import pandas as pd
import re
import os
import copy
from datetime import datetime, timedelta
# from IPython import display
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

rd_n=0

while True:
    rd_n+=1
    print("="*80)
    print("Round", rd_n)
    
    conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
    print(str(datetime.now()))

    
    ##### userlist #####
    cursor=conn.cursor()
    cursor.execute("select `Username` from `edm`.`member` ;") #where `Role`='學生';")
    u=cursor.fetchall()
    cursor.close()
    userlist=[]
    for i in u:
        userlist.append(i[0]) 


    ##### new data_index #####
    if os.path.isfile("./lastindex.txt") ==False:
        x= open("lastindex.txt","w",encoding="utf-8")
        ind=0
        x.write(str(ind))
        x.close()
        print("now index:",ind)
        cursor=conn.cursor()
        cursor.execute("select * from edm_log")
        data=cursor.fetchall()
        cursor.close()
        len_data=len(data)
        print("新資料數：", len_data)
    if os.path.isfile("./lastindex.txt"):
        with open("lastindex.txt","r",encoding="utf-8") as x: 
            ind=x.read()
        print("now index:",ind)
        cursor=conn.cursor()
        cursor.execute("select * from edm_log where log_id>{}".format(int(ind)))
        data=cursor.fetchall()
        cursor.close()
        len_data=len(data)
        print("新資料數：", len_data)
        
        
    if len_data!=0:
        ind2=data[-1][0]
        print("target index:", ind2)

        ##### remark, cleancode, username #####
        print("=======start: remark, cleancode, username=======")
        for i in range(len(data)):
            remark=""
            code= data[i][2]
            if pd.notnull(code):
                if "#" in code:
                    for r in re.findall("#.*[\n]?", code):
                        remark+=r
                        code="\n".join(code.split(r))
                    code=code.strip().replace("\'","\"")
                    remark=remark.strip().replace("\'","\"")
                else:
                    code=code.replace("\'","\"")
                    remark=remark.replace("\'","\"")
            username=data[i][1].split("/")[2]
            
            print(username)
            print("==code==")
            print(code)
            print("==remark==")
            print(remark)
            print("="*50)
            if len(remark)>1000:
                remark=remark[:1000]
            try:
                sql='''UPDATE `edm`.`edm_log` 
                SET `CleanCode` = '{}', `Remark` = '{}', `Username` = '{}'
                WHERE (`log_id` = {});'''.format(code, remark, username, data[i][0])
                cursor=conn.cursor()
                row=cursor.execute(sql)
                conn.commit()
                cursor.close()
            except:
                print("ERROR!!:", data[i][0],"\n==code==\n",code,"\n==remark==\n", remark,"\n==username==\n", username)
            
        print("=======finish: remark, cleancode, username=======")

        
        ##### new_data_df #####
        cursor=conn.cursor()
        cursor.execute("select * from edm_log where log_id>{}".format(int(ind)))
        data=cursor.fetchall()
        cursor.close()
        
        title=['log_id', 'user_id', 'code', 'event', 'isError', 'kernelId', 'notebookName', 'notebookPath', 'cellId', 'timestamp', 'text', 'codeLength', 'errorName', 'errorValue','Remark','Cleancode','Username']
        df=pd.DataFrame()
        for i in range(len(title)):
            li=[]
            for j in data:
                li.append(j[i])
            df[title[i]]=li
        print("本次資料：",len(df))

        ##### notebook now #####
        notebook_now="Untitled.ipynb"
        cursor=conn.cursor()
        cursor.execute("select `NotebookName` from `edm`.`section` where `SectionTimeStart` < '{}';".format(str(datetime.now())))
        sec=cursor.fetchall()
        cursor.close()
        if sec!=():
            notebook_now=sec[-1][0]
        print("notebook now:", notebook_now)

        ##### implementation #####
        redata=pd.read_excel("python_re.xlsx", sheet_name="全")
        redata=redata[pd.notnull(redata["KeypointRE"])]
        redata=redata.reset_index(drop=True)
        
        notebook_list=list(set(redata["NotebookName"]))
        for user in userlist:
            print("\n==========renewing implementation:", user, "=============")
            df2=df[df["Username"]==user]
            for notebook in notebook_list:       
                print("renewing notebook:", notebook)
                df3=df2[df2["notebookName"]==notebook]
                df3=df3[pd.notnull(df3["code"])]
                
                redata2=redata[redata["NotebookName"]==notebook]
                redata2=redata2.reset_index(drop=True)
                code_mix="\n".join(list(df3["code"]))
                
#                 print("user資料:", len(df3))
#                 print("re項目數：",len(redata2))
#                 print("===codemix===")
#                 print(code_mix)
#                 print("============")
                
                for i in range(len(redata2)):
                    a=redata2.loc[i,"KeypointNo"]
                    cursor=conn.cursor()
                    cursor.execute('''
                    select `KeypointCount`,`Catched`, `KeypointDone` 
                    from `edm`.`implementation` 
                    where `NotebookName` = '{}' and `Username` = '{}' and `KeypointNo`={};
                    '''.format(notebook, user, a))
                    y=cursor.fetchall()
                    cursor.close()
                    
                    # done, catched_plus, count_plus
                    count=y[0][0]
                    if pd.isnull(count): count=0
                    catched=y[0][1]
                    if pd.isnull(catched): catched=""
                    done=y[0][2]
                    if pd.isnull(done): done=0
                    catched_plus=""
                    x=re.findall(redata2.loc[i,"KeypointRE"], code_mix)
                    count_plus=len(x)
                    if x !=[]:
                        done=1
                        catched_plus="\n".join(x)
                    if pd.isnull(count):
                        count=count_plus
                    else:
                        count+=count_plus
                    if pd.isnull(catched):
                        catched=catched_plus
                    else:
                        catched+=catched_plus    
                    if catched_plus!="":
                        print("user:",user,"    target:",redata2.loc[i,"KeypointRE"], "\n    本次完成:", done, "    計次:",count, "    結果:",catched_plus)
                        print("="*50)
                        
                    cursor=conn.cursor()
                    catched=catched.replace("\'","\"")
                    row=cursor.execute('''
                    UPDATE `edm`.`implementation` 
                    SET `KeypointDone` = {} , `KeypointCount` = {}, `Catched` = '{}'
                    WHERE `NotebookName`='{}' and `Username`='{}' and `KeypointNo`={};
                    '''.format(done, count, catched, notebook, user, a))
                    conn.commit()
                    cursor.close()
                    
        print("finish: implementation renew" )
        print("="*50)

        with open("lastindex.txt","w",encoding="utf-8") as x:
            x.write(str(ind2))
            print("reset index:", ind2, "\n")
    conn.close()
    
    time.sleep(10)
    if datetime.now() > datetime(2021,1,13,23,59,59,0):
        conn.close()
        break

In [ ]:
# conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

# cursor=conn.cursor()
# cursor.execute("select `Username` from `edm`.`member` ;") #where `Role`='學生';")
# u=cursor.fetchall()
# cursor.close()
# userlist=[]
# for i in u:
#     userlist.append(i[0]) 

# for user in userlist[-3:]:
#     print("=====",user,"=====")
#     cursor=conn.cursor()
#     cursor.execute("select `notebookName` from `edm`.`edm_log` where `Username`='{}';".format(user))
#     n=cursor.fetchall()
#     cursor.close()
#     nblist=[]
#     for i in n:
#         nblist.append(i[0])
#     nblist=list(set(nblist))
#     for nb in nblist:
#         print(nb)
#         sql='''select `log_id`, `Remark`, `RemarkLength` from `edm`.`edm_log` 
#         where `notebookName`='{}' and `Username`='{}';'''.format(nb, user)
#         cursor=conn.cursor()
#         cursor.execute(sql)
#         d=cursor.fetchall()
#         cursor.close()
#         for i in range(len(d)):
#             rmk=len(d[i][1])
#             if i==0:
#                 rmk_len_renew=rmk
#             elif i!=0:
#                 rmk_len_renew+=rmk
#             sql='''UPDATE `edm`.`edm_log` 
#             SET `RemarkLength`={}  
#             WHERE (`log_id` = '{}');'''.format(rmk_len_renew, d[i][0])
#             cursor=conn.cursor()
#             row=cursor.execute(sql)
#             conn.commit()
#             cursor.close()

### fix

In [ ]:
sql='''
UPDATE `edm`.`edm_log` 
SET `NotebookName` = '【Prac08】 Real-time Mask Inventory Data.ipynb' 
WHERE (`Username`='109522011' and `NotebookName` like "%Prac08%");
'''
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
cursor=conn.cursor()
row=cursor.execute(sql)
conn.commit()
cursor.close()

## error_data

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
cursor=conn.cursor()
row=cursor.execute('''select `Username`,`notebookName`,`kernelId`,`cellId`,`errorName`,`errorValue`,`timestamp` 
                    from `edm`.`edm_log` 
                    where `event` = 'code_execution';''')
err=cursor.fetchall()
cursor.close()

In [ ]:
Username=[]
NotebookName=[]
KernelId=[]
CellId=[]
ErrorName=[]
ErrorValue=[]
Timestamp=[]

for e in err:
    Username.append(e[0])
    NotebookName.append(e[1])
    KernelId.append(e[2])
    CellId.append(e[3])
    ErrorName.append(e[4])
    ErrorValue.append(e[5])
    Timestamp.append(e[6])

df=pd.DataFrame({
    "Username":Username,
    "NotebookName":NotebookName,
    "KernelId":KernelId,
    "CellId":CellId,
    "ErrorName":ErrorName,
    "ErrorValue":ErrorValue,
    "Timestamp":Timestamp})

In [ ]:
df=df.fillna("")

In [ ]:
df.ErrorValue=list(map(lambda x: x.split("(<")[0], df.ErrorValue))
df.Timestamp=list(map(lambda x: datetime.strptime(":".join(x.split(":")[:-1]), "%Y-%m-%d-%H:%M:%S"), df.Timestamp))

In [ ]:
df=df.sort_values("Timestamp")

In [ ]:
df.head()

In [ ]:
Username1=[]
NotebookName1=[]
KernelId1=[]
CellId1=[]
ErrorName1=[]
ErrorValue1=[]
Timestamp1=[]
Timespent=[]
Trycount=[]
Solved=[]

Username_list=list(set(df["Username"]))
for user in Username_list:
    print("==========="+user+"===========")
    df1=df[df["Username"]==user]
    NotebookName_list=list(set(df1["NotebookName"]))
    for nb in NotebookName_list:
        df2=df1[df1["NotebookName"]==nb]
        KernelId_list=list(set(df2["KernelId"]))
        for knl in KernelId_list:
            print("\\"+nb+"\\")
            df3=df2[df2["KernelId"]==knl]
            CellId_list=list(set(df3["CellId"]))
            for cell in CellId_list:
                df4=df3[df3["CellId"]==cell]
                df4=df4.reset_index(drop=True)
                
                ErrorName_pre=""
                ErrorValue_pre=""
                timespent=timedelta(0)
                time_pre=df4.loc[0, "Timestamp"] 
                try_count=0
                solved=0
                for i in range(len(df4)):
                    ErrorName_now=df4.loc[i, "ErrorName"]
                    ErrorValue_now=df4.loc[i, "ErrorValue"]
                    time_now=df4.loc[i, "Timestamp"]    
                    
                   
                    if ErrorName_now==ErrorName_pre and ErrorValue_now==ErrorValue_pre:
                        timespent+=(time_now-time_pre)
                        try_count+=1
                        if i==len(df4)-1:
                            print(">>"+ErrorName_pre+"【"+ErrorValue_pre+"】 Solved:"+str(solved)+" Try:"+str(try_count))
                            print("      @"+str(time_now)+" ["+str(timespent.seconds)+"]")
                            Username1.append(user)
                            NotebookName1.append(nb)
                            KernelId1.append(knl)
                            CellId1.append(cell)
                            ErrorName1.append(ErrorName_pre)
                            ErrorValue1.append(ErrorValue_pre)
                            Timestamp1.append(time_now)
                            Timespent.append(timespent.seconds)
                            Trycount.append(try_count)
                            Solved.append(solved)                            
                    elif ErrorName_pre!="" and ErrorValue_now!=ErrorValue_pre:
                        timespent+=(time_now-time_pre)
                        try_count+=1
                        solved=1
                        print(">>"+ErrorName_pre+"【"+ErrorValue_pre+"】 Solved:"+str(solved)+" Try:"+str(try_count))
                        print("      @"+str(time_now)+" ["+str(timespent.seconds)+"]")
                        Username1.append(user)
                        NotebookName1.append(nb)
                        KernelId1.append(knl)
                        CellId1.append(cell)
                        ErrorName1.append(ErrorName_pre)
                        ErrorValue1.append(ErrorValue_pre)
                        Timestamp1.append(time_now)
                        Timespent.append(timespent.seconds)
                        Trycount.append(try_count)
                        Solved.append(solved)

                        timespent=timedelta(0)
                        try_count=0
                        solved=0
                        
                    ErrorName_pre=copy.deepcopy(ErrorName_now)
                    ErrorValue_pre=copy.deepcopy(ErrorValue_now)
                    time_pre=copy.deepcopy(time_now)

dfx=pd.DataFrame({
    "Username":Username1,
    "NotebookName":NotebookName1,
    "KernelId":KernelId1,
    "CellId":CellId1,
    "ErrorName":ErrorName1,
    "ErrorValue":ErrorValue1,
    "Timestamp":Timestamp1,
    "Timespent":Timespent,
    "Trycount":Trycount,
    "Solved":Solved})

In [ ]:
dfx=dfx[dfx["ErrorName"]!=""]
dfx=dfx.reset_index(drop=True)
dfx.head(100)

In [ ]:
#df[(df["Username"]=="hch") & (df["NotebookName"]=="書籍分群.ipynb")]

In [ ]:
#dfx[(dfx["Username"]=="hch") & (dfx["NotebookName"]=="書籍分群.ipynb")]

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
cursor=conn.cursor()
row=cursor.execute('''select distinct`WeekNumber`,`Date` from `edm`.`section` where `Date` is not Null order by `WeekNumber`;''')
s=cursor.fetchall()
cursor.close()

In [ ]:
s

In [ ]:
weeknumber=[]
for i in range(len(dfx)):
    week=0
    for j,k in s:
        if dfx.loc[i,"Timestamp"]>k:
            week=j
    weeknumber.append(week)
dfx["WeekNumber"]=weeknumber

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
cursor=conn.cursor()
row=cursor.execute('''select `SectionNumber`,`NotebookName` from `edm`.`section` where `NotebookName` like "【%";''')
n=cursor.fetchall()
cursor.close()

In [ ]:
n

In [ ]:
sectionnumber=[]
for i in range(len(dfx)):
    sn=0
    for j,k in n:
        if dfx.loc[i,"NotebookName"]==k:
            sn=j
    sectionnumber.append(sn)
dfx["SectionNumber"]=sectionnumber

In [ ]:
dfx=dfx[dfx["Username"]!=""].reset_index(drop=True)
dfx.head()

In [ ]:
dfx.to_excel("error_data.xlsx")

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
for i in range(len(dfx)):
    a="`,`".join(list(dfx.columns))
    b=dfx.iloc[i,0]
    c=dfx.iloc[i,1]
    d=dfx.iloc[i,2]
    e=dfx.iloc[i,3]
    f=dfx.iloc[i,4]
    g=dfx.iloc[i,5].replace("'",'"')
    h=dfx.iloc[i,6]
    j=dfx.iloc[i,7]
    k=dfx.iloc[i,8]
    l=dfx.iloc[i,9]
    m=dfx.iloc[i,10]
    n=dfx.iloc[i,11]
    
    sql='''INSERT INTO `edm`.`error` (`{}`) VALUES ('{}','{}','{}','{}','{}','{}','{}',{},{},{},{},{});'''.format(a,b,c,d,e,f,g,h,j,k,l,m,n)
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

# renew_implementation_by_notebook

In [ ]:
import time
import datetime
import pymysql
import numpy as np
import pandas as pd
import re
import os
import copy
from datetime import datetime, timedelta
# from IPython import display
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")


# while True:
#     print(str(datetime.now()))

##### userlist #####
cursor=conn.cursor()
cursor.execute("select `Username` from `edm`.`member` ;") #where `Role`='學生';")
u=cursor.fetchall()
cursor.close()
userlist=[]
for i in u:
    userlist.append(i[0]) 

notebook_now=input("input the target notebook name:")
##### data #####
cursor=conn.cursor()
cursor.execute("select * from edm_log where `notebookName`='{}'".format(notebook_now))
data=cursor.fetchall()
cursor.close()


print("renewing notebook:", notebook_now)
title=['log_id', 'user_id', 'code', 'event', 'isError', 'kernelId', 'notebookName', 'notebookPath', 'cellId', 'timestamp', 'text', 'codeLength', 'errorName', 'errorValue','Remark','Cleancode','Username']
df=pd.DataFrame()
for i in range(len(title)):
    li=[]
    for j in data:
        li.append(j[i])
    df[title[i]]=li
print("本次資料：",len(df))

##### implementation #####
redata=pd.read_excel("python_re.xlsx", sheet_name="全")
redata=redata[redata["NotebookName"]==notebook_now]
redata=redata.reset_index(drop=True)


for user in userlist:
    print("==========renewing implementation:", user)
    df3=df[df["Username"]==user]
    df3=df3[pd.notnull(df3["code"])]
    code_mix="\n".join(list(df3["code"]))
    #print("user資料:", len(df3))
    #print("===codemix===")
    #print(code_mix)
    #print("============")
    for i in range(len(redata)):
        a=redata.loc[i,"KeypointNo"]
        # done, catched_plus, count_plus
        x=re.findall(redata.loc[i,"KeypointRE"], code_mix)
        count=len(x)
        done=0
        if x!=[]:
            done=1
        catched="\n".join(x)    
        print("user:",user,"    target:",redata.loc[i,"KeypointRE"], "\n    本次完成:", done, "    計次:",count, "    結果:",catched)
        
        cursor=conn.cursor()
        catched=catched.replace("\'","\"")
        row=cursor.execute('''
        UPDATE `edm`.`implementation` 
        SET `KeypointDone` = {} , `KeypointCount` = {}, `Catched` = '{}'  WHERE `NotebookName`='{}' and `Username`='{}' and `KeypointNo`={};
        '''.format(done, count, catched, notebook_now, user, a))
        conn.commit()
        cursor.close()
print("finish: implementation renew" )
print("="*50)

#     with open("lastindex.txt","w",encoding="utf-8") as x:
#         x.write(str(ind))
#     time.sleep(10)
    
#     if datetime.now() > datetime(2021,1,13,23,59,59,0):
#         conn.close()
#         break


# prepare database

## member data

In [ ]:
md=pd.read_excel("member.xlsx")
md.columns=md.iloc[0,:]
md=md.drop(0,axis=0)
md=md.reset_index(drop=True)
md=md[["姓名","帳號"]]
md["帳號"]=pd.Series(list(md.loc[:,"帳號"]), dtype="int64")

qn=pd.read_excel("pre-questionnaire.xlsx",sheet_name="clean")
qnx=qn[["帳號","姓名","單位","Gender","degree","Grade","codename"]]

member=pd.merge(md, qnx, on=["帳號","姓名"])
member

In [ ]:
# #清空表格
# cursor=conn.cursor()
# row=cursor.execute("TRUNCATE TABLE edm.member;")
# conn.commit()
# cursor.close()

In [ ]:
for i in range(len(member)):
    a=member.loc[i,"codename"]
    b="學生"
    c=member.loc[i,"帳號"]
    d=member.loc[i,"單位"]
    e=member.loc[i,"degree"]
    f=member.loc[i,"姓名"]
    g=member.loc[i,"Grade"]
    h=member.loc[i,"Gender"]    
    cursor=conn.cursor()
    row=cursor.execute('''
    INSERT INTO `edm`.`member` (`Name`, `Role`, `Username`,`Department`,`Degree`,`Truename`,`Grade`,`Gender`) 
    VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}');
    '''.format(a,b,c,d,e,f,g,h))
    conn.commit()
    cursor.close()

### group

In [ ]:
import pymysql
import re
import os
import copy
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from datetime import datetime, timedelta
import re
import requests
import os
from urllib.request import urlretrieve
#from urllib.parse import urlparse
from urllib.request import urlopen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

main="https://ncueeclass.ncu.edu.tw/course/teamStage/4599"
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': ".\\downloaded"}
options.add_experimental_option('prefs', prefs)

u="m946108005@tmu.edu.tw"
p="tjp3tjp3"

driver=webdriver.Chrome("D:/programming_useful_material/chromedriver.exe",chrome_options=options)
driver.get(main)
#time.sleep(2)
element_login=driver.find_element_by_link_text('Login')
element_login.click()

element_account=driver.find_element_by_xpath('//*[@id="account"]/div/input')
element_account.send_keys(u)
element_password=driver.find_element_by_xpath('//*[@id="password"]/div/div[1]/input')
element_password.send_keys(p)
element_send=driver.find_element_by_xpath('//*[@id="login_form"]/div[6]/div/button')
element_send.click()

#####group#####
time.sleep(1)
element_group=driver.find_element_by_link_text('小組專區')
element_group.click()

html=driver.page_source
soup=BeautifulSoup(html, "html.parser")
trs=soup.select("table#teamStageListTable tbody tr")
for tr in trs:
    stds=""
    grp=int(tr.select("td")[1].text.strip().replace(" ","")[1])
    stds+=tr.select("td")[2].text+", "+tr.select("td")[3].text
    stds=stds.split(", ")
    for std in stds:
        print(std, grp)
        
        cursor=conn.cursor()
        cursor.execute("select `Name`,`Username` from `edm`.`member` where `Truename`='{}';".format(std)) 
        section=cursor.fetchall()
        cursor.close()
        if section!=():
            na=section[0][0]
            usna=section[0][1]
        else:
            for i in range(len(std)):
                x=list(std)
                x[i]="?"
                stdtry="".join(x)
                cursor=conn.cursor()
                cursor.execute("select `Name`,`Username` from `edm`.`member` where `Truename`='{}';".format(stdtry)) 
                section=cursor.fetchall()
                cursor.close()
                if section!=():
                    na=section[0][0]
                    usna=section[0][1]
                    break

        sql='''UPDATE `edm`.`member` SET `Group` = {} WHERE (`Username` = '{}' and `Name`='{}');'''.format(grp, usna, na)
        cursor=conn.cursor()
        row=cursor.execute(sql)
        conn.commit()
        cursor.close()
driver.close()

### 檢驗點成績

In [ ]:
#quiz1 & quiz2

conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
sql="select `Username`, `SectionName`, `SectionScore` from `edm`.`member_class` where `SectionType`='quiz';"
cursor=conn.cursor()
cursor.execute(sql)
p=cursor.fetchall()
cursor.close()

In [ ]:
Username=[]
SectionName=[]
SectionScore=[]
for i,j,k in p:
    Username.append(i)
    SectionName.append(j)
    SectionScore.append(k)
df=pd.DataFrame({"Username":Username,"SectionName":SectionName,"SectionScore":SectionScore})

In [ ]:
df1=df[df["SectionName"]=="【Quiz06】Quiz (1)"]
df2=df[df["SectionName"]=="【Quiz12】Quiz(2)"]

In [ ]:
for i in df1.index:
    a=df1.loc[i,"Username"]
    b=df1.loc[i,"SectionScore"]
    sql='''UPDATE  `edm`.`member` SET `quiz1` = {} WHERE (`Username`='{}');'''.format(b, a)
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

In [ ]:
for i in df2.index:
    a=df2.loc[i,"Username"]
    b=df2.loc[i,"SectionScore"]
    sql='''UPDATE  `edm`.`member` SET `quiz2` = {} WHERE (`Username`='{}');'''.format(b, a)
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

In [ ]:
#datathon

conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
sql="select `Username`, `SectionScore` from `edm`.`member_class` where `SectionName`='【Pract16】Datathon';"
cursor=conn.cursor()
cursor.execute(sql)
p=cursor.fetchall()
cursor.close()

In [ ]:
for i,j in p:
    sql='''UPDATE  `edm`.`member` SET `datathon` = {} WHERE (`Username`='{}');'''.format(j, i)
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

In [ ]:
#datathon_accomplish

data=pd.read_excel("practice_score.xlsx")
data=data[data["SectionName"]=="【Pract16】Datathon"].reset_index(drop=True)
data.head()

In [ ]:
data1=data[["StudentId","accomplish"]].groupby("StudentId").sum()
data1["accomplish"]=(data1["accomplish"]/8) *100
data1.head()

In [ ]:
for i in data1.index:
    sql='''UPDATE  `edm`.`member` SET `datathon_accomplish` = {} WHERE (`Username`='{}');'''.format(data1.loc[i,"accomplish"], i)
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

## pre-questionnaire

In [ ]:
qn=pd.read_excel("pre-questionnaire.xlsx",sheet_name="clean", encoding="utf-8")

In [ ]:
li=list(qn.columns)
for i in ["filling_tm","dpt","gender","degree","grade","name"]:
    li.remove(i)
qn=qn[li]

In [ ]:
#清空表格
cursor=conn.cursor()
row=cursor.execute("TRUNCATE TABLE `edm`.`pre-q`;")
conn.commit()
cursor.close()

In [ ]:
for i in range(len(qn)):
    print(i)
    x=list(qn.loc[i,:])
    xx=[]
    ll=[]
    for j in range(len(x)):
        if pd.notnull(x[j]):
            ll.append(li[j])
            xx.append(x[j])
    xx=tuple(xx)
    ll="`,`".join(ll)
    sql="INSERT INTO `edm`.`pre-q` (`%s`) VALUES "%ll+str(xx)
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

## After-questionnaire

In [ ]:
qn=pd.read_excel("after-questionnaire.xlsx",sheet_name="clean", encoding="utf-8")
qn

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
li=list(qn.columns)
for i in range(len(qn)):
    print(i)
    x=list(qn.loc[i,:])
    xx=[]
    ll=[]
    for j in range(len(x)):
        if pd.notnull(x[j]):
            ll.append(li[j])
            xx.append(x[j])
    xx=tuple(xx)
    ll="`,`".join(ll)
    sql="INSERT INTO `edm`.`after-q` (`%s`) VALUES "%ll+str(xx)
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

In [ ]:
# #清空表格
# cursor=conn.cursor()
# row=cursor.execute("TRUNCATE TABLE `edm`.`after-q`;")
# conn.commit()
# cursor.close()

## implementation data

In [ ]:
# 清空表格
# cursor=conn.cursor()
# row=cursor.execute("TRUNCATE TABLE `edm`.`implementation`;")
# conn.commit()
# cursor.close()

In [ ]:
# 清空section
# cursor=conn.cursor()
# row=cursor.execute("DELETE FROM `edm`.`implementation` WHERE `SectionNumber` = 22;")
# conn.commit()
# cursor.close()

In [ ]:
#修改資料
# cursor=conn.cursor()
# row=cursor.execute("UPDATE `edm`.`implementation`  SET `SectionNumber` =18 WHERE (`SectionName` like '%Week12%');")
# conn.commit()
# cursor.close()

### by section

In [ ]:
cursor=conn.cursor()
cursor.execute("select `Username` from `edm`.`member`;")
u=cursor.fetchall()
cursor.close()
userlist=[]
for i in u:
    userlist.append(i[0]) 


kp=pd.read_excel("python_re.xlsx", sheet_name="全")
n=int(input("section number:"))
kp=kp[kp["SectionNumber"]==n]

mkp=pd.DataFrame()
for u in userlist:
    pre_mkp=copy.deepcopy(kp)
    pre_mkp["Username"]=[u]*len(pre_mkp)
    mkp=pd.concat([mkp,pre_mkp], axis=0, ignore_index=True)

##mkp["NotebookName"]=["Untitled.ipynb"]*len(mkp)

for i in range(len(mkp)):
    a=mkp.loc[i,"Username"]
    b=mkp.loc[i,"SectionNumber"]
    c=mkp.loc[i,"SectionName"]
    d=mkp.loc[i,"NotebookName"]
    e=mkp.loc[i,"KeypointName"].replace("\'","\"")
    f=mkp.loc[i,"KeypointExample"].replace("\'","\"")
    g=mkp.loc[i,"KeypointNo"]
    print("=============================")
    print(i)
    print(a,"\n",b,"\n",c,"\n",d,"\n",e,"\n",f,"\n",g)
    cursor=conn.cursor()
    row=cursor.execute('''INSERT INTO `edm`.`implementation` 
    (`Username`, `SectionNumber`, `SectionName`,`NotebookName`,`KeypointName`,`KeypointExample`,`KeypointNo`) 
    VALUES ('{}', {}, '{}', '{}', '{}', '{}', '{}');'''.format(a,b,c,d,e,f,g))
    conn.commit()
    cursor.close()

### by section & user

In [ ]:
# cursor=conn.cursor()
# cursor.execute("select `Username` from `edm`.`member`;")
# u=cursor.fetchall()
# cursor.close()
# userlist=[]
# for i in u:
#     userlist.append(i[0]) 
user=input("請輸入使用者帳號：")

kp=pd.read_excel("python_re.xlsx", sheet_name="全")
n=int(input("section number:"))
kp=kp[kp["SectionNumber"]==n]

mkp=copy.deepcopy(kp)
mkp["Username"]=[user]*len(mkp)
mkp=mkp.reset_index(drop=True)
##mkp["NotebookName"]=["Untitled.ipynb"]*len(mkp)

for i in range(len(mkp)):
    a=mkp.loc[i,"Username"]
    b=mkp.loc[i,"SectionNumber"]
    c=mkp.loc[i,"SectionName"]
    d=mkp.loc[i,"NotebookName"]
    e=mkp.loc[i,"KeypointName"]
    f=mkp.loc[i,"KeypointExample"].replace("\'","\"")
    g=mkp.loc[i,"KeypointNo"]
    print("=============================")
    print(i)
    print(a,"\n",b,"\n",c,"\n",d,"\n",e,"\n",f,"\n",g)
    cursor=conn.cursor()
    row=cursor.execute('''INSERT INTO `edm`.`implementation` 
    (`Username`, `SectionNumber`, `SectionName`,`NotebookName`,`KeypointName`,`KeypointExample`,`KeypointNo`) 
    VALUES ('{}', {}, '{}', '{}', '{}', '{}', '{}');'''.format(a,b,c,d,e,f,g))
    conn.commit()
    cursor.close()

## section data

### renew section

In [ ]:
#清空表格
cursor=conn.cursor()
row=cursor.execute("TRUNCATE TABLE edm.section;")
conn.commit()
cursor.close()

In [ ]:
sec=pd.read_excel("section.xlsx", sheet_name="1")
sec

In [ ]:
li=list(sec.columns)
for i in range(len(sec)):
    x=list(sec.loc[i,:])
    xx=[]
    ll=[]
    for j in range(len(x)):
        if pd.notnull(x[j]):
            ll.append(li[j])
            xx.append(x[j])
    xx=tuple(xx)
    ll="`,`".join(ll)
    sql="INSERT INTO `edm`.`section` (`%s`) VALUES "%ll+str(xx)
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

### section initial content

In [ ]:
import pymysql
import numpy as np
import pandas as pd
import re
import os
import copy
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
from urllib.request import urlretrieve
#from urllib.parse import urlparse
from urllib.request import urlopen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#字數不算"\n"
#html_list=os.listdir("./original_html")
html_list=[input("新html檔案名稱")]

for i in range(len(html_list)):
    html=open("./original_html/"+html_list[i], encoding="utf-8")
    x=html.read()
    html.close()
    x=BeautifulSoup(x, "html.parser")
    xx=x.select("div#notebook-container")[0].select("div.inner_cell")
    code=""
    remark=""
    for j in range(len(xx)):
        xx[j]=xx[j].text.replace("\n\n", "").strip("\n")
    while "\n" in xx:
        xx.remove("\n")
    #print(xx)
    
    #section_name
    section_name=html_list[i].split(".html")[0]
    print(section_name)
    
    #InitialContent 
    initial_content="\n".join(xx).replace("'",'"')
    
    #InitialContent_RmCt
    md=""
    for j in xx:
        if "¶" in j:
            md+=j
    rm=re.findall("#.*[\n]?", initial_content)
    initial_content_rmct=len("".join(rm).replace("\n",""))+len(md.replace("\n",""))
#     print(md)
#     print(rm)
    print(initial_content_rmct)

    #InitialContent_CdCt
    initial_content_cdct=len(initial_content.replace("\n",""))-initial_content_rmct
    print(initial_content_cdct)
    
    sql='''UPDATE `edm`.`section` 
    SET `InitialContent`='{}', `InitialContent_CdCt`='{}',`InitialContent_RmCt`='{}' 
    WHERE (`SectionName`='{}');'''.format(initial_content, initial_content_cdct, initial_content_rmct, section_name)
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()
    print("======================================")
    

## practice data

In [ ]:
import pymysql
import numpy as np
import pandas as pd
import re
import os
import copy
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
from urllib.request import urlretrieve
#from urllib.parse import urlparse
from urllib.request import urlopen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
html_list=os.listdir("./original_html")
for i in range(len(html_list)):
    html=open("./original_html/"+html_list[i], encoding="utf-8")
    x=html.read()
    html.close()
    x=BeautifulSoup(x, "html.parser")
    xx=x.select("div#notebook-container")[0].select("div.inner_cell")
    for j in range(len(xx)): 
        xx[j]=xx[j].text.replace("'",'"').strip()
        if xx[j]=="\n": xx[j]=""
        if "!!! please DO NOT rename this file !!!" in xx[j]: xx[j]=""
        if "student id:\nname" in xx[j]: xx[j]=""
    while "" in xx: xx.remove("")

    section_name=html_list[i].split(".html")[0]
    print(section_name)
    
    sql="select `WeekNumber`, `SectionNumber` from `edm`.`section` where `SectionName`='{}';".format(section_name)
    cursor=conn.cursor()
    cursor.execute(sql)
    data=cursor.fetchall()
    cursor.close()
    week_number=data[0][0]
    section_number=data[0][1]
    
    if "Prac" in html_list[i] and "Prac08" not in section_name:
        for j in range(len(xx)):
            print(xx[j])
            print("===========")
            
            sql='''INSERT INTO `edm`.`practice` 
            (`WeekNumber`, `SectionNumber`, `SectionName`,`PracticeQuestion`) 
            VALUES ({}, {}, '{}', '{}');'''.format(week_number,section_number,section_name,xx[j])
            cursor=conn.cursor()
            row=cursor.execute(sql)
            conn.commit()
            cursor.close()
        print("========================================")
        
    elif "Prac" in html_list[i] and "Prac08" in section_name:
        for j in range(len(xx)):
            if "¶" in xx[j]:  
                print(xx[j])
                print("===========")
                sql='''INSERT INTO `edm`.`practice` 
                (`WeekNumber`, `SectionNumber`, `SectionName`,`PracticeQuestion`) 
                VALUES ({}, {}, '{}', '{}');'''.format(week_number,section_number,section_name,xx[j])
                cursor=conn.cursor()
                row=cursor.execute(sql)
                conn.commit()
                cursor.close()            
        print("========================================")

In [ ]:
#僅更新一份練習
this=input("請輸入html檔案全名")
path="./original_html/"+this

if this.startswith("【Prac"):
    html=open(path, encoding="utf-8")
    x=html.read()
    html.close()
    x=BeautifulSoup(x, "html.parser")
    xx=x.select("div#notebook-container")[0].select("div.inner_cell")
    xxx=[]
    for j in range(len(xx)): 
        xx[j]=xx[j].text.replace("'",'"').strip()
        if xx[j]=="\n": xx[j]=""
        if "!!! please DO NOT rename this file !!!" in xx[j]: xx[j]=""
        if "student id" in xx[j].lower(): xx[j]=""
    for j in range(len(xx)):
        if "¶" in xx[j]:         
            xxx.append(xx[j])        
            
elif this.startswith("【Week"):
    html=open(path, encoding="utf-8")
    x=html.read()
    html.close()
    x=BeautifulSoup(x, "html.parser")
    xx=x.select("div#notebook-container")[0].select("div.inner_cell")
    for j in range(len(xx)): 
        if "【Practice】" in xx[j].text:
            key=j
    xx=xx[key+1:]
    xxx=[]
    for j in range(len(xx)): 
        xx[j]=xx[j].text.replace("'",'"').strip()
        if "reference and reading" in xx[j]: xx[j]=""   
        if "¶" in xx[j]:         
            xxx.append(xx[j])        
        
section_name=this.split(".html")[0]
print(section_name)

sql="select `WeekNumber`, `SectionNumber` from `edm`.`section` where `SectionName`='{}';".format(section_name)
cursor=conn.cursor()
cursor.execute(sql)
data=cursor.fetchall()
cursor.close()
week_number=data[0][0]
section_number=data[0][1]

for j in range(len(xxx)):
    print(xxx[j])
    print("===========")

    sql='''INSERT INTO `edm`.`practice` 
    (`WeekNumber`, `SectionNumber`, `SectionName`,`PracticeQuestion`) 
    VALUES ({}, {}, '{}', '{}');'''.format(week_number,section_number,section_name,xxx[j])
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()
print("========================================")

### practice_score

In [ ]:
#製作打分數表格

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
sql="select `SectionName`, `PracticeQuestion`, `practice_id` from `edm`.`practice`;"
cursor=conn.cursor()
cursor.execute(sql)
p=cursor.fetchall()
cursor.close()

In [ ]:
m=pd.read_excel("member.xlsx")
m.columns=m.iloc[0,:]
m=m.drop(0, axis=0)
Name=list(m["姓名"])
StudentId=list(m["帳號"])

In [ ]:
SectionName=[]
PracticeQuestion=[]
practice_id=[]
for i, j, k in p:
    SectionName.append(i)
    PracticeQuestion.append(j)
    practice_id.append(k)

In [ ]:
df=pd.DataFrame()
for i in range(len(Name)):
    dfx=pd.DataFrame({"SectionName":SectionName, "PracticeQuestion":PracticeQuestion, "practice_id":practice_id})
    dfx["Name"]=[Name[i]]*len(SectionName)
    dfx["StudentId"]=[StudentId[i]]*len(SectionName)
    df=pd.concat([df, dfx], axis=0)

In [ ]:
df=df[["SectionName","Name","StudentId","PracticeQuestion","practice_id"]]
df=df.sort_values(["SectionName","StudentId","practice_id"])
df.to_excel("practice_score.xlsx")

In [ ]:
#依practice_id輸入分數

In [ ]:
data=pd.read_excel("practice_score.xlsx")
data=data[data["SectionName"]=="【Pract16】Datathon"].reset_index(drop=True)
data.head()

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
for i in range(len(data)):
    a=data.loc[i,"StudentId"]
    b=data.loc[i,"practice_id"]
    c=float(data.loc[i,"accomplish"])
    if pd.isnull(c):
        c=0

    sql='''UPDATE  `edm`.`member_practice` SET `achievement` = {} WHERE (`practice_id`={} and `Username`='{}');'''.format(c, b, a)
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

In [ ]:
# #清空
# cursor=conn.cursor()
# row=cursor.execute("TRUNCATE TABLE edm.practice;")
# conn.commit()
# cursor.close()

In [ ]:
#刪除資料
# cursor=conn.cursor()
# row=cursor.execute("DELETE FROM `edm`.`practice` WHERE (`SectionName` = '【Pract16】Datathon');")
# conn.commit()
# cursor.close()

## member_pratice data

In [ ]:
import pymysql
import numpy as np
import pandas as pd
import re
import os
import copy
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
from urllib.request import urlretrieve
#from urllib.parse import urlparse
from urllib.request import urlopen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# cursor=conn.cursor()
# cursor.execute("select `event`, `text`,`cellId` from `edm`.`edm_log` where `Username`='{}' and `notebookName`='{}';".format(usr,nb))
# data=cursor.fetchall()
# cursor.close()


# sql='''UPDATE `edm`.`member_class` 
# SET `ExternalPasteCt` = {}, `ExternalPasteLength` = {}, `ExternalPasteText` = '{}', `CopyCt` = {}, `CopyTextLength` = {}, `PasteCt` = {}, `PasteTextLength` = {}
# WHERE (`Username` ='{}' and `SectionName`='{}');
# '''.format(ExternalPasteCt, ExternalPasteLength, ExternalPasteText, CopyCt, CopyTextLength, PasteCt, PasteTextLength, usr, nb.split(".ipynb")[0])
# cursor=conn.cursor()
# row=cursor.execute(sql)
# conn.commit()
# cursor.close() 

### 整批作業資料整理

In [ ]:
cursor=conn.cursor()
cursor.execute("select distinct `SectionName` from `edm`.`practice`;")
prac_sections=cursor.fetchall()
cursor.close()
prac_sections=list(prac_sections)
for i in range(len(prac_sections)): prac_sections[i]=prac_sections[i][0]

Q=[]
QQ=[]
C=[]
O=[]
P=[]
S=[]

conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

hw_list=os.listdir("./homework")
for hw in hw_list:
    for p in prac_sections:
        if p.split("】")[0][-2:]+"】" in hw:
            std_list=os.listdir("./homework/"+hw)
            std_list.remove('index.html')
            
            cursor=conn.cursor()
            cursor.execute("select `practice_id`,`PracticeQuestion` from `edm`.`practice` where `SectionName`='{}';".format(p))
            qlist=cursor.fetchall()
            cursor.close()
            qlist=list(qlist)
            practice_id=[]
            for i in range(len(qlist)): 
                practice_id.append(qlist[i][0])
                qlist[i]=qlist[i][1]
            
            print("===============【",p,"】===============")
            
            for std in std_list:
                print("=====【",std,"】=====")
                week_assignments=os.listdir("./homework/"+hw+"/"+std+"/attach")
                for assignment in week_assignments:
                    if "【Prac" in assignment:
                        #print(assignment)
                        try:
                            html=open("./homework/"+hw+"/"+std+"/attach/"+assignment, encoding="utf-8")
                            x=html.read()
                            html.close()

                            x=BeautifulSoup(x, "html.parser")

                            xx=x.select("div#notebook-container")[0].select("div.inner_cell")
                            for i in range(len(xx)):
                                xx[i]=xx[i].text.replace("'",'"')
                                if xx[i]=="\n": xx[i]=""
                                if "please do not rename this file" in xx[i].lower(): xx[i]=""
                                if "student id" in xx[i].lower(): xx[i]=""
                            while "" in xx: xx.remove("")

                            content=""
                            contents=x.select("div#notebook-container")[0].select("div.cell.border-box-sizing")
                            for c in contents:
                                if c.select("div.inner_cell")!=[]:
                                    for inner in c.select("div.inner_cell"):
                                        content+=inner.text
                                        content+="\n"
                                if c.select("div.output")!=[]: 
                                    for output in c.select("div.output"):
                                        content+=output.text
                                        content+="\n"
                            content=content.replace("'",'"')
                            #print("="*20+"\n"+content)

                            code="".join(xx)
                            anscode=[]
                            for i in range(len(qlist)):
                                if i==0:
                                    code=code.split(qlist[i])[1]
                                if i!=0:
                                    anscode.append(code.split(qlist[i])[0].strip())
                                    code=code.split(qlist[i])[1]
                            anscode.append(code.strip())

                            inner_cell_without_q=[]
                            for xxx in xx:
                                if xxx.strip() not in qlist:
                                    inner_cell_without_q.append(xxx)
                            for wq in inner_cell_without_q:
                                content=content.replace(wq, "", 1)
                            #print("="*20+"\n"+content)

                            ansoutput=[]
                            for i in range(len(qlist)):
                                if i==0:
                                    content=content.split(qlist[i])[1]
                                if i!=0:
                                    ansoutput.append(content.split(qlist[i])[0].strip( ))
                                    content=content.split(qlist[i])[1]
                            ansoutput.append(content.strip( ))
    #                             for c in anscode:
    #                                 for i in range(len(ansoutput)):
    #                                     cc=c.split("\n\n\n\n\n")
    #                                     for ccc in cc:
    #                                         if ccc in ansoutput[i]:
    #                                             ansoutput[i]=ansoutput[i].replace(ccc, "")
    #                             ansoutput=list(map(lambda x: x.strip(), ansoutput))

                            if len(qlist)!=len(anscode) or len(anscode)!=len(ansoutput) or len(ansoutput)!=len(qlist):
                                print("!!!!!ERROR!!!!!")
                            else:
                                Q.extend(qlist)
                                QQ.extend(practice_id)
                                C.extend(anscode)
                                O.extend(ansoutput)
                                P.extend([assignment]*len(qlist))
                                S.extend([std]*len(qlist))
                        except:
                            print("==========!!!!!!!ERROR!!!!!!!==========")
                            print("【",std,"】")
                            print(assignment)
                            print("===============================")

for i in range(len(Q)):
    sql='''INSERT INTO `edm`.`member_practice`
    (`practice_id`, `Username`, `PracticeCode`,`PracticeOutput`) 
    VALUES ({}, '{}', '{}', '{}');'''.format(QQ[i], S[i][:9], C[i].replace("'",'"'), O[i].replace("'",'"'))
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()     

### 單次作業資料整理

In [ ]:
Q=[]
QQ=[]
C=[]
O=[]
P=[]
S=[]

conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

hw=input("請輸入作業資料夾名稱")
p=input("請輸入SectionName")

hw_list=os.listdir("./homework")
std_list=os.listdir("./homework/"+hw)
std_list.remove('index.html')

cursor=conn.cursor()
cursor.execute("select `practice_id`,`PracticeQuestion` from `edm`.`practice` where `SectionName`='{}';".format(p))
qlist=cursor.fetchall()
cursor.close()
qlist=list(qlist)
practice_id=[]
for i in range(len(qlist)): 
    practice_id.append(qlist[i][0])
    qlist[i]=qlist[i][1]

#print("===============【",p,"】===============")
            
for std in std_list:
    print("=====【",std,"】=====")
    week_assignments=os.listdir("./homework/"+hw+"/"+std+"/attach")
    for assignment in week_assignments:
        if "【Prac" in assignment:
            #print(assignment)
            try:
                html=open("./homework/"+hw+"/"+std+"/attach/"+assignment, encoding="utf-8")
                x=html.read()
                html.close()

                x=BeautifulSoup(x, "html.parser")
                xx=x.select("div#notebook-container")[0].select("div.inner_cell")
                for i in range(len(xx)):
                    xx[i]=xx[i].text.replace("'",'"')
                    if xx[i]=="\n": xx[i]=""
                    if "please do not rename this file" in xx[i].lower(): xx[i]=""
                    if "student id" in xx[i].lower(): xx[i]=""
                while "" in xx: xx.remove("")

                content=""
                contents=x.select("div#notebook-container")[0].select("div.cell.border-box-sizing")
                for c in contents:
                    if c.select("div.inner_cell")!=[]:
                        for inner in c.select("div.inner_cell"):
                            content+=inner.text
                            content+="\n"
                    if c.select("div.output")!=[]: 
                        for output in c.select("div.output"):
                            content+=output.text
                            content+="\n"
                content=content.replace("'",'"')

                code="".join(xx)
                anscode=[]
                for i in range(len(qlist)):
                    if i==0:
                        code=code.split(qlist[i])[1]
                    if i!=0:
                        anscode.append(code.split(qlist[i])[0].strip())
                        code=code.split(qlist[i])[1]
                anscode.append(code.strip())

                inner_cell_without_q=[]
                for xxx in xx:
                    if xxx.strip() not in qlist:
                        inner_cell_without_q.append(xxx)
                for wq in inner_cell_without_q:
                    content=content.replace(wq, "", 1)
                ansoutput=[]
                for i in range(len(qlist)):
                    if i==0:
                        content=content.split(qlist[i])[1]
                    if i!=0:
                        ansoutput.append(content.split(qlist[i])[0].strip( ))
                        content=content.split(qlist[i])[1]
                ansoutput.append(content.strip( ))

                if len(qlist)!=len(anscode) or len(anscode)!=len(ansoutput) or len(ansoutput)!=len(qlist):
                    print("!!!!!ERROR!!!!!")
                else:
                    Q.extend(qlist)
                    QQ.extend(practice_id)
                    C.extend(anscode)
                    O.extend(ansoutput)
                    P.extend([assignment]*len(qlist))
                    S.extend([std]*len(qlist))
            except:
                print("==========!!!!!!!ERROR!!!!!!!==========")
                print("【",std,"】")
                print(assignment)
print("preparement finished!!!")

for i in range(len(Q)):
    sql='''INSERT INTO `edm`.`member_practice`
    (`practice_id`, `Username`, `PracticeCode`,`PracticeOutput`) 
    VALUES ({}, '{}', '{}', '{}');'''.format(QQ[i], S[i][:9], C[i].replace("'",'"'), O[i].replace("'",'"'))
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

### 個別(人-作業)補齊

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

hw_list=os.listdir("./homework")
std=input("學號 (姓名):")
p=input("Practice assignment name: ")
pp=input("SectionName")
for h in hw_list:
    if p.split("】")[0][-2:]+"】" in h:
        hw=h

html=open("./homework/"+hw+"/"+std+"/attach/"+p, encoding="utf-8")
x=html.read()
html.close()
x=BeautifulSoup(x, "html.parser")
xx=x.select("div#notebook-container")[0].select("div.inner_cell")

qlist=[]
for i in xx:
    if "¶" in i.text:
        if "student id" not in i.text.lower() and "please DO NOT rename this file" not in i.text:
            #print(i.text)
            qlist.append(i.text.strip().replace("'",'"'))


# ####(同學誤刪標題時用)
# qlist=qlist[:-1]+["02. Using pie chart to analyze the data and explain your finding¶"]+qlist[-1:]
# ####

code=""
for i in range(len(xx)):
    xx[i]=xx[i].text
    if xx[i]=="\n": xx[i]=""
    if "please do not rename this file" in xx[i].lower(): xx[i]=""
    if "student id" in xx[i].lower(): xx[i]=""
while "" in xx: xx.remove("")    
#     if "【Pratice】" in xx[i] or "【Practice】" in xx[i]:
#         key=i

# ####(同學誤刪標題時用)
#     if "plt.pie(df01_sum.adult, labels=df01_sum.index" in xx[i]:
#         key2=i
# xx=xx[:key2]+["02. Using pie chart to analyze the data and explain your finding¶"]+xx[key2:]
# ####

# xx=xx[key+1:]

code="".join(xx).replace("'",'"')

content=""
contents=x.select("div#notebook-container")[0].select("div.cell.border-box-sizing")
for c in contents:
    if c.select("div.inner_cell")!=[]:
        for inner in c.select("div.inner_cell"):
            content+=inner.text
    if c.select("div.output")!=[]: 
        for output in c.select("div.output"):
            content+=output.text
content=content.replace("'",'"')

#print(content)
# ####(同學誤刪標題時用)
# key=content.index("from matplotlib.font_manager import FontProperties")
# content=content[:key]+"02. Using pie chart to analyze the data and explain your finding¶"+content[key:]
# ####

# if "【Pratice】" in content: content=content.split("【Pratice】")[1].replace("'",'"')
# if "【Practice】" in content: content=content.split("【Practice】")[1].replace("'",'"')

inner_cell_without_q=[]
for xxx in xx:
    if xxx.strip() not in qlist:
        inner_cell_without_q.append(xxx)
for wq in inner_cell_without_q:
    content=content.replace(wq, "", 1)

anscode=[]
for i in range(len(qlist)):
    if i==0:
        code=code.split(qlist[i])[1]
    if i!=0:
        anscode.append(code.split(qlist[i])[0].strip())
        code=code.split(qlist[i])[1]
anscode.append(code.strip())

ansoutput=[]
for i in range(len(qlist)):
    if i==0:
        content=content.split(qlist[i])[1]
        print(qlist[i])
    if i!=0:
        ansoutput.append(content.split(qlist[i])[0].strip( ))
        print("==========")
        print(content.split(qlist[i])[0].strip( ))
        print("====================")
        print(qlist[i])
        content=content.split(qlist[i])[1]
ansoutput.append(content.strip( ))

cursor=conn.cursor()
cursor.execute("select `practice_id`,`PracticeQuestion` from `edm`.`practice` where `SectionName` like '%{}%';".format(p[:9]))
qdata=cursor.fetchall()
cursor.close()
qdata=list(qdata)

if len(qlist)!=len(anscode) or len(anscode)!=len(ansoutput) or len(ansoutput)!=len(qlist):
    print("●●!!ERROR!!●●")
else:
    print("length of qlist:", len(qlist))
    
    for i in range(len(qlist)):
        for j in range(len(qdata)):
            if qlist[i][:10] in qdata[j][1]:
                practice_id=qdata[j][0]
                print(practice_id)
                print(qdata[j][1])
                print("=====")
                print(anscode[i])
                print("=====")
                print(ansoutput[i])
                
                try:
                    sql='''INSERT INTO `edm`.`member_practice`
                    (`practice_id`, `Username`, `PracticeCode`,`PracticeOutput`) 
                    VALUES ({}, '{}', '{}', '{}');'''.format(practice_id, std[:9], anscode[i].replace("'",'"'), ansoutput[i].replace("'",'"'))
                    cursor=conn.cursor()
                    row=cursor.execute(sql)
                    conn.commit()
                    cursor.close()     
                except:
                    sql='''UPDATE `edm`.`member_practice` 
                    SET `PracticeCode` = '{}', `PracticeOutput` = '{}' WHERE (`Username` ='{}' and `practice_id`={});
                    '''.format(anscode[i].replace("'",'"'), ansoutput[i].replace("'",'"'), std[:9], practice_id)
                    cursor=conn.cursor()
                    row=cursor.execute(sql)
                    conn.commit()
                    cursor.close()                         
                print("==============")

                #     print([assignment]*len(qlist))
#     print([std]*len(qlist))

### 清除資料

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
#刪除資料
cursor=conn.cursor()
row=cursor.execute("DELETE FROM `edm`.`member_practice` WHERE (`practice_id`<=5);")
conn.commit()
cursor.close()

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
#清空表格
cursor=conn.cursor()
row=cursor.execute("TRUNCATE TABLE edm.member_practice;")
conn.commit()
cursor.close()

## member-class data

In [ ]:
#清空表格
# cursor=conn.cursor()
# row=cursor.execute("TRUNCATE TABLE edm.member_class;")
# conn.commit()
# cursor.close()

In [ ]:
#修改表格
# cursor=conn.cursor()
# row=cursor.execute("UPDATE `edm`.`member_class` SET `SectionType` = 'practice' WHERE (`SectionName` = '【Pract16】Datathon');")
# conn.commit()
# cursor.close()

In [ ]:
md=pd.read_excel("member.xlsx")
md.columns=md.iloc[0,:]
md=md.drop(0,axis=0)
md=md.reset_index(drop=True)

mb=list(md["帳號"])

In [ ]:
sec=pd.read_excel("section.xlsx", sheet_name="1")
sec=sec[["SectionType","SectionName","SectionTimeStart","SectionNumber"]]

In [ ]:
mc_df=pd.DataFrame()
for i in mb:
    sec["Username"]=[i]*len(sec)
    mc_df=pd.concat([mc_df, sec], axis=0, ignore_index=True)

In [ ]:
mc_df.head()

In [ ]:
for i in range(len(mc_df)):
    x=mc_df.loc[i,:]
    li=mc_df.columns
    xx=[]
    ll=[]
    for j in range(len(x)):
        if pd.notnull(x[j]):
            ll.append(li[j])
            xx.append(x[j])
    ll="`,`".join(ll)
    xx=tuple(xx)
    cursor=conn.cursor()
    row=cursor.execute("INSERT INTO `edm`.`member_class` (`%s`) VALUES "%ll+str(xx))
    conn.commit()
    cursor.close()    

### renew(從下載檔案)

In [ ]:
import time
import datetime
import pymysql
import numpy as np
import pandas as pd
import re
import os
import copy
from datetime import datetime, timedelta

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

In [ ]:
conn.close()

####  research_participant_informed_consent submitted

In [ ]:
newdata=pd.read_excel("research_participant_informed_consent.xlsx")
newdata=newdata[newdata["Submitted"]=="V"]
newdata=newdata.reset_index(drop=True)

In [ ]:
newdata

In [ ]:
for i in range(len(newdata)):
    u=newdata.loc[i,"Username"]
    section="research participant informed consent"
    #sc=newdata.loc[i,"成績"]
    sb="已交"
    #sbt=newdata.loc[i,"filling_tm"]

    sql='''UPDATE `edm`.`member_class` 
    SET `Submitted`='{}' WHERE (`Username`={} and `SectionName`='{}');'''.format(sb, u, section)
    
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

#### pre-questionnaire submitted

In [ ]:
newdata=pd.read_excel("pre-questionnaire.xlsx")
#newdata.columns=newdata.loc[0,:]
#newdata=newdata.drop([0],axis=0)
#newdata=newdata.reset_index(drop=True)

In [ ]:
newdata

In [ ]:
for i in range(len(newdata)):
    u=newdata.loc[i,"id"]
    section="pre-queationnaire"
    #sc=newdata.loc[i,"成績"]
    sb="已交"
    sbt=newdata.loc[i,"filling_tm"]

    sql='''UPDATE `edm`.`member_class` 
    SET `SubmitTime`='{}', `Submitted`='{}'
    WHERE (`Username`={} and `SectionName`='{}');'''.format(sbt, sb, u, section)
    
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

#### pre-test submitted

In [ ]:
newdata=pd.read_excel("pre-test_memberscore.xlsx")
newdata.columns=newdata.loc[0,:]
newdata=newdata.drop([0],axis=0)
newdata=newdata.reset_index(drop=True)

In [ ]:
newdata.head()

In [ ]:
for i in range(len(newdata)):
    u=newdata.loc[i,"帳號"]
    section="pre-test"
    sc=newdata.loc[i,"成績"]
    sb="已交"
    sbt=newdata.loc[i,"交卷時間"]

    sql='''UPDATE `edm`.`member_class` 
    SET `SubmitTime`='{}', `Submitted`='{}', `SectionScore`='{}'
    WHERE (`Username`={} and `SectionName`='{}');'''.format(sbt, sb, sc, u, section)
    
    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

#### update one section score 

In [ ]:
update_data=pd.read_excel("quiz06_score.xlsx", sheet_name="Quiz06")

In [ ]:
update_data

In [ ]:
for i in range(len(update_data)):
    score=update_data.loc[i,"Quiz06"]
    usrname=update_data.loc[i,"帳號"]
    sql='''UPDATE `edm`.`member_class` 
    SET `Submitted`='{}', `SectionScore`='{}'
    WHERE (`Username`={} and `SectionName`='{}');'''.format("已交", score, usrname, "【Quiz06】Quiz (1)")

    cursor=conn.cursor()
    row=cursor.execute(sql)
    conn.commit()
    cursor.close()

### auto-renew

In [ ]:
import pymysql
import re
import os
import copy
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from datetime import datetime, timedelta
import re
import requests
import os
from urllib.request import urlretrieve
#from urllib.parse import urlparse
from urllib.request import urlopen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### homework

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

main="https://ncueeclass.ncu.edu.tw/course/teamStage/4599"
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': ".\\downloaded"}
options.add_experimental_option('prefs', prefs)

u="m946108005@tmu.edu.tw"
p="tjp3tjp3"

driver=webdriver.Chrome("D:/programming_useful_material/chromedriver.exe",chrome_options=options)
driver.get(main)
#time.sleep(2)
element_login=driver.find_element_by_link_text('Login')
element_login.click()

element_account=driver.find_element_by_xpath('//*[@id="account"]/div/input')
element_account.send_keys(u)
element_password=driver.find_element_by_xpath('//*[@id="password"]/div/input')
element_password.send_keys(p)
element_send=driver.find_element_by_xpath('//*[@id="login_form"]/div[6]/div/button')
element_send.click()

#####homework#####
time.sleep(1)
element_hw=driver.find_element_by_xpath('//*[@id="mbox-inline"]/div/div[9]/ul/li[8]/a')
element_hw.click()

html=driver.page_source
soup=BeautifulSoup(html, "html.parser")
trs=soup.select("table#homeworkListTable tbody tr")

for tr in trs:
    tds=tr.select("td")
    #hw_na
    hw_na=tds[1].text.strip()
    #href
    href=tds[3].select("a")[0].get("href")
    #start_time
    if tds[4].text !="-":
        start_time=str(time.gmtime().tm_year)+"-"+tds[4].text+":00"
    elif tds[4].text =="-":
        start_time=""
    #end_time
    if tds[5].text !="-":
        end_time=str(time.gmtime().tm_year)+"-"+tds[5].text+":00"
    elif tds[5].text =="-":
        end_time=""
    #group_hw
    if tds[2].select("span")!=[]: group_hw="y"
    else: group_hw="n"
    
    if group_hw=="y":
        print(hw_na)
        driver.get("https://ncueeclass.ncu.edu.tw"+href)
        
        html=driver.page_source
        soup=BeautifulSoup(html, "html.parser")
        trs1=soup.select("table#submitList_table tbody tr")
        for tr1 in trs1:
            tds=tr1.select("td")
            
            #group
            grp=tds[2].text.replace(" ","")[1]
            
            #submit_time
            submit_time=str(time.gmtime().tm_year)+"-"+tds[5].text+":00"
            
            #score
            score=tds[6].text.strip()
            
            #TimeSpent
            if start_time!="":
                timespent=datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S")-datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
                timespent=(timespent.days*24)+(timespent.seconds/3600)
            elif start_time=="":
                timespent=0
            
            #submit_type
            if end_time!="":
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") > datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="已交"
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") < datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="遲交"
            elif end_time=="":
                submit_type="已交"        
                
            cursor=conn.cursor()
            cursor.execute("select `Username` from `edm`.`member` where `Group`='{}';".format(grp)) 
            section=cursor.fetchall()
            cursor.close()
            for user in section:
                print(user[0])
                sql='''UPDATE `edm`.`member_class` 
                SET `Submitted` = '{}',`SubmitTime` = '{}',`SectionScore` = {},`TimeSpent(h)` = {}  
                WHERE (`Username` = {} and `SectionName`='{}' );'''.format(submit_type, submit_time, int(score), float(timespent), int(user[0]), hw_na)
                cursor=conn.cursor()
                row=cursor.execute(sql)
                conn.commit()
                cursor.close() 
                
    elif group_hw=="n":
        print(hw_na)
        driver.get("https://ncueeclass.ncu.edu.tw"+href)
        
        html=driver.page_source
        soup=BeautifulSoup(html, "html.parser")
        trs1=soup.select("table#submitList_table tbody tr")
        for tr1 in trs1:
            tds=tr1.select("td")
            #username
            user=tds[2].select("div.fs-hint")[0].text           
            #submit_time
            submit_time=str(time.gmtime().tm_year)+"-"+tds[4].text+":00"
            #score
            if tds[5].text.strip()!="-":
                score=int(tds[5].text.strip())
            elif tds[5].text.strip()=="-":
                score=""
            #TimeSpent
            if start_time!="":
                timespent=datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S")-datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
                timespent=round((timespent.days*24)+(timespent.seconds/3600), 3)
            elif start_time=="":
                timespent=0
            #submit_type
            if end_time!="":
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") > datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="已交"
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") < datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="遲交"
            elif end_time=="":
                submit_type="已交"        
            cursor=conn.cursor()
            cursor.execute("select `SectionName` from `edm`.`section`;") 
            sections=cursor.fetchall()
            cursor.close()
            for sec in sections:
                if "【Pract"+hw_na[3:5]+"】" in sec[0] or "【Week"+hw_na[3:5]+"】" in sec[0]:
                    print(user, sec[0])
                    sql='''UPDATE `edm`.`member_class` 
                    SET `Submitted` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {}  
                    WHERE (`Username` = {} and `SectionName`='{}' );'''.format(submit_type, submit_time, timespent, int(user), sec[0])
                    cursor=conn.cursor()
                    row=cursor.execute(sql)
                    conn.commit()
                    cursor.close()
                    if score!="":
                        sql='''UPDATE `edm`.`member_class` 
                        SET `SectionScore` = {} WHERE (`Username` = {} and `SectionName`='{}' );'''.format(score, int(user), sec[0])
                        cursor=conn.cursor()
                        row=cursor.execute(sql)
                        conn.commit()
                        cursor.close()

#### test

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

main="https://ncueeclass.ncu.edu.tw/course/teamStage/4599"
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': ".\\downloaded"}
options.add_experimental_option('prefs', prefs)

u="m946108005@tmu.edu.tw"
p="tjp3tjp3"

driver=webdriver.Chrome("D:/programming_useful_material/chromedriver.exe",chrome_options=options)
driver.get(main)
#time.sleep(2)
element_login=driver.find_element_by_link_text('Login')
element_login.click()

element_account=driver.find_element_by_xpath('//*[@id="account"]/div/input')
element_account.send_keys(u)
element_password=driver.find_element_by_xpath('//*[@id="password"]/div/input')
element_password.send_keys(p)
element_send=driver.find_element_by_xpath('//*[@id="login_form"]/div[6]/div/button')
element_send.click()

#####test#####
time.sleep(1)
element_test=driver.find_element_by_xpath('//*[@id="mbox-inline"]/div/div[9]/ul/li[9]/a')
element_test.click()

html=driver.page_source
soup=BeautifulSoup(html, "html.parser")
trs=soup.select("table#examListTable tbody tr")

for tr in trs:
    tds=tr.select("td")
    #test_na
    test_na=tds[1].text.strip()
    print(test_na)
    #href
    href=tds[3].select("a")[0].get("href")
    #test_yet
    test_yet=tds[3].text.strip()
    
    if test_yet !="0":
        driver.get("https://ncueeclass.ncu.edu.tw"+href)        
        html1=driver.page_source
        soup1=BeautifulSoup(html1, "html.parser")
        trs1=soup1.select("table#highestScore-list tbody tr")
        element_submit_info=driver.find_elements_by_xpath('//*[@id="highestScore-list"]/tbody/tr/td[6]/div/a')

        for tr1 in trs1:
            tds=tr1.select("td")
            #username
            usrn=tds[1].select("div.fs-hint")[0].text.strip()
            print(usrn)
            #score
            score=tds[2].text.strip()
            print(score)
            #submit_info
            ind=trs1.index(tr1)
            element_submit_info[ind].click()
            time.sleep(2)
            html2=driver.page_source
            soup2=BeautifulSoup(html2, "html.parser")
            trs2=soup2.select("table#recordList tbody tr")
            starttime=trs2[0].select("td")[3].text
            endtime=trs2[-1].select("td")[4].text
            print(starttime)
            print(endtime)
            element_close_submit_info=driver.find_element_by_xpath('//*[@id="ajaxMdModalId"]/div/div/div[1]/button')
            element_close_submit_info.click()
            time.sleep(1)
            #TimeSpent
            timespent=datetime.strptime(endtime, "%Y-%m-%d %H:%M:%S")-datetime.strptime(starttime, "%Y-%m-%d %H:%M:%S")
            timespent=round((timespent.days*24)+(timespent.seconds/3600), 3)

            sql='''UPDATE `edm`.`member_class` 
            SET `Submitted` = '{}',`SectionTimeStart` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {},`SectionScore` = {}
            WHERE (`Username` = {} and `SectionName`='{}' );'''.format("已交", starttime, endtime, timespent, score, usrn, test_na)
            cursor=conn.cursor()
            row=cursor.execute(sql)
            conn.commit()
            cursor.close()
            print("="*50)

#### questionaire

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
main="https://ncueeclass.ncu.edu.tw/course/teamStage/4599"
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': ".\\downloaded"}
options.add_experimental_option('prefs', prefs)

u="m946108005@tmu.edu.tw"
p="tjp3tjp3"

driver=webdriver.Chrome("D:/programming_useful_material/chromedriver.exe",chrome_options=options)
driver.get(main)
#time.sleep(2)
element_login=driver.find_element_by_link_text('Login')
element_login.click()

element_account=driver.find_element_by_xpath('//*[@id="account"]/div/input')
element_account.send_keys(u)
element_password=driver.find_element_by_xpath('//*[@id="password"]/div/input')
element_password.send_keys(p)
element_send=driver.find_element_by_xpath('//*[@id="login_form"]/div[6]/div/button')
element_send.click()

#####questionaire#####
time.sleep(1)
element_ques=driver.find_element_by_xpath('//*[@id="mbox-inline"]/div/div[9]/ul/li[10]/a')
element_ques.click()

html=driver.page_source
soup=BeautifulSoup(html, "html.parser")
trs=soup.select("table#questionnaireListTable tbody tr")

for tr in trs:
    tds=tr.select("td")
    #ques_na
    ques_na=tds[1].text.strip()
    print(ques_na)
    #href
    href=tds[3].select("a")[0].get("href")
    #limit_time
    limit_time=datetime.strptime(str(datetime.now().year)+"-"+tds[2].text.strip()+":00",  "%Y-%m-%d %H:%M:%S")
    
    if datetime.now()>limit_time:
        driver.get("https://ncueeclass.ncu.edu.tw"+href)        
        html1=driver.page_source
        soup1=BeautifulSoup(html1, "html.parser")
        trs1=soup1.select("table#statisticsResultTable tbody tr")

        for tr1 in trs1:
            tds=tr1.select("td")
            #username
            usrn=tds[2].text.strip()
            print(usrn)
            #submit_time
            submit_time=tds[5].text.strip()
            #TimeSpent
            timespent=0

            sql='''UPDATE `edm`.`member_class` 
            SET `Submitted` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {}
            WHERE (`Username` = {} and `SectionName`='{}' );'''.format("已交", submit_time, timespent, usrn, ques_na)
            cursor=conn.cursor()
            row=cursor.execute(sql)
            conn.commit()
            cursor.close()
            print("="*50)

#### mix (homework+test+questionaire)

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

main="https://ncueeclass.ncu.edu.tw/course/4691"
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': ".\\downloaded"}
options.add_experimental_option('prefs', prefs)

u="m946108005@tmu.edu.tw"
p="tjp3tjp3"

driver=webdriver.Chrome("D:/programming_useful_material/chromedriver.exe",chrome_options=options)
driver.get(main)
#time.sleep(2)
element_login=driver.find_element_by_link_text('Login')
element_login.click()

element_account=driver.find_element_by_xpath('//*[@id="account"]/div/input')
element_account.send_keys(u)
element_password=driver.find_element_by_xpath('//*[@id="password"]/div/div[1]/input')
element_password.send_keys(p)
element_send=driver.find_element_by_xpath('//*[@id="login_form"]/div[6]/div/button')
element_send.click()

#####homework#####
time.sleep(1)
element_hw=driver.find_element_by_xpath('//*[@id="mbox-inline"]/div/div[9]/ul/li[8]/a')
element_hw.click()

html=driver.page_source
soup=BeautifulSoup(html, "html.parser")
trs=soup.select("table#homeworkListTable tbody tr")

for tr in trs:
    tds=tr.select("td")
    #hw_na
    hw_na=tds[1].text.strip()
    #href
    href=tds[3].select("a")[0].get("href")
    #start_time
    if tds[4].text !="-":
        start_time=str(time.gmtime().tm_year)+"-"+tds[4].text+":00"
    elif tds[4].text =="-":
        start_time=""
    sql='''select `SectionTimeStart` from `edm`.`section` where `SectionName` like '%{}%' and `SectionTimeStart` is not null;
    '''.format(re.search("[0-9]+", hw_na).group())
    cursor=conn.cursor()
    cursor.execute(sql) 
    tt=cursor.fetchall()
    cursor.close()    
    if tt!=():
        start_time=str(tt[0][0])
    #end_time
    if tds[5].text !="-" and ":" in tds[5].text: 
        end_time=str(time.gmtime().tm_year)+"-"+tds[5].text+":00"
    if tds[5].text !="-" and ":" not in tds[5].text: 
        end_time=tds[5].text+" 09:00:00"
    elif tds[5].text =="-":
        end_time=""
    #group_hw
    if tds[2].select("span")!=[]: group_hw="y"
    else: group_hw="n"
    
    if group_hw=="y":
        print(hw_na)
        driver.get("https://ncueeclass.ncu.edu.tw"+href)
        
        html=driver.page_source
        soup=BeautifulSoup(html, "html.parser")
        trs1=soup.select("table#submitList_table tbody tr")
        for tr1 in trs1:
            tds=tr1.select("td")
            
            #group
            grp=tds[2].text.replace(" ","")[1]

            #submit_time
            if tds[5].text=="-":
                submit_time=""
            else:
                submit_time=str(time.gmtime().tm_year)+"-"+tds[5].text+":00"
            
            #score
            if tds[6].text.strip()!="-":
                score=int(tds[6].text.strip())
            elif tds[6].text.strip()=="-":
                score=""
            #TimeSpent
            if start_time!="" and submit_time!="":
                timespent=datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S")-datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
                timespent=(timespent.days*24)+(timespent.seconds/3600)
            elif start_time=="" or submit_time=="":
                timespent=0
            
            #submit_type
            if end_time!="" and submit_time!="":
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") > datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="已交"
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") < datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="遲交"
            elif end_time=="":
                submit_type="已交"        
                
            cursor=conn.cursor()
            cursor.execute("select `Username` from `edm`.`member` where `Group`='{}';".format(grp)) 
            section=cursor.fetchall()
            cursor.close()
            for user in section:
                print(user[0])
                sql='''UPDATE `edm`.`member_class` 
                SET `Submitted` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {}  
                WHERE (`Username` = {} and `SectionName`='{}' );'''.format(submit_type, submit_time, float(timespent), int(user[0]), hw_na)
                cursor=conn.cursor()
                row=cursor.execute(sql)
                conn.commit()
                cursor.close() 
                if score!="":
                    sql='''UPDATE `edm`.`member_class` 
                    SET `SectionScore` = {}  
                    WHERE (`Username` = {} and `SectionName`='{}' );'''.format(int(score), int(user[0]), hw_na)
                    cursor=conn.cursor()
                    row=cursor.execute(sql)
                    conn.commit()
                    cursor.close()              
                
    elif group_hw=="n":
        print(hw_na)
        driver.get("https://ncueeclass.ncu.edu.tw"+href)
        
        html=driver.page_source
        soup=BeautifulSoup(html, "html.parser")
        trs1=soup.select("table#submitList_table tbody tr")
        for tr1 in trs1:
            tds=tr1.select("td")
            #username
            user=tds[2].select("div.fs-hint")[0].text           
            #submit_time
            if tds[4].text=="-":
                submit_time=""
            else:
                submit_time=str(time.gmtime().tm_year)+"-"+tds[4].text+":00"
            #score
            if tds[5].text.strip()!="-":
                score=int(tds[5].text.strip())
            elif tds[5].text.strip()=="-":
                score=""
            #TimeSpent
            if start_time!="" and submit_time!="":
                timespent=datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S")-datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
                timespent=round((timespent.days*24)+(timespent.seconds/3600), 3)
            elif start_time=="" or submit_time=="":
                timespent=0
            #submit_type
            if end_time!="" and submit_time!="":
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") > datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="已交"
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") < datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="遲交"
            elif end_time=="":
                submit_type="已交"        
            cursor=conn.cursor()
            cursor.execute("select `SectionName` from `edm`.`section`;") 
            sections=cursor.fetchall()
            cursor.close()
            for sec in sections:
                if "Prac"  in sec[0] or "Week" in sec[0]:
                    if re.findall("[0-9]+",hw_na.split("】")[0])[0] in sec[0]:
                        print(user, sec[0])
                        sql='''UPDATE `edm`.`member_class` 
                        SET `Submitted` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {}  
                        WHERE (`Username` = {} and `SectionName`='{}' );'''.format(submit_type, submit_time, timespent, int(user), sec[0])
                        cursor=conn.cursor()
                        row=cursor.execute(sql)
                        conn.commit()
                        cursor.close()
                        if score!="":
                            sql='''UPDATE `edm`.`member_class` 
                            SET `SectionScore` = {} WHERE (`Username` = {} and `SectionName`='{}' );'''.format(score, int(user), sec[0])
                            cursor=conn.cursor()
                            row=cursor.execute(sql)
                            conn.commit()
                            cursor.close()
                        
#####test#####
driver.get(main)
time.sleep(1)
element_test=driver.find_element_by_xpath('//*[@id="mbox-inline"]/div/div[9]/ul/li[9]/a')
element_test.click()

html=driver.page_source
soup=BeautifulSoup(html, "html.parser")
trs=soup.select("table#examListTable tbody tr")

for tr in trs:
    tds=tr.select("td")
    #test_na
    test_na=tds[1].text.strip()
    print(test_na)
    #href
    href=tds[3].select("a")[0].get("href")
    #test_yet
    test_yet=tds[3].text.strip()
    
    if test_yet !="0":
        cursor=conn.cursor()
        cursor.execute("select `SectionNumber` from `edm`.`section` where `NotebookName`='{}';".format(test_na)) 
        ScNb=cursor.fetchall()
        ScNb=int(ScNb[0][0])
        cursor.close()
        
        driver.get("https://ncueeclass.ncu.edu.tw"+href)        
        html1=driver.page_source
        soup1=BeautifulSoup(html1, "html.parser")
        trs1=soup1.select("table#highestScore-list tbody tr")
        element_submit_info=driver.find_elements_by_xpath('//*[@id="highestScore-list"]/tbody/tr/td[6]/div/a')

        for tr1 in trs1:
            tds=tr1.select("td")
            #username
            usrn=tds[1].select("div.fs-hint")[0].text.strip()
            print(usrn)
            #score
            score=tds[2].text.strip()
            print(score)
            #submit_info
            ind=trs1.index(tr1)
            element_submit_info[ind].click()
            time.sleep(2)
            html2=driver.page_source
            soup2=BeautifulSoup(html2, "html.parser")
            trs2=soup2.select("table#recordList tbody tr")
            starttime=trs2[0].select("td")[3].text
            endtime=trs2[-1].select("td")[4].text
            print(starttime)
            print(endtime)
            element_close_submit_info=driver.find_element_by_xpath('//*[@id="ajaxMdModalId"]/div/div/div[1]/button')
            element_close_submit_info.click()
            time.sleep(1)
            #TimeSpent
            timespent=datetime.strptime(endtime, "%Y-%m-%d %H:%M:%S")-datetime.strptime(starttime, "%Y-%m-%d %H:%M:%S")
            timespent=round((timespent.days*24)+(timespent.seconds/3600), 3)

            sql='''UPDATE `edm`.`member_class` 
            SET `Submitted` = '{}',`SectionTimeStart` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {},`SectionScore` = {}
            WHERE (`Username` = {} and `SectionNumber`={} );'''.format("已交", starttime, endtime, timespent, score, usrn, ScNb)
            cursor=conn.cursor()
            row=cursor.execute(sql)
            conn.commit()
            cursor.close()
            print("="*50)

#####questionaire#####
driver.get(main)
time.sleep(1)
element_ques=driver.find_element_by_xpath('//*[@id="mbox-inline"]/div/div[9]/ul/li[10]/a')
element_ques.click()

html=driver.page_source
soup=BeautifulSoup(html, "html.parser")
trs=soup.select("table#questionnaireListTable tbody tr")

for tr in trs:
    tds=tr.select("td")
    #ques_na
    ques_na=tds[1].text.strip()
    print(ques_na)
    #href
    href=tds[3].select("a")[0].get("href")
    #limit_time
    limit_time=datetime.strptime(str(datetime.now().year)+"-"+tds[2].text.strip()+":00",  "%Y-%m-%d %H:%M:%S")
    
    if datetime.now()>limit_time:
        driver.get("https://ncueeclass.ncu.edu.tw"+href)        
        html1=driver.page_source
        soup1=BeautifulSoup(html1, "html.parser")
        trs1=soup1.select("table#statisticsResultTable tbody tr")

        for tr1 in trs1:
            tds=tr1.select("td")
            #username
            usrn=tds[2].text.strip()
            print(usrn)
            #submit_time
            submit_time=tds[5].text.strip()
            #TimeSpent
            timespent=0

            sql='''UPDATE `edm`.`member_class` 
            SET `Submitted` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {}
            WHERE (`Username` = {} and `SectionName`='{}' );'''.format("已交", submit_time, timespent, usrn, ques_na)
            cursor=conn.cursor()
            row=cursor.execute(sql)
            conn.commit()
            cursor.close()
            print("="*50)
driver.close()

#### mix 跨年後

In [ ]:
'''
待修改：
ee-class 跨年後，會失去「作業繳交期限」前一年的「時間」資訊，僅保留日期。
除了爬蟲資訊會錯誤，爬到的資料若將資料庫舊有資訊覆蓋，也會讓遲交資訊遺失。
應該將繳交期限時間也儲存入資料庫，如期限有變更就覆蓋，但跨年後就不自動修改前一年的繳交期限資訊。
'''
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

main="https://ncueeclass.ncu.edu.tw/course/4691"
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': ".\\downloaded"}
options.add_experimental_option('prefs', prefs)

u="m946108005@tmu.edu.tw"
p="tjp3tjp3"

driver=webdriver.Chrome("D:/programming_useful_material/chromedriver.exe",chrome_options=options)
driver.get(main)
#time.sleep(2)
element_login=driver.find_element_by_link_text('Login')
element_login.click()

element_account=driver.find_element_by_xpath('//*[@id="account"]/div/input')
element_account.send_keys(u)
element_password=driver.find_element_by_xpath('//*[@id="password"]/div/div[1]/input')
element_password.send_keys(p)
element_send=driver.find_element_by_xpath('//*[@id="login_form"]/div[6]/div/button')
element_send.click()

#####homework#####
time.sleep(1)
element_hw=driver.find_element_by_xpath('//*[@id="mbox-inline"]/div/div[9]/ul/li[8]/a')
element_hw.click()

html=driver.page_source
soup=BeautifulSoup(html, "html.parser")
trs=soup.select("table#homeworkListTable tbody tr")

for tr in trs:
    tds=tr.select("td")
    #hw_na
    hw_na=tds[1].text.strip()
    num=int(re.findall("[0-9]+",hw_na.split("】")[0])[0])
    if num==15:
        timing=" 09:00:00"
    elif num==16:
        timing=" 11:20:00"
    else:
        continue
    #href
    href=tds[3].select("a")[0].get("href")
    #start_time
    if tds[4].text !="-":
        start_time=str(time.gmtime().tm_year)+"-"+tds[4].text+":00"
    elif tds[4].text =="-":
        start_time=""
    sql='''select `SectionTimeStart` from `edm`.`section` where `SectionName` like '%{}%' and `SectionTimeStart` is not null;
    '''.format(re.search("[0-9]+", hw_na).group())
    cursor=conn.cursor()
    cursor.execute(sql) 
    tt=cursor.fetchall()
    cursor.close()    
    if tt!=():
        start_time=str(tt[0][0])
    #end_time
    if tds[5].text !="-" and ":" in tds[5].text: 
        end_time=str(time.gmtime().tm_year)+"-"+tds[5].text+":00"
    if tds[5].text !="-" and ":" not in tds[5].text: 
        end_time=tds[5].text+timing
    elif tds[5].text =="-":
        end_time=""
    #group_hw
    if tds[2].select("span")!=[]: group_hw="y"
    else: group_hw="n"
    
    if group_hw=="y":
        print(hw_na)
        driver.get("https://ncueeclass.ncu.edu.tw"+href)
        
        html=driver.page_source
        soup=BeautifulSoup(html, "html.parser")
        trs1=soup.select("table#submitList_table tbody tr")
        for tr1 in trs1:
            tds=tr1.select("td")
            
            #group
            grp=tds[2].text.replace(" ","")[1]

            #submit_time
            if tds[5].text=="-":
                submit_time=""
            else:
                if tds[5].text.startswith("12"):
                    submit_time="2020"+"-"+tds[5].text+":00"
                elif tds[5].text.startswith("01"):
                    submit_time=str(time.gmtime().tm_year)+"-"+tds[5].text+":00"
            #score
            if tds[6].text.strip()!="-":
                score=int(tds[6].text.strip())
            elif tds[6].text.strip()=="-":
                score=""
            #TimeSpent
            if start_time!="" and submit_time!="":
                timespent=datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S")-datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
                timespent=(timespent.days*24)+(timespent.seconds/3600)
            elif start_time=="" or submit_time=="":
                timespent=0
            
            #submit_type
            if end_time!="" and submit_time!="":
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") > datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="已交"
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") < datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="遲交"
            elif end_time=="":
                submit_type="已交"        
                
            cursor=conn.cursor()
            cursor.execute("select `Username` from `edm`.`member` where `Group`='{}';".format(grp)) 
            section=cursor.fetchall()
            cursor.close()
            for user in section:
                print(user[0])
                sql='''UPDATE `edm`.`member_class` 
                SET `Submitted` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {}  
                WHERE (`Username` = {} and `SectionName`='{}' );'''.format(submit_type, submit_time, float(timespent), int(user[0]), hw_na)
                cursor=conn.cursor()
                row=cursor.execute(sql)
                conn.commit()
                cursor.close() 
                if score!="":
                    sql='''UPDATE `edm`.`member_class` 
                    SET `SectionScore` = {}  
                    WHERE (`Username` = {} and `SectionName`='{}' );'''.format(int(score), int(user[0]), hw_na)
                    cursor=conn.cursor()
                    row=cursor.execute(sql)
                    conn.commit()
                    cursor.close()              
                
    elif group_hw=="n":
        print(hw_na)
        driver.get("https://ncueeclass.ncu.edu.tw"+href)
        
        html=driver.page_source
        soup=BeautifulSoup(html, "html.parser")
        trs1=soup.select("table#submitList_table tbody tr")
        for tr1 in trs1:
            tds=tr1.select("td")
            #username
            user=tds[2].select("div.fs-hint")[0].text           
            #submit_time
            if tds[4].text=="-":
                submit_time=""
            else:
                if tds[4].text.startswith("12"):
                    submit_time="2020"+"-"+tds[4].text+":00"
                elif tds[4].text.startswith("01"):
                    submit_time=str(time.gmtime().tm_year)+"-"+tds[4].text+":00"
            #score
            if tds[5].text.strip()!="-":
                score=int(tds[5].text.strip())
            elif tds[5].text.strip()=="-":
                score=""
            #TimeSpent
            if start_time!="" and submit_time!="":
                timespent=datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S")-datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
                timespent=round((timespent.days*24)+(timespent.seconds/3600), 3)
            elif start_time=="" or submit_time=="":
                timespent=0
            #submit_type
            if end_time!="" and submit_time!="":
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") > datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="已交"
                if datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S") < datetime.strptime(submit_time, "%Y-%m-%d %H:%M:%S"):
                    submit_type="遲交"
            elif end_time=="":
                submit_type="已交"        
            cursor=conn.cursor()
            cursor.execute("select `SectionName` from `edm`.`section`;") 
            sections=cursor.fetchall()
            cursor.close()
            for sec in sections:
                if "Prac"  in sec[0] or "Week" in sec[0]:
                    if re.findall("[0-9]+",hw_na.split("】")[0])[0] in sec[0]:
                        print(user, sec[0])
                        sql='''UPDATE `edm`.`member_class` 
                        SET `Submitted` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {}  
                        WHERE (`Username` = {} and `SectionName`='{}' );'''.format(submit_type, submit_time, timespent, int(user), sec[0])
                        cursor=conn.cursor()
                        row=cursor.execute(sql)
                        conn.commit()
                        cursor.close()
                        if score!="":
                            sql='''UPDATE `edm`.`member_class` 
                            SET `SectionScore` = {} WHERE (`Username` = {} and `SectionName`='{}' );'''.format(score, int(user), sec[0])
                            cursor=conn.cursor()
                            row=cursor.execute(sql)
                            conn.commit()
                            cursor.close()
                        
#####test#####
# driver.get(main)
# time.sleep(1)
# element_test=driver.find_element_by_xpath('//*[@id="mbox-inline"]/div/div[9]/ul/li[9]/a')
# element_test.click()

# html=driver.page_source
# soup=BeautifulSoup(html, "html.parser")
# trs=soup.select("table#examListTable tbody tr")

# for tr in trs:
#     tds=tr.select("td")
#     #test_na
#     test_na=tds[1].text.strip()
#     print(test_na)
#     #href
#     href=tds[3].select("a")[0].get("href")
#     #test_yet
#     test_yet=tds[3].text.strip()
    
#     if test_yet !="0":
#         cursor=conn.cursor()
#         cursor.execute("select `SectionNumber` from `edm`.`section` where `NotebookName`='{}';".format(test_na)) 
#         ScNb=cursor.fetchall()
#         ScNb=int(ScNb[0][0])
#         cursor.close()
        
#         driver.get("https://ncueeclass.ncu.edu.tw"+href)        
#         html1=driver.page_source
#         soup1=BeautifulSoup(html1, "html.parser")
#         trs1=soup1.select("table#highestScore-list tbody tr")
#         element_submit_info=driver.find_elements_by_xpath('//*[@id="highestScore-list"]/tbody/tr/td[6]/div/a')

#         for tr1 in trs1:
#             tds=tr1.select("td")
#             #username
#             usrn=tds[1].select("div.fs-hint")[0].text.strip()
#             print(usrn)
#             #score
#             score=tds[2].text.strip()
#             print(score)
#             #submit_info
#             ind=trs1.index(tr1)
#             element_submit_info[ind].click()
#             time.sleep(2)
#             html2=driver.page_source
#             soup2=BeautifulSoup(html2, "html.parser")
#             trs2=soup2.select("table#recordList tbody tr")
#             starttime=trs2[0].select("td")[3].text
#             endtime=trs2[-1].select("td")[4].text
#             print(starttime)
#             print(endtime)
#             element_close_submit_info=driver.find_element_by_xpath('//*[@id="ajaxMdModalId"]/div/div/div[1]/button')
#             element_close_submit_info.click()
#             time.sleep(1)
#             #TimeSpent
#             timespent=datetime.strptime(endtime, "%Y-%m-%d %H:%M:%S")-datetime.strptime(starttime, "%Y-%m-%d %H:%M:%S")
#             timespent=round((timespent.days*24)+(timespent.seconds/3600), 3)

#             sql='''UPDATE `edm`.`member_class` 
#             SET `Submitted` = '{}',`SectionTimeStart` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {},`SectionScore` = {}
#             WHERE (`Username` = {} and `SectionNumber`={} );'''.format("已交", starttime, endtime, timespent, score, usrn, ScNb)
#             cursor=conn.cursor()
#             row=cursor.execute(sql)
#             conn.commit()
#             cursor.close()
#             print("="*50)

#####questionaire#####
driver.get(main)
time.sleep(1)
element_ques=driver.find_element_by_xpath('//*[@id="mbox-inline"]/div/div[9]/ul/li[10]/a')
element_ques.click()

html=driver.page_source
soup=BeautifulSoup(html, "html.parser")
trs=soup.select("table#questionnaireListTable tbody tr")

for tr in trs:
    tds=tr.select("td")
    #ques_na
    ques_na=tds[1].text.strip()
    print(ques_na)
    if ques_na=='Survey of leaning motivation and background 學習動機及背景問卷':
        continue
    #href
    href=tds[3].select("a")[0].get("href")
    #limit_time
    limit_time=datetime.strptime(str(datetime.now().year)+"-"+tds[2].text.strip()+":00",  "%Y-%m-%d %H:%M:%S")
    
    if datetime.now()>limit_time:
        driver.get("https://ncueeclass.ncu.edu.tw"+href)        
        html1=driver.page_source
        soup1=BeautifulSoup(html1, "html.parser")
        trs1=soup1.select("table#statisticsResultTable tbody tr")

        for tr1 in trs1:
            tds=tr1.select("td")
            #username
            usrn=tds[2].text.strip()
            print(usrn)
            #submit_time
            submit_time=tds[5].text.strip()
            #TimeSpent
            timespent=0

            sql='''UPDATE `edm`.`member_class` 
            SET `Submitted` = '{}',`SubmitTime` = '{}',`TimeSpent(h)` = {}
            WHERE (`Username` = {} and `SectionName`='{}' );'''.format("已交", submit_time, timespent, usrn, ques_na)
            cursor=conn.cursor()
            row=cursor.execute(sql)
            conn.commit()
            cursor.close()
            print("="*50)
driver.close()

### section final content 

In [ ]:
import pymysql
import numpy as np
import pandas as pd
import re
import os
import copy
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
from urllib.request import urlretrieve
#from urllib.parse import urlparse
from urllib.request import urlopen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
hw_list=os.listdir("./homework")

hw_list.remove("【Mid09】Midterm_Paper organization and report")
hw_list.remove("【Prac02】 Finding your own keywords for LA or EDM")

print(hw_list)

In [ ]:
#字數不算"\n"
for hw in hw_list:
    std_list=os.listdir("./homework/"+hw)
    std_list.remove('index.html')
    print("==========【",hw,"】==========")
    
    for std in std_list:
        print("=====【",std,"】=====")
        week_assignments=os.listdir("./homework/"+hw+"/"+std+"/attach")
        
        for assignment in week_assignments:
            a=re.findall("【.+】",assignment)[0]
            print("###",assignment,"###")
            try:
                html=open("./homework/"+hw+"/"+std+"/attach/"+assignment, encoding="utf-8")
                x=html.read()
                html.close()
                x=BeautifulSoup(x, "html.parser")

                xx=x.select("div#notebook-container")[0].select("div.inner_cell")
                code=""
                remark=""
                for j in range(len(xx)):
                    xx[j]=xx[j].text.replace("\n\n", "").strip("\n")
                while "\n" in xx:
                    xx.remove("\n")

                total="\n".join(xx).replace("'",'"')
                for j in xx:
                    if "¶" in j:
                        remark+=j+"\n"
                    elif re.findall("#.*[\n]?", j)!=[]:
                        remark+="".join(re.findall("#.*[\n]?", j))+"\n"
                        code+="\n".join(re.split("#.*[\n]?",j))+"\n"
                    else:
                        code+=j+"\n"
                code=code.replace("'",'"')

                while "\n\n" in code:
                    code=code.replace("\n\n", "\n")

                remark=remark.replace("'",'"')

                sql='''UPDATE `edm`.`member_class` 
                SET `NotebookLength_final` = {},`NotebookContent_final` = '{}',
                `CodeLength_final` = {},`CodeContent_final` = '{}',
                `RemarkLength_final` = {},`RemarkContent_final` = '{}'
                WHERE (`Username` = {} and `SectionName` like '%{}%');'''.format(len(total.replace("\n","")), total, len(code.replace("\n","")), code, len(remark.replace("\n","")), remark, int(std[:9]), a )
                cursor=conn.cursor()
                row=cursor.execute(sql)
                conn.commit()
                cursor.close()

                print(remark)
                print("===")
                print(code)
                print("====================")           
            except:
                print("●【ERROR】●")

#### by file

In [ ]:
std=int(input("請輸入學號"))
file=input("請輸入檔案路徑")
a=re.findall("【.+】",file)[0]

html=open(file, encoding="utf-8")
x=html.read()
html.close()
x=BeautifulSoup(x, "html.parser")

xx=x.select("div#notebook-container")[0].select("div.inner_cell")
code=""
remark=""
for j in range(len(xx)):
    xx[j]=xx[j].text.replace("\n\n", "").strip("\n")
while "\n" in xx:
    xx.remove("\n")

total="\n".join(xx).replace("'",'"')
for j in xx:
    if "¶" in j:
        remark+=j+"\n"
    elif re.findall("#.*[\n]?", j)!=[]:
        remark+="".join(re.findall("#.*[\n]?", j))+"\n"
        code+="\n".join(re.split("#.*[\n]?",j))+"\n"
    else:
        code+=j+"\n"
code=code.replace("'",'"')

while "\n\n" in code:
    code=code.replace("\n\n", "\n")

remark=remark.replace("'",'"')

sql='''UPDATE `edm`.`member_class` 
SET `NotebookLength_final` = {},`NotebookContent_final` = '{}',
`CodeLength_final` = {},`CodeContent_final` = '{}',
`RemarkLength_final` = {},`RemarkContent_final` = '{}'
WHERE (`Username` = {} and `SectionName` like '%{}%');'''.format(len(total.replace("\n","")), total, len(code.replace("\n","")), code, len(remark.replace("\n","")), remark, std, a )
cursor=conn.cursor()
row=cursor.execute(sql)
conn.commit()
cursor.close()

print(remark)
print("===")
print(code)
print("====================")           

### 更新EditTime_Inclass, BlankTime_Inclass, EditTime_Notinclass, EditDaysCt 

#### 找出課內/課外操作時間間隔之分佈

In [ ]:
# #課內/課外時間練習操作間距時間差
# import matplotlib.pyplot as plt
# section=pd.read_excel("section.xlsx")
# date=list(set(section["Date"][pd.notnull(section["Date"])]))

# cursor=conn.cursor()
# cursor.execute("select `timestamp`,`Username` from `edm`.`edm_log`;")
# tms=cursor.fetchall()
# cursor.close()
# tms=list(tms)
# io=[]
# dt=[]
# ur=[]
# for i in range(len(tms)):
#     day="-".join(tms[i][0].split("-")[:3])
#     u=tms[i][1]
#     tms[i]=":".join(tms[i][0].split(":")[:3])
#     if day in date and int(tms[i].split("-")[-1][:2])>=9 and int(tms[i].split("-")[-1][:2])<12:        
#         io.append(1)
#         dt.append(day)
#         ur.append(u)
#     else:
#         io.append(0)
#         dt.append(day)
#         ur.append(u)
    
# tms=list(map(lambda t: datetime.strptime(t, "%Y-%m-%d-%H:%M:%S"), tms))
# t_df=pd.DataFrame({"time":tms, "inclass":io, "date":dt, "user": ur})

# inclass_td=[]
# outclass_td=[]
# for d in date:
#     print(d)
#     t_df2=t_df[t_df["date"]==d]
#     users=list(set(t_df2["user"]))
#     for user in users:
#         t_df3=t_df2[t_df2["user"]==user]
#         inclass_tm=list(t_df3["time"][t_df3["inclass"]==1])
#         for i in range(1,len(inclass_tm)):
#             t=(inclass_tm[i]-inclass_tm[i-1]).seconds
#             inclass_td.append(t)
# #             if t>1000:
# #                 print(inclass_tm[i-1],"~",inclass_tm[i],"==>",t)
#         outclass_tm=list(t_df3["time"][t_df3["inclass"]==0])
#         for i in range(1,len(outclass_tm)):
#             t=(outclass_tm[i]-outclass_tm[i-1]).seconds
#             outclass_td.append(t)
# #             if t>1000:
# #                 print(outclass_tm[i-1],"~",outclass_tm[i],"==>",t)

In [ ]:
# inclass_td_sr=pd.Series(inclass_td)
# print(inclass_td_sr.describe())
# iqr=inclass_td_sr.describe()["75%"]-inclass_td_sr.describe()["25%"]
# print("===============")
# print("IQR=",iqr)
# inclass_uplimit=inclass_td_sr.describe()["75%"]+(1.5*iqr)
# print("Q3+1.5xIQR=",inclass_uplimit)

# plt.figure(figsize=(16,4))
# plt.hist(inclass_td_sr[inclass_td_sr<1000], bins=1000)
# plt.show()

In [ ]:
# outclass_td_sr=pd.Series(outclass_td)
# print(outclass_td_sr.describe())
# iqr=outclass_td_sr.describe()["75%"]-outclass_td_sr.describe()["25%"]
# print("===============")
# print("IQR=",iqr)
# outclass_uplimit=outclass_td_sr.describe()["75%"]+(1.5*iqr)
# print("Q3+1.5xIQR=",outclass_uplimit)

# plt.figure(figsize=(16,4))
# plt.hist(outclass_td_sr[outclass_td_sr<1000], bins=1000)
# plt.show()

#### 計算EditTime_Inclass, BlankTime_Inclass, EditTime_Notinclass, EditDaysCt
- inclass_td Q3+1.5xIQR = 93.0 / inclass_uplimit=300
- outclass_td Q3+1.5xIQR = 69.5 / outclass_uplimit=300

In [ ]:
inclass_uplimit=300
outclass_uplimit=300

conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")


from datetime import datetime, timedelta
import matplotlib.pyplot as plt

section=pd.read_excel("section.xlsx")
date=list(set(section["Date"][pd.notnull(section["Date"])]))

cursor=conn.cursor()
cursor.execute("select distinct `Username` from `edm_backup`.`member_class`;")
usrs=cursor.fetchall()
cursor.close()
usrs=list(usrs)
for i in range(len(usrs)):
    usrs[i]=usrs[i][0]

for usr in usrs:
    cursor=conn.cursor()
    cursor.execute("select distinct `notebookName` from `edm_backup`.`edm_log` where `Username`='{}';".format(usr))
    nbs=cursor.fetchall()
    cursor.close()
    nbs=list(nbs)
    for i in range(len(nbs)):
        nbs[i]=nbs[i][0]
    for nb in nbs:
        cursor=conn.cursor()
        cursor.execute("select `timestamp` from `edm_backup`.`edm_log` where `Username`='{}' and `notebookName`='{}';".format(usr,nb))
        tms=cursor.fetchall()
        cursor.close()
        
        d=set()  #不重複日期
        io=[]  #in class/ not in class
        dt=[]
        tms=list(tms)
        for i in range(len(tms)):
            day="-".join(tms[i][0].split("-")[:3])
            d.add(day)
            tms[i]=":".join(tms[i][0].split(":")[:3])
            if day in date and int(tms[i].split("-")[-1][:2])>=9 and int(tms[i].split("-")[-1][:2])<12:        
                io.append(1)
                dt.append(day)
            else:
                io.append(0)
                dt.append(day)
        tms=list(map(lambda t: datetime.strptime(t, "%Y-%m-%d-%H:%M:%S"), tms))
        t_df=pd.DataFrame({"time":tms, "inclass":io, "date":dt})        
        d=list(d)
        
        inclass_td=[]
        inclass_delay=[]  
        for day in list(set(dt)):
            t_df2=t_df[t_df["date"]==day]
            inclass_tm=list(t_df2["time"][t_df2["inclass"]==1])
            for i in range(1,len(inclass_tm)):
                t=(inclass_tm[i]-inclass_tm[i-1]).seconds
                if t > inclass_uplimit:
                    inclass_td.append(inclass_uplimit)
                    inclass_delay.append(t-inclass_uplimit)
                else:
                    inclass_td.append(t)
                
        outclass_td=[]
        for day in list(set(dt)):
            t_df2=t_df[t_df["date"]==day]
            outclass_tm=list(t_df2["time"][t_df2["inclass"]==0])
            for i in range(1,len(outclass_tm)):
                t=(outclass_tm[i]-outclass_tm[i-1]).seconds
                if t >outclass_uplimit:
                    outclass_td.append(outclass_uplimit)
                else:
                    outclass_td.append(t)
        
        sql='''
        UPDATE `edm_backup`.`member_class` 
        SET `EditTime_Inclass` = {}, `BlankTime_Inclass` = {}, `EditTime_Notinclass` = {}, `EditDaysCt` = {}
        WHERE (`Username` ={} and `SectionName`='{}');
        '''.format(sum(inclass_td), sum(inclass_delay), sum(outclass_td), len(d), usr, nb.split(".ipynb")[0])
        cursor=conn.cursor()
        row=cursor.execute(sql)
        conn.commit()
        cursor.close()
        
        print(usr)        
        print(nb)
        print("in class time: ",sum(inclass_td))
        print("in class delay: ",sum(inclass_delay))
        print("not in class time: ",sum(outclass_td))
        print("閱讀日數:", len(d))
        print("================================")


### copy-paste data

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

cursor=conn.cursor()
cursor.execute("select distinct `Username` from `edm_backup`.`member_class`;")
usrs=cursor.fetchall()
cursor.close()
usrs=list(usrs)
for i in range(len(usrs)):
    usrs[i]=usrs[i][0]

for usr in usrs:
    cursor=conn.cursor()
    cursor.execute("select distinct `notebookName` from `edm_backup`.`edm_log` where `Username`='{}';".format(usr))
    nbs=cursor.fetchall()
    cursor.close()
    nbs=list(nbs)
    for i in range(len(nbs)):
        nbs[i]=nbs[i][0]
    for nb in nbs:
        cursor=conn.cursor()
        cursor.execute("select `event`, `text`,`cellId`,`CleanCode` from `edm_backup`.`edm_log` where `Username`='{}' and `notebookName`='{}';".format(usr,nb))
        data=cursor.fetchall()
        cursor.close()
        data=list(data)
        event=[]
        text=[]
        cellId=[]
        cleancode=[]
        for i in data:
            event.append(i[0])
            text.append(i[1])
            cellId.append(i[2])
            cleancode.append(i[3]) 
        df=pd.DataFrame({"event":event, "text":text, "cellId":cellId, "cleancode":cleancode})
        
        #CopyCt 
        CopyCt =len(df[df["event"]=="code_copy"])
        CopyText="".join(list(df["text"][df["event"]=="code_copy"]))
        CopyTextLength=len("".join(CopyText.split("\n")))
        PasteCt =len(df[df["event"]=="code_paste"])
        PasteText="".join(list(df["text"][df["event"]=="code_paste"]))
        PasteTextLength=len("".join(PasteText.split("\n")))
        
        df1=df[df["event"]=="code_paste"]
        #e=list(df1["event"])
        #t=list(df1["text"])
        copytext=""
        ExternalPasteCt=0
        ExternalPasteText=""
        for i in df1.index:
            previouscode=" ".join(list(df.loc[:i,"cleancode"]))
            if df1.loc[i,"text"] in previouscode:
                ExternalPasteCt+=0
            elif df1.loc[i,"text"] not in previouscode:
                ExternalPasteCt+=1
                ExternalPasteText+=df1.loc[i,"text"]+"\n"
        ExternalPasteLength=len("".join(ExternalPasteText.split("\n")))
        ExternalPasteText=ExternalPasteText.replace("'",'"')
        
        sql='''UPDATE `edm_backup`.`member_class` 
        SET `ExternalPasteCt` = {}, `ExternalPasteLength` = {}, `ExternalPasteText` = '{}', `CopyCt` = {}, `CopyTextLength` = {}, `PasteCt` = {}, `PasteTextLength` = {}
        WHERE (`Username` ='{}' and `SectionName`='{}');
        '''.format(ExternalPasteCt, ExternalPasteLength, ExternalPasteText, CopyCt, CopyTextLength, PasteCt, PasteTextLength, usr, nb.split(".ipynb")[0])
        cursor=conn.cursor()
        row=cursor.execute(sql)
        conn.commit()
        cursor.close() 
        
        print(usr)        
        print(nb)
        print("CopyCt", CopyCt)
        print("CopyTextLength", CopyTextLength)
        print("PasteCt", PasteCt)
        print("PasteTextLength", PasteTextLength)
        print("ExternalPasteCt", ExternalPasteCt)
        print("ExternalPasteLength", ExternalPasteLength)
        print("ExternalPasteText", ExternalPasteText)
        print("================================")        

## calender

In [ ]:
#沒有用

# conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
# for i in range(124):
#     x=str(datetime(2020,9,14)+timedelta(i))
#     print(x)
#     sql='''INSERT INTO `edm`.`calender` (`date`) VALUES ('{}')'''.format(x)
#     cursor=conn.cursor()
#     row=cursor.execute(sql)
#     conn.commit()
#     cursor.close()

#  Training用data

### feature from questionnaire

In [ ]:
qs=pd.read_excel("pre-questionnaire.xlsx",  sheet_name="clean (2)")
qs=qs[['id', 'wk_expr','non_py_expr','non_py_prof','py_expr','py_prof']]

In [ ]:
d1={"Yes":1, "No":0}
d2={"Proficient":3,"Good":2,"Rough":1}
qs["wk_expr"]=qs["wk_expr"].map(d1)
qs["non_py_expr"]=qs["non_py_expr"].map(d1)
qs["py_expr"]=qs["py_expr"].map(d1)
qs["non_py_prof"]=qs["non_py_prof"].map(d2)
qs["py_prof"]=qs["py_prof"].map(d2)
qs=qs.fillna(0)

In [ ]:
ptsc=pd.read_excel("pre-test_memberscore.xlsx")
#ptsc.columns=ptsc.loc[0,:]
#ptsc=ptsc.drop(0, axis=0)

In [ ]:
ptsc=ptsc[["帳號","成績"]].rename(columns={"帳號":"id","成績":"pretest_score"})

qs["id"]=qs["id"].astype("int32")
ptsc["id"]=ptsc["id"].astype("int32")

qs=pd.merge(qs, ptsc, on="id", how="left")

In [ ]:
qs

### quiz_score

In [ ]:
# quiz1=pd.read_excel("quiz06_score.xlsx", sheet_name="Quiz06")
# quiz2=pd.read_excel("quiz12_score.xlsx")

In [ ]:
# quiz=pd.merge(quiz1,quiz2[["帳號","成績"]], on="帳號", how="left").rename(columns={"帳號":"Username","成績":"Quiz12"})

In [ ]:
# quiz

In [ ]:
# quiz2=pd.read_excel("quiz12.xlsx", sheet_name="答題記錄 (2)")
# quiz2.columns=quiz2.loc[1,:]
# quiz2=quiz2.drop([0,1], axis=0).reset_index(drop=True)
# quiz2["帳號"]=quiz2["帳號"].astype("int32")

In [ ]:
# quiz2["Quiz12_basic"]=(quiz2.T[quiz2.T.index=="basic"].sum()/10)*100
# quiz2["Quiz12_library"]=(quiz2.T[quiz2.T.index=="library"].sum()/10)*100

In [ ]:
# quiz=pd.merge(quiz,quiz2.drop(["basic","library","成績","姓名"], axis=1).rename(columns={"帳號":"Username"}),on="Username", how="left")
# quiz=quiz.drop("姓名", axis=1)

In [ ]:
# #Quiz06
# Quiz06_group=[]
# for i in list(quiz["Quiz06"]):
#     if i <60:
#         Quiz06_group.append("C")
#     elif i>=60 and i<80:
#         Quiz06_group.append("B")
#     elif i>=80:
#         Quiz06_group.append("A")
# print(len(quiz[quiz["Quiz06"]<60]))
# print(len(quiz[(quiz["Quiz06"]>=60)&(quiz["Quiz06"]<80)]))
# print(len(quiz[quiz["Quiz06"]>=80]))

In [ ]:
# #Quiz12
# Quiz12_group=[]
# for i in list(quiz["Quiz12"]):
#     if i <60:
#         Quiz12_group.append("C")
#     elif i>=60 and i<80:
#         Quiz12_group.append("B")
#     elif i>=80:
#         Quiz12_group.append("A")
# print(len(quiz[quiz["Quiz12"]<60]))
# print(len(quiz[(quiz["Quiz12"]>=60)&(quiz["Quiz12"]<80)]))
# print(len(quiz[quiz["Quiz12"]>=80]))

In [ ]:
# #Quiz12_basic
# Quiz12basic_group=[]
# for i in list(quiz["Quiz12_basic"]):
#     if i <60:
#         Quiz12basic_group.append("C")
#     elif i>=60 and i<80:
#         Quiz12basic_group.append("B")
#     elif i>=80:
#         Quiz12basic_group.append("A")
# print(len(quiz[quiz["Quiz12_basic"]<60]))
# print(len(quiz[(quiz["Quiz12_basic"]>=60)&(quiz["Quiz12_basic"]<80)]))
# print(len(quiz[quiz["Quiz12_basic"]>=80]))

In [ ]:
# #Quiz12_library
# Quiz12library_group=[]
# for i in list(quiz["Quiz12_library"]):
#     if i <60:
#         Quiz12library_group.append("C")
#     elif i>=60 and i<80:
#         Quiz12library_group.append("B")
#     elif i>=80:
#         Quiz12library_group.append("A")
# print(len(quiz[quiz["Quiz12_library"]<60]))
# print(len(quiz[(quiz["Quiz12_library"]>=60)&(quiz["Quiz12_library"]<80)]))
# print(len(quiz[quiz["Quiz12_library"]>=80]))

In [ ]:
# quiz["Quiz06_group"]=Quiz06_group
# quiz["Quiz12_group"]=Quiz12_group
# quiz["Quiz12basic_group"]=Quiz12basic_group
# quiz["Quiz12library_group"]=Quiz12library_group

In [ ]:
#quiz.to_excel("檢驗點成績.xlsx")
quiz=pd.read_excel("檢驗點成績.xlsx")

In [ ]:
quiz.head()

### (不使用) final_sore (final部分改用老師給的未調整成績)

In [ ]:
# practice_score=pd.read_excel("practice_score.xlsx")
# sc_datathon=practice_score[practice_score["SectionName"]=="【Pract16】Datathon"]
# sc_datathon=sc_datathon[["SectionName","StudentId","accomplish"]].groupby(["SectionName","StudentId"]).sum().reset_index(drop=False)

# sc_datathon["datathon_ac_score"]=(sc_datathon["accomplish"]/8)*100
# sc_datathon["StudentId"]=sc_datathon["StudentId"].astype("int32")

In [ ]:
# sc_datathon.head()

In [ ]:
# fnsc=pd.read_excel("eeclass_score.xlsx")
# fnsc=fnsc[["帳號","總成績"]].rename(columns={"帳號":"Username","總成績":"final_score"})
# fnsc=fnsc.drop(32, axis=0)
# fnsc["Username"]=fnsc["Username"].astype("int32")

In [ ]:
# fnsc.describe()

In [ ]:
# fnsc.head()

### member_week_inclass notebook

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

li="Username SectionName SectionNumber Submitted SectionScore CodeLength_final RemarkLength_final EditTime_Inclass EditTime_Notinclass ExternalPasteLength CopyCt PasteCt".split()
sql='''
select `{}` from `edm_backup`.`member_class` where `SectionType`='practice' or `SectionType`='lesson';
'''.format("`,`".join(li))
cursor=conn.cursor()
row=cursor.execute(sql)
mwp=cursor.fetchall()
cursor.close()

df=pd.DataFrame()
for i in range(len(mwp)):
    dfx=pd.DataFrame(list(mwp[i])).T
    df=pd.concat([df, dfx], axis=0)
df.columns=li

In [ ]:
df["No_submitted"]=pd.isnull(df['Submitted'])
df["Late_submitted"]=df['Submitted']=="遲交"

del df["Submitted"]

In [ ]:
df=df.reset_index(drop=True)

In [ ]:
#ExecutionCt

df["ExecutionCt"]=0
sns=list(set(df["SectionName"]))
uss=list(set(df["Username"]))

for sn in sns:
    for us in uss:
        sql='''
        select count(*) from `edm_backup`.`edm_log` where `notebookName`='{}' and `Username`='{}' and `event`='code_execution';
        '''.format(sn+".ipynb", us)
        cursor=conn.cursor()
        row=cursor.execute(sql)
        c=cursor.fetchall()
        cursor.close()
        ind=df[(df["Username"]==us)&(df["SectionName"]==sn)].index[0]
        df.loc[ind, "ExecutionCt"]=c[0][0]

In [ ]:
df.head()

#### error系列

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
sql='''
select * from `edm_backup`.`error`;'''
cursor=conn.cursor()
rows=cursor.execute(sql)
error=cursor.fetchall()
cursor.close()
li="error_id Username NotebookName KernelId CellId ErrorName ErrorValue Timestamp Timespent Trycount Solved WeekNumber SectionNumber".split()
df_error=pd.DataFrame()
for i in range(len(error)):
    dfx=pd.DataFrame(list(error[i])).T
    df_error=pd.concat([df_error, dfx], axis=0)
df_error.columns=li

In [ ]:
df_error["SectionName"]=list(map(lambda x: x.split(".")[0], list(df_error["NotebookName"])))

In [ ]:
df_error.head(10)

In [ ]:
section_list=list(set(df["SectionName"]))

df_error["InClassSection"]=list(map(lambda x: 1 if x in section_list else 0, df_error["SectionName"]))
df_error_outclass=df_error[df_error["InClassSection"]==0]

print("error個數：",len(df_error))
print("非課內error個數：",len(df_error_outclass))

In [ ]:
li="SyntaxError NameError AttributeError TypeError KeyError ValueError IndentationError IndexError".split()
for i in li:
    df[i+"_Ct"]=[0]*len(df)
    df[i+"_FailfixCt"]=[0]*len(df)
    df[i+"_Fixtime"]=[np.nan]*len(df)
#     del df[i+"_Failfixperc"]

In [ ]:
df.head(1)

In [ ]:
df_error1=df_error[["SectionName", "Username", "ErrorName","error_id"]].groupby(["SectionName", "Username", "ErrorName"]).count()
df_error1=df_error1.reset_index(drop=False)
df_error1.head()

In [ ]:
df_error2=df_error[["SectionName", "Username", "ErrorName","Solved"]].groupby(["SectionName", "Username", "ErrorName"]).sum()
df_error2=df_error2.reset_index(drop=False)
df_error2.head()

In [ ]:
df_error["Timespent"]=list(map(lambda x: int(x), df_error["Timespent"]))
df_error3=df_error[df_error["Solved"]==1]
df_error3["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error3["Timespent"]))
df_error3=df_error3[["SectionName", "Username", "ErrorName","Timespent"]].groupby(["SectionName", "Username", "ErrorName"]).mean()
df_error3=df_error3.reset_index(drop=False)
df_error3.head()

In [ ]:
df_error1=pd.merge(df_error1, df_error2, on=["SectionName","Username","ErrorName"], how="left")
df_error1=pd.merge(df_error1, df_error3, on=["SectionName","Username","ErrorName"], how="left")

In [ ]:
# df_error1["failfixperc"]=1-(df_error1["Solved"]/df_error1["error_id"])
df_error1["failfixct"]=df_error1["error_id"]-df_error1["Solved"]

In [ ]:
df_error1.head(30)

In [ ]:
# sns=list(set(df["SectionName"]))
# uss=list(set(df["Username"]))

for er in li:
    print("=============="+er+"===============")
    dfx=df_error1[df_error1["ErrorName"]==er]
    print(len(dfx))
    for sn in sns:
        print("=="+sn+"==")
        for us in uss:
            dfx1=dfx[(dfx["SectionName"]==sn)&(dfx["Username"]==str(us))]
            if len(dfx1)!=0:
                ct=int(dfx1.loc[:,"error_id"])
                fp=float(dfx1.loc[:,"failfixct"])
                ts=float(dfx1.loc[:,"Timespent"])
                ind=df[(df["SectionName"]==sn)&(df["Username"]==us)].index[0]
                df.loc[ind, er+"_Ct"]=ct
                df.loc[ind, er+"_FailfixCt"]=fp
                df.loc[ind, er+"_Fixtime"]=ts
                print(str(us)+"-->"+str(ct)+", "+str(fp)+", "+str(ts))

In [ ]:
df_error4=df_error[df_error["Solved"]==1]
df_error4["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error4["Timespent"]))
df_error4=df_error4[["SectionName", "Username", "Timespent"]].groupby(["SectionName", "Username"]).sum()
df_error4=df_error4.reset_index(drop=False)
df_error4=df_error4.rename(columns={"Timespent":"DebugTime"})
df_error4.head()

In [ ]:
df=pd.merge(df, df_error4, on=["SectionName","Username"], how="left")

In [ ]:
df.head(50)

#### (不使用) pattern系列_from log 

In [ ]:
# import re
# from tqdm import tqdm

In [ ]:
# conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

# sql='''select `log_id`,`Username`,`notebookName`,`CleanCode` from `edm_backup`.`edm_log`;'''
# cursor=conn.cursor()
# row=cursor.execute(sql)
# lg=cursor.fetchall()
# cursor.close()

In [ ]:
# Log_id=[]
# Username=[]
# NotebookName=[]
# CleanCode=[]
# for i in range(len(lg)):
#     Log_id.append(lg[i][0])
#     Username.append(lg[i][1])
#     NotebookName.append(lg[i][2])
#     CleanCode.append(lg[i][3])

# df_logcd=pd.DataFrame({"Log_id":Log_id,"Username":Username,"NotebookName":NotebookName,"CleanCode":CleanCode})

In [ ]:
# df_logcd["CleanCode"]=df_logcd["CleanCode"].fillna("")
# df_logcd.head(30)

In [ ]:
# pattern_SimpleCondition_logct=[]
# pattern_MultipleCondition_logct=[]
# pattern_WhileIteration_logct=[]
# pattern_ForInIteration_logct=[]
# pattern_NestIteration_logct=[]
# pattern_DefFunction_logct=[]

# print("total:", len(df_logcd))
# for xxx in range(len(df_logcd)):
#     print(xxx)
#     x=list(df_logcd["CleanCode"])[xxx]
#     #MultipleCondition
#     target=copy.deepcopy(x)
#     mc=0
#     for i in list(reversed(range(6))):
#         #print("========"+str(i)+"========")
#         c="[ ]{%d}if.*:.*\n([ ]{%d}.*\n)+?([ ]{%d}elif.*:.*\n)+?([ ]{%d}.*)([ ]{%d}else:.*\n)+?([ ]{%d}.*)"%(4*i, 4*i+4,4*i, 4*i+4,4*i, 4*i+4)
#         c1="[ ]{%d}if.*:.*\n([ ]{%d}.*\n)+?([ ]{%d}elif.*:.*\n)+?([ ]{%d}.*)"%(4*i, 4*i+4,4*i, 4*i+4)
#         c2="[ ]{%d}if.*:.*\n([ ]{%d}.*\n)+?([ ]{%d}else:.*\n)+?([ ]{%d}.*)"%(4*i, 4*i+4,4*i, 4*i+4)
#         for j in re.finditer(c,target):
#             target=target.replace(j.group(),"")
#             mc+=1
#         for j in re.finditer(c1,target):
#             target=target.replace(j.group(),"")
#             mc+=1
#         for j in re.finditer(c2,target):
#             target=target.replace(j.group(),"")
#             mc+=1
            
#     #SimpleCondition
#     c="if .+:.*\n"
#     sc=len(list(re.finditer(c,target)))

#     #WhileIteration
#     c="while .+:.*\n"
#     wi=len(list(re.finditer(c,x)))

#     #ForInIteration
#     c="for .+ in .+:.*\n"
#     fi=len(list(re.finditer(c,x)))

#     #NestIteration
#     ni=0
#     for i in range(6):
#         #print("======="+str(i)+"=======")
#         c="[ ]{%d}((while .+:)|(for .+ in .+:)|(if .+:)).*\n([ ]{%d}.*\n)+"%(4*i, 4*i+4)
#         for j in re.finditer(c,x):
#             x1=j.group().replace("\n"+" "*(4*i), "\n").strip()
#             c1="((while .+:)|(for .+ in .+:|(if .+:))).*\n([ ]{4}.*\n)"
#             ni+=len(list(re.finditer(c1,x1)))
                
#     #DefFunction
#     c="def .+\(.*\):"
#     fc=len(list(re.finditer(c,x)))
    
#     pattern_SimpleCondition_logct.append(sc)
#     pattern_MultipleCondition_logct.append(mc)
#     pattern_WhileIteration_logct.append(wi)
#     pattern_ForInIteration_logct.append(fi)
#     pattern_NestIteration_logct.append(ni)
#     pattern_DefFunction_logct.append(fc)

In [ ]:
# df_logcd["pattern_SimpleCondition_logct"]=pattern_SimpleCondition_logct
# df_logcd["pattern_MultipleCondition_logct"]=pattern_MultipleCondition_logct
# df_logcd["pattern_WhileIteration_logct"]=pattern_WhileIteration_logct
# df_logcd["pattern_ForInIteration_logct"]=pattern_ForInIteration_logct
# df_logcd["pattern_NestIteration_logct"]=pattern_NestIteration_logct
# df_logcd["pattern_DefFunction_logct"]=pattern_DefFunction_logct

In [ ]:
# df_logcd.head(30)

In [ ]:
# df_logcd["SectionName"]=list(map(lambda x: x.split(".")[0], df_logcd["NotebookName"]))
# df_logcd1=df_logcd.groupby(["Username","SectionName"]).sum().reset_index(drop=False)
# del df_logcd1["Log_id"]

In [ ]:
# df["Username"]=list(map(lambda x: str(x), df["Username"]))
# df=pd.merge(df, df_logcd1, on=["Username","SectionName"], how="left")

In [ ]:
# df.head(100)

#### pattern系列_from notebook

In [ ]:
import re
from tqdm import tqdm

In [ ]:
#conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")

sql='''select `Username`,`SectionName`,`CodeContent_final` from `edm_backup`.`member_class` where SectionType='practice' or SectionType='lesson';'''
cursor=conn.cursor()
row=cursor.execute(sql)
cc=cursor.fetchall()
cursor.close()

Username=[]
SectionName=[]
CodeContent_final=[]
for i in range(len(cc)):
    Username.append(cc[i][0])
    SectionName.append(cc[i][1])
    CodeContent_final.append(cc[i][2])

df_sc=pd.DataFrame({"Username":Username,"SectionName":SectionName,"CodeContent_final":CodeContent_final})

In [ ]:
df_sc["CodeContent_final"]=df_sc["CodeContent_final"].fillna("")
df_sc.head()

In [ ]:
#df_sc.loc[98,"CodeContent_final"]

In [ ]:
pattern_SimpleCondition_nbct=[]
pattern_MultipleCondition_nbct=[]
pattern_WhileIteration_nbct=[]
pattern_ForInIteration_nbct=[]
pattern_NestIteration_nbct=[]
pattern_DefFunction_nbct=[]

print("total:", len(df_sc))
for xxx in range(len(df_sc)):
    print(xxx)
    x=list(df_sc["CodeContent_final"])[xxx]
    
    #MultipleCondition
    target=copy.deepcopy(x)
    mc=0
    for i in list(reversed(range(6))):
        #print("========"+str(i)+"========")
        c="[ ]{%d}if.*:.*\n([ ]{%d}.*\n)+?([ ]{%d}elif.*:.*\n)+?([ ]{%d}.*)([ ]{%d}else:.*\n)+?([ ]{%d}.*)"%(4*i, 4*i+4,4*i, 4*i+4,4*i, 4*i+4)
        c1="[ ]{%d}if.*:.*\n([ ]{%d}.*\n)+?([ ]{%d}elif.*:.*\n)+?([ ]{%d}.*)"%(4*i, 4*i+4,4*i, 4*i+4)
        c2="[ ]{%d}if.*:.*\n([ ]{%d}.*\n)+?([ ]{%d}else:.*\n)+?([ ]{%d}.*)"%(4*i, 4*i+4,4*i, 4*i+4)
        for j in re.finditer(c,target):
            target=target.replace(j.group(),"")
            mc+=1
        for j in re.finditer(c1,target):
            target=target.replace(j.group(),"")
            mc+=1
        for j in re.finditer(c2,target):
            target=target.replace(j.group(),"")
            mc+=1
            
    #SimpleCondition
    c="if .+:.*\n"
    sc=len(list(re.finditer(c,target)))

    #WhileIteration
    c="while .+:.*\n"
    wi=len(list(re.finditer(c,x)))

    #ForInIteration
    c="for .+ in .+:.*\n"
    fi=len(list(re.finditer(c,x)))

    #NestIteration
    ni=0
    for i in range(6):
        #print("======="+str(i)+"=======")
        c="[ ]{%d}((while .+:)|(for .+ in .+:)|(if .+:)).*\n([ ]{%d}.*\n)+"%(4*i, 4*i+4)
        for j in re.finditer(c,x):
            x1=j.group().replace("\n"+" "*(4*i), "\n").strip()
            c1="((while .+:)|(for .+ in .+:|(if .+:))).*\n([ ]{4}.*\n)"
            ni+=len(list(re.finditer(c1,x1)))
                
    #DefFunction
    c="def .+\(.*\):"
    fc=len(list(re.finditer(c,x)))
    
    pattern_SimpleCondition_nbct.append(sc)
    pattern_MultipleCondition_nbct.append(mc)
    pattern_WhileIteration_nbct.append(wi)
    pattern_ForInIteration_nbct.append(fi)
    pattern_NestIteration_nbct.append(ni)
    pattern_DefFunction_nbct.append(fc)

In [ ]:
df_sc["pattern_SimpleCondition_nbct"]=pattern_SimpleCondition_nbct
df_sc["pattern_MultipleCondition_nbct"]=pattern_MultipleCondition_nbct
df_sc["pattern_WhileIteration_nbct"]=pattern_WhileIteration_nbct
df_sc["pattern_ForInIteration_nbct"]=pattern_ForInIteration_nbct
df_sc["pattern_NestIteration_nbct"]=pattern_NestIteration_nbct
df_sc["pattern_DefFunction_nbct"]=pattern_DefFunction_nbct

In [ ]:
df_sc["Username"]=list(map(lambda x: str(x), df_sc["Username"]))
df["Username"]=list(map(lambda x: str(x), df["Username"]))

In [ ]:
df=pd.merge(df, df_sc, on=["Username","SectionName"], how="left")
del df["CodeContent_final"]

In [ ]:
df.head()

#### (不使用) modulemethod系列_from log

In [ ]:
# df_logcd=df_logcd[["Log_id","Username","NotebookName","CleanCode"]]
# df_logcd.head()

In [ ]:
# Username=[]
# NotebookName=[]
# Codemix=[]
# for usr in list(set(df_logcd["Username"])):
#     df_logcdx=df_logcd[df_logcd["Username"]==usr]
#     for nb in list(set(df_logcdx["NotebookName"])):
#         df_logcdxx=df_logcdx[df_logcdx["NotebookName"]==nb]
#         cdmix="\n".join(list(df_logcdxx["CleanCode"]))
#         Username.append(usr)
#         NotebookName.append(nb)
#         Codemix.append(cdmix)
# df_cdmix=pd.DataFrame({"Username":Username,"NotebookName":NotebookName,"Codemix":Codemix})

In [ ]:
# Module=[]
# Module_ct=[]
# for cd in Codemix:
#     m=[]

#     r="import .+ as .+"
#     r1="import .+"
#     for i in re.finditer(r,cd):
#         mm=i.group().replace("import ","").split(" as ")[0].split(".")[0]
#         if mm not in m:
#             m.append(mm)
#         cd=cd.replace(i.group(),"")
#     for i in re.finditer(r1,cd):
#         mm=i.group().replace("import ","")
#         if "," in mm:
#             mm=list(map(lambda x : x.strip(), mm.split(",")))
#             for mmx in mm:
#                 if mmx not in m:
#                     m.append(mmx.split(".")[0])
#         else:
#             if mm not in m:
#                 m.append(mm.split(".")[0].strip())
#         m=list(set(m))
#     Module_ct.append(len(m))
#     Module.append(" ".join(m))

In [ ]:
# df_cdmix["Module_ct"]=Module_ct
# df_cdmix["Module"]=Module

In [ ]:
# df_cdmix["SectionName"]=list(map(lambda x: x.split(".")[0], df_cdmix["NotebookName"]))

In [ ]:
# df=pd.merge(df, df_cdmix[["Username","SectionName","Module_ct","Module"]], 
#             on=["Username","SectionName"], how="left")

In [ ]:
# modulemethod=[]
# modulemethod.extend(dir(np.array([1,2,3])))
# modulemethod.extend(dir(pd.DataFrame()))
# modulemethod.extend(dir(pd.Series()))
# modulemethod=list(set(modulemethod))

In [ ]:
# len(modulemethod)

In [ ]:
# set(dir(np.array([1,2,3]))) & set(dir(pd.DataFrame()))

In [ ]:
# #不同套件中objects的方法會有重複，考量同名方法的功能差異不大，在此視為同一種方法。
# modulemethod_ct=[]
# modulemethod_log_usect=[]
# modulemethod_log_use=[]

# for cd in Codemix:
#     mc=0
#     mluc=0
#     mlu=""

#     for mth in dir(np):
#         if len(list(re.finditer("np\."+mth,cd)))!=0:
#             mc+=1
#             mlu+=list(re.finditer("np\."+mth,cd))[0].group()+" "
#             mluc+=len(list(re.finditer("np\."+mth, cd)))
#     for mth in dir(pd):
#         if len(list(re.finditer("pd\."+mth,cd)))!=0:
#             mc+=1
#             mlu+=list(re.finditer("pd\."+mth,cd))[0].group()+" "
#             mluc+=len(list(re.finditer("pd\."+mth,cd)))
#     for mth in dir(plt):
#         if len(list(re.finditer("((plt)|(pt))\."+mth,cd)))!=0:
#             mc+=1
#             mlu+=list(re.finditer("((plt)|(pt))\."+mth,cd))[0].group()+" "
#             mluc+=len(list(re.finditer("((plt)|(pt))\."+mth,cd)))
#     for mth in modulemethod:
#         if len(list(re.finditer("\."+mth,cd)))!=0:
#             mc+=1
#             mlu+=list(re.finditer("\."+mth,cd))[0].group()+" "
#             mluc+=len(list(re.finditer("\."+mth,cd)))
#     modulemethod_ct.append(mc)
#     modulemethod_log_usect.append(mluc)
#     modulemethod_log_use.append(mlu)

In [ ]:
# df_cdmix["modulemethod_ct"]=modulemethod_ct
# df_cdmix["modulemethod_log_usect"]=modulemethod_log_usect
# df_cdmix["modulemethod_log_use"]=modulemethod_log_use

In [ ]:
# df_cdmix

In [ ]:
# df=pd.merge(df, df_cdmix[["Username","SectionName","modulemethod_ct","modulemethod_log_usect","modulemethod_log_use"]], 
#             on=["Username","SectionName"], how="left")

#### modulemethod系列_from notebook

In [ ]:
modulemethod=[]
modulemethod.extend(dir(np.array([1,2,3])))
modulemethod.extend(dir(pd.DataFrame()))
modulemethod.extend(dir(pd.Series()))
modulemethod=list(set(modulemethod))

In [ ]:
len(modulemethod)

In [ ]:
Module=[]
Module_ct=[]
modulemethod_ct=[]
modulemethod_nb_use=[]
modulemethod_nb_usect=[]


for cd in list(df_sc["CodeContent_final"]):
    mnuc=0
    mnu=""

    for mth in dir(np):
        if len(list(re.finditer("np\."+mth,cd)))!=0:
            mnu+=list(re.finditer("np\."+mth,cd))[0].group()+" "
            mnuc+=len(list(re.finditer("np\."+mth, cd)))
    for mth in dir(pd):
        if len(list(re.finditer("pd\."+mth,cd)))!=0:
            mnu+=list(re.finditer("pd\."+mth,cd))[0].group()+" "
            mnuc+=len(list(re.finditer("pd\."+mth,cd)))
    for mth in dir(plt):
        if len(list(re.finditer("((plt)|(pt))\."+mth,cd)))!=0:
            mnu+=list(re.finditer("((plt)|(pt))\."+mth,cd))[0].group()+" "
            mnuc+=len(list(re.finditer("((plt)|(pt))\."+mth,cd)))
    for mth in modulemethod:
        if len(list(re.finditer("\."+mth,cd)))!=0:
            mnu+=list(re.finditer("\."+mth,cd))[0].group()+" "
            mnuc+=len(list(re.finditer("\."+mth,cd)))
    
    modulemethod_ct.append(len(mnu.split()))
    modulemethod_nb_usect.append(mnuc)
    modulemethod_nb_use.append(mnu)
    
    m=[]
    r="import .+ as .+"
    r1="import .+"
    for i in re.finditer(r,cd):
        mm=i.group().replace("import ","").split(" as ")[0].split(".")[0]
        if mm not in m:
            m.append(mm)
        cd=cd.replace(i.group(),"")
    for i in re.finditer(r1,cd):
        mm=i.group().replace("import ","")
        if "," in mm:
            mm=list(map(lambda x : x.strip(), mm.split(",")))
            for mmx in mm:
                if mmx not in m:
                    m.append(mmx.split(".")[0])
        else:
            if mm not in m:
                m.append(mm.split(".")[0].strip())
        m=list(set(m))
    Module_ct.append(len(m))
    Module.append(" ".join(m))

In [ ]:
df_sc["modulemethod_nb_usect"]=modulemethod_nb_usect
df_sc["modulemethod_nb_use"]=modulemethod_nb_use
df_sc["modulemethod_ct"]=modulemethod_ct
df_sc["Module_ct"]=Module_ct
df_sc["Module"]=Module

In [ ]:
df=pd.merge(df, df_sc[["Username","SectionName",
                       "modulemethod_nb_usect","modulemethod_nb_use","modulemethod_ct",
                      "Module_ct","Module"]], on=["Username","SectionName"], how="left")

In [ ]:
list(df.columns)

#### (不使用) basicmethod系列_from log

In [ ]:
# #str(), int(), float(), list(), tuple(), dict(), set()
# basicmethod=[]
# basicmethod.extend(dir(str()))
# basicmethod.extend(dir(int()))
# basicmethod.extend(dir(float()))
# basicmethod.extend(dir(list()))
# basicmethod.extend(dir(tuple()))
# basicmethod.extend(dir(dict()))
# basicmethod.extend(dir(set()))
# basicmethod=list(set(basicmethod))

In [ ]:
# basicmethod

In [ ]:
# basicmethod_ct=[]
# basicmethod_log_usect=[]
# basicmethod_log_use=[]

# for cd in Codemix:
#     mc=0
#     mluc=0
#     mlu=""
#     for mth in basicmethod:
#         if len(list(re.finditer("\."+mth,cd)))!=0:
#             mc+=1
#             mlu+=list(re.finditer("\."+mth,cd))[0].group()+" "
#             mluc+=len(list(re.finditer("\."+mth,cd)))
#     basicmethod_ct.append(mc)
#     basicmethod_log_usect.append(mluc)
#     basicmethod_log_use.append(mlu)

In [ ]:
# df_cdmix["basicmethod_ct"]=basicmethod_ct
# df_cdmix["basicmethod_log_usect"]=basicmethod_log_usect
# df_cdmix["basicmethod_log_use"]=basicmethod_log_use

In [ ]:
# df_cdmix.head()

In [ ]:
# df=pd.merge(df, df_cdmix[["Username","SectionName","basicmethod_ct","basicmethod_log_usect","basicmethod_log_use"]], 
#             on=["Username","SectionName"], how="left")

#### basicmethod系列_from notebook

In [ ]:
#str(), int(), float(), list(), tuple(), dict(), set()
basicmethod=[]
basicmethod.extend(dir(str()))
basicmethod.extend(dir(int()))
basicmethod.extend(dir(float()))
basicmethod.extend(dir(list()))
basicmethod.extend(dir(tuple()))
basicmethod.extend(dir(dict()))
basicmethod.extend(dir(set()))
basicmethod=list(set(basicmethod))

In [ ]:
basicmethod_ct=[]
basicmethod_nb_use=[]
basicmethod_nb_usect=[]

for cd in list(df_sc["CodeContent_final"]):
    mnuc=0
    mnu=""
    for mth in basicmethod:
        if len(list(re.finditer("\."+mth,cd)))!=0:
            mnu+=list(re.finditer("\."+mth,cd))[0].group()+" "
            mnuc+=len(list(re.finditer("\."+mth,cd)))
    basicmethod_ct.append(len(mnu.split()))
    basicmethod_nb_usect.append(mnuc)
    basicmethod_nb_use.append(mnu)

In [ ]:
df_sc["basicmethod_ct"]=basicmethod_ct
df_sc["basicmethod_nb_usect"]=basicmethod_nb_usect
df_sc["basicmethod_nb_use"]=basicmethod_nb_use

In [ ]:
df=pd.merge(df, df_sc[["Username","SectionName","basicmethod_nb_usect","basicmethod_nb_use","basicmethod_ct"]], 
            on=["Username","SectionName"], how="left")

In [ ]:
list(df.columns)

#### 累計系列

In [ ]:
df=df.sort_values(["Username","SectionNumber"]).reset_index(drop=True)

In [ ]:
df["Grand_Module_ct"]=[0]*len(df)
df["Grand_modulemethod_ct"]=[0]*len(df)

for usr in list(set(df["Username"])):
    print(usr)
    dfx=df[df["Username"]==usr]
    mm=[]
    m=[]
    for i in dfx.index:
        m.extend(df.loc[i,"Module"].split())
        mm.extend(df.loc[i,"modulemethod_nb_use"].split())
        df.loc[i,"Grand_Module_ct"]=len(set(m))
        df.loc[i,"Grand_modulemethod_ct"]=len(set(mm))

In [ ]:
df["RemarkLength_final"]=df["RemarkLength_final"].fillna(0)
df["Grand_RemarkLength"]=[0]*len(df)

for usr in list(set(df["Username"])):
    print(usr)
    dfx=df[df["Username"]==usr]
    rm=0
    for i in dfx.index:
        rm+=df.loc[i,"RemarkLength_final"]
        df.loc[i,"Grand_RemarkLength"]=rm

In [ ]:
df["Grand_basicmethod_ct"]=[0]*len(df)

for usr in list(set(df["Username"])):
    print(usr)
    dfx=df[df["Username"]==usr]
    bm=[]
    for i in dfx.index:
        bm.extend(df.loc[i,"basicmethod_nb_use"].split())
        df.loc[i,"Grand_basicmethod_ct"]=len(set(bm))

#### 其他

In [ ]:
# #WeekNumber

sc=pd.read_excel("section.xlsx")
d={}
for i in sc.index:
    d.update({sc.loc[i,"SectionName"]:sc.loc[i,"WeekNumber"]})

df["WeekNumber"]=df["SectionName"].map(d)

In [ ]:
# list(df.columns)

In [ ]:
df["Username"]=df["Username"].astype("int32")
df=pd.merge(df, qs, left_on="Username", right_on="id", how="left").drop("id", axis=1)

In [ ]:
list(df.columns)

In [ ]:
df.to_excel("member_section_trainingdata.xlsx")
# df=pd.read_excel("member_section_trainingdata.xlsx")
# if "Unnamed: 0" in df.columns:
#     df=df.drop("Unnamed: 0", axis=1)

#### 週次合併版本

In [ ]:
df["Module"]=df["Module"].fillna("")
df["modulemethod_nb_use"]=df["modulemethod_nb_use"].fillna("")
df["basicmethod_nb_use"]=df["basicmethod_nb_use"].fillna("")

In [ ]:
df1=df[['Username', 'WeekNumber','Module','Module_ct',
        'modulemethod_ct','modulemethod_nb_use',
        'basicmethod_ct','basicmethod_nb_use']]
df1.head()

In [ ]:
Username=[]
WeekNumber=[]
Module=[]
Module_ct=[]
Modulemethod_use=[]
Modulemethod_ct=[]
Basicmethod_use=[]
Basicmethod_ct=[]
Grand_Module_ct=[]
Grand_modulemethod_ct=[]
Grand_basicmethod_ct=[]

for usr in list(set(df1["Username"])):
    df1x=df1[df1["Username"]==usr]
    grand_m=0
    grand_mm=0
    grand_bm=0
    for wk in list(set(df1x["WeekNumber"])):
        df1xx=df1x[df1x["WeekNumber"]==wk]
        module=list(set(" ".join(list(df1xx["Module"])).split()))
        modulemethod_use=list(set(" ".join(list(df1xx["modulemethod_nb_use"])).split()))
        basicmethod_use=list(set(" ".join(list(df1xx["basicmethod_nb_use"])).split()))
        grand_m+=len(module)
        grand_mm+=len(modulemethod_use)
        grand_bm+=len(basicmethod_use)
        
        Username.append(usr)
        WeekNumber.append(wk)
        Module.append(" ".join(module))
        Module_ct.append(len(module))
        Modulemethod_use.append(" ".join(modulemethod_use))
        Modulemethod_ct.append(len(modulemethod_use))
        Basicmethod_use.append(" ".join(basicmethod_use))
        Basicmethod_ct.append(len(basicmethod_use))
        Grand_Module_ct.append(grand_m)
        Grand_modulemethod_ct.append(grand_mm)
        Grand_basicmethod_ct.append(grand_bm)

df1=pd.DataFrame({
    "Username":Username,
    "WeekNumber":WeekNumber,
    "Module":Module,
    "Module_ct":Module_ct,
    "Grand_Module_ct":Grand_Module_ct,
    "Modulemethod_use":Modulemethod_use,
    "Modulemethod_ct":Modulemethod_ct,
    "Grand_modulemethod_ct":Grand_modulemethod_ct,
    "Basicmethod_use":Basicmethod_use,
    "Basicmethod_ct":Basicmethod_ct,
    "Grand_basicmethod_ct":Grand_basicmethod_ct})

In [ ]:
df1.head()

In [ ]:
sc=pd.read_excel("section.xlsx")
d={}
for i in sc.index:
    d.update({sc.loc[i,"SectionName"]:sc.loc[i,"WeekNumber"]})

df_error["WeekNumber"]=df_error["SectionName"].map(d)

In [ ]:
#df_error["Timespent"]=df_error["Timespent"].astype("int32")

df_error3=df_error[df_error["Solved"]==1]
df_error3["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error3["Timespent"]))
df_error3=df_error3[["WeekNumber", "Username", "ErrorName","Timespent"]].groupby(["WeekNumber", "Username", "ErrorName"]).mean()
df_error3=df_error3.reset_index(drop=False)
df_error3.head()

In [ ]:
sns=list(set(df["SectionName"]))
uss=list(set(df["Username"]))

df2=df[['Username', 'WeekNumber']].drop_duplicates(keep="last").reset_index(drop=True)
li="SyntaxError NameError AttributeError TypeError KeyError ValueError IndentationError IndexError".split()
week_list=list(set(df_error3["WeekNumber"]))
for i in li:
    df2[i+"_Fixtime"]=[np.nan]*len(df2)
for er in li:
    print("=============="+er+"===============")
    dfx=df_error3[df_error3["ErrorName"]==er]
    for wk in week_list:
        print("=="+str(wk)+"==")
        for us in uss:
            dfx1=dfx[(dfx["WeekNumber"]==wk)&(dfx["Username"]==str(us))]
            if len(dfx1)!=0:
                ts=float(dfx1.loc[:,"Timespent"])
                ind=df2[(df2["WeekNumber"]==wk)&(df2["Username"]==us)].index[0]
                df2.loc[ind, er+"_Fixtime"]=ts

In [ ]:
df2.head()

In [ ]:
#.sum()
df["CodeLength_final"]=df["CodeLength_final"].fillna(0).astype("int32")
df['SectionScore']=df['SectionScore'].fillna(0).astype("int32")
df["CopyCt"]=df["CopyCt"].fillna(0).astype("int32")
df["EditTime_Inclass"]=df["EditTime_Inclass"].fillna(0).astype("int32")
df["EditTime_Notinclass"]=df["EditTime_Notinclass"].fillna(0).astype("int32")
df["ExternalPasteLength"]=df["ExternalPasteLength"].fillna(0).astype("int32")
df["PasteCt"]=df["PasteCt"].fillna(0).astype("int32")

df3=df[['Username',
 'WeekNumber',
 'CodeLength_final',
 'RemarkLength_final',
 'Grand_RemarkLength',
 'EditTime_Inclass', 'EditTime_Notinclass',
 'ExternalPasteLength',
 'ExecutionCt', 'CopyCt', 'PasteCt',
 'No_submitted', 'Late_submitted',
 'SyntaxError_Ct', 'NameError_Ct', 'AttributeError_Ct', 'TypeError_Ct',
 'KeyError_Ct', 'ValueError_Ct','IndentationError_Ct', 'IndexError_Ct',
 'SyntaxError_FailfixCt', 'NameError_FailfixCt', 'AttributeError_FailfixCt', 'TypeError_FailfixCt',
 'KeyError_FailfixCt', 'ValueError_FailfixCt', 'IndentationError_FailfixCt', 'IndexError_FailfixCt',
 'pattern_SimpleCondition_nbct', 'pattern_MultipleCondition_nbct', 'pattern_WhileIteration_nbct',
 'pattern_ForInIteration_nbct', 'pattern_NestIteration_nbct', 'pattern_DefFunction_nbct',
 'modulemethod_nb_usect','basicmethod_nb_usect','DebugTime'    
       ]].groupby(['Username', 'WeekNumber']).sum().reset_index(drop=False)

In [ ]:
df4=df[['Username', 'WeekNumber', 'SectionScore']].groupby(['Username', 'WeekNumber']).mean().reset_index(drop=False)

In [ ]:
df1=pd.merge(df1, df2, on=['Username','WeekNumber'], how="left")
df1=pd.merge(df1, df3, on=['Username','WeekNumber'], how="left")
df1=pd.merge(df1, df4, on=['Username','WeekNumber'], how="left")
df1=pd.merge(df1, qs, left_on=['Username'], right_on=["id"], how="left").drop("id", axis=1)

In [ ]:
set(list(df.columns))-set(list(df1.columns))

In [ ]:
for usr in list(set(df1["Username"])):
    print(usr)
    dfx=df1[df1["Username"]==usr]
    rm=0
    for i in dfx.index:
        rm+=df1.loc[i,"RemarkLength_final"]
        df1.loc[i,"Grand_RemarkLength"]=rm

In [ ]:
list(df1.columns)

In [ ]:
df1=df1[['Username',
 'WeekNumber',
 'SectionScore',
 'Module',
 'Module_ct',
 'Grand_Module_ct',
 'Modulemethod_use',
 'Modulemethod_ct',
 'Grand_modulemethod_ct',
 'modulemethod_nb_usect',
 'Basicmethod_use',
 'Basicmethod_ct',
 'Grand_basicmethod_ct',
 'basicmethod_nb_usect',
 'SyntaxError_Fixtime',
 'NameError_Fixtime',
 'AttributeError_Fixtime',
 'TypeError_Fixtime',
 'KeyError_Fixtime',
 'ValueError_Fixtime',
 'IndentationError_Fixtime',
 'IndexError_Fixtime',
 'CodeLength_final',
 'RemarkLength_final',
 'Grand_RemarkLength',
 'EditTime_Inclass',
 'EditTime_Notinclass',
 'DebugTime',
 'ExternalPasteLength',
 'ExecutionCt',
 'CopyCt',
 'PasteCt',
 'No_submitted',
 'Late_submitted',
 'SyntaxError_Ct',
 'NameError_Ct',
 'AttributeError_Ct',
 'TypeError_Ct',
 'KeyError_Ct',
 'ValueError_Ct',
 'IndentationError_Ct',
 'IndexError_Ct',
 'SyntaxError_FailfixCt',
 'NameError_FailfixCt',
 'AttributeError_FailfixCt',
 'TypeError_FailfixCt',
 'KeyError_FailfixCt',
 'ValueError_FailfixCt',
 'IndentationError_FailfixCt',
 'IndexError_FailfixCt',
 'pattern_SimpleCondition_nbct',
 'pattern_MultipleCondition_nbct',
 'pattern_WhileIteration_nbct',
 'pattern_ForInIteration_nbct',
 'pattern_NestIteration_nbct',
 'pattern_DefFunction_nbct',
 'wk_expr',
 'non_py_expr',
 'non_py_prof',
 'py_expr',
 'py_prof',
 'pretest_score']]

In [ ]:
df1.shape

In [ ]:
df1.head()

In [ ]:
df1.to_excel("member_weeknb_trainingdata.xlsx")

### member_semester_inclass notebook

In [ ]:
df2=df1[['Username','Module', 'Modulemethod_use', 'Basicmethod_use']]
df2.head()

In [ ]:
Username=[]
Module=[]
Module_ct=[]
Modulemethod_use=[]
Modulemethod_ct=[]
Basicmethod_use=[]
Basicmethod_ct=[]

for usr in list(set(df2["Username"])):
    df1x=df2[df2["Username"]==usr]

    module=list(set(" ".join(list(df1x["Module"])).split()))
    modulemethod_use=list(set(" ".join(list(df1x["Modulemethod_use"])).split()))
    basicmethod_use=list(set(" ".join(list(df1x["Basicmethod_use"])).split()))

    Username.append(usr)
    Module.append(" ".join(module))
    Module_ct.append(len(module))
    Modulemethod_use.append(" ".join(modulemethod_use))
    Modulemethod_ct.append(len(modulemethod_use))
    Basicmethod_use.append(" ".join(basicmethod_use))
    Basicmethod_ct.append(len(basicmethod_use))

df2=pd.DataFrame({
    "Username":Username,
    "Module":Module,
    "Module_ct":Module_ct,
    "Modulemethod_use":Modulemethod_use,
    "Modulemethod_ct":Modulemethod_ct,
    "Basicmethod_use":Basicmethod_use,
    "Basicmethod_ct":Basicmethod_ct})

In [ ]:
df2.head()

In [ ]:
df_error3=df_error[(df_error["Solved"]==1)&(df_error["InClassSection"]==1)]
df_error3["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error3["Timespent"]))
df_error3=df_error3[["Username", "ErrorName","Timespent"]].groupby(["Username", "ErrorName"]).mean()
df_error3=df_error3.reset_index(drop=False)
df_error3.head()

In [ ]:
df3=df[['Username']].drop_duplicates(keep="last").reset_index(drop=True)
li="SyntaxError NameError AttributeError TypeError KeyError ValueError IndentationError IndexError".split()
for i in li:
    df3[i+"_Fixtime"]=[np.nan]*len(df2)
for er in li:
    print("=============="+er+"===============")
    dfx=df_error3[df_error3["ErrorName"]==er]
    for us in uss:
        dfx1=dfx[dfx["Username"]==str(us)]
        if len(dfx1)!=0:
            ts=float(dfx1.loc[:,"Timespent"])
            ind=df3[df3["Username"]==us].index[0]
            df3.loc[ind, er+"_Fixtime"]=ts

In [ ]:
df3.head()

In [ ]:
#.sum()
df4=df1[['Username',
 'CodeLength_final',
 'RemarkLength_final',
 'EditTime_Inclass', 'EditTime_Notinclass','DebugTime',
 'ExternalPasteLength',
 'ExecutionCt', 'CopyCt', 'PasteCt',
 'No_submitted', 'Late_submitted',
 'SyntaxError_Ct', 'NameError_Ct', 'AttributeError_Ct', 'TypeError_Ct',
 'KeyError_Ct', 'ValueError_Ct','IndentationError_Ct', 'IndexError_Ct',
 'SyntaxError_FailfixCt', 'NameError_FailfixCt', 'AttributeError_FailfixCt', 'TypeError_FailfixCt',
 'KeyError_FailfixCt', 'ValueError_FailfixCt', 'IndentationError_FailfixCt', 'IndexError_FailfixCt',
 'pattern_SimpleCondition_nbct', 'pattern_MultipleCondition_nbct', 'pattern_WhileIteration_nbct',
 'pattern_ForInIteration_nbct', 'pattern_NestIteration_nbct', 'pattern_DefFunction_nbct',
 'modulemethod_nb_usect','basicmethod_nb_usect'  
       ]].groupby(['Username']).sum().reset_index(drop=False)

In [ ]:
df5=df1[['Username', 'SectionScore']].groupby(['Username']).mean().reset_index(drop=False)

In [ ]:
df2=pd.merge(df2, df3, on=['Username'], how="left")
df2=pd.merge(df2, df4, on=['Username'], how="left")
df2=pd.merge(df2, df5, on=['Username'], how="left")
df2=pd.merge(df2, qs, left_on=['Username'], right_on=["id"], how="left").drop("id", axis=1)

In [ ]:
set(list(df1.columns))-set(list(df2.columns))

In [ ]:
# df2["Username"]=df2["Username"].astype("int32")
# df2=pd.merge(df2, fnsc, on="Username", how="left")#加入datathon成績做為可能的target
# df2=pd.merge(df2, sc_datathon[["StudentId","datathon_ac_score"]], left_on="Username", right_on="StudentId", how="left").drop("StudentId", axis=1)

In [ ]:
df2

In [ ]:
df2.to_excel("member_semesternb_trainingdata.xlsx")

### member_datathon_inclass notebook

In [ ]:
df_datathon=df[df["SectionName"]=="【Pract16】Datathon"].reset_index(drop=True)

In [ ]:
df_datathon.head()

In [ ]:
# practice_score=pd.read_excel("practice_score.xlsx")
# sc_datathon=practice_score[practice_score["SectionName"]=="【Pract16】Datathon"]
# sc_datathon=sc_datathon[["SectionName","StudentId","accomplish"]].groupby(["SectionName","StudentId"]).sum().reset_index(drop=False)

# sc_datathon["datathon_ac_score"]=(sc_datathon["accomplish"]/8)*100
# sc_datathon["StudentId"]=sc_datathon["StudentId"].astype("int32")

In [ ]:
# df_datathon["Username"]=df_datathon["Username"].astype("int32")
# df_datathon=pd.merge(df_datathon, sc_datathon, left_on=["Username","SectionName"], right_on=["StudentId","SectionName"], how="left")

In [ ]:
# df_datathon["Username"]=df_datathon["Username"].astype("int32")
# df_datathon=pd.merge(df_datathon, qs, left_on="Username", right_on="id", how="left").drop("id", axis=1)

In [ ]:
df_datathon.to_excel("member_datathon_trainingdata.xlsx")

### member_week_log

#### error系列

In [ ]:
df_error=df_error.reset_index(drop=True)

In [ ]:
df_error.head()

In [ ]:
s=((1, datetime(2020, 9, 16, 0, 0)),
 (2, datetime(2020, 9, 23, 0, 0)),
 (3, datetime(2020, 9, 30, 0, 0)),
 (4, datetime(2020, 10, 7, 0, 0)),
 (5, datetime(2020, 10, 14, 0, 0)),
 (6, datetime(2020, 10, 21, 0, 0)),
 (7, datetime(2020, 10, 28, 0, 0)),
 (8, datetime(2020, 11, 4, 0, 0)),
 (9, datetime(2020, 11, 11, 0, 0)),
 (11, datetime(2020, 11, 25, 0, 0)),
 (12, datetime(2020, 12, 2, 0, 0)),
 (13, datetime(2020, 12, 9, 0, 0)),
 (14, datetime(2020, 12, 16, 0, 0)),
 (15, datetime(2020, 12, 23, 0, 0)),
 (16, datetime(2020, 12, 30, 0, 0)))

weeknumber=[]
for i in range(len(df_error)):
    week=0
    for j,k in s:
        if df_error.loc[i,"Timestamp"]>k:
            week=j
    weeknumber.append(week)
df_error["WeekNumber"]=weeknumber

In [ ]:
#df_error[df_error["NotebookName"]=='DKT.ipynb']

In [ ]:
df_error1=df_error[["WeekNumber", "Username", "ErrorName","error_id"]].groupby(["WeekNumber", "Username", "ErrorName"]).count()
df_error1=df_error1.reset_index(drop=False)
#df_error1.head()

In [ ]:
df_error2=df_error[["WeekNumber", "Username", "ErrorName","Solved"]].groupby(["WeekNumber", "Username", "ErrorName"]).sum()
df_error2=df_error2.reset_index(drop=False)
#df_error2.head()

In [ ]:
df_error3=df_error[df_error["Solved"]==1]
df_error3["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error3["Timespent"]))
df_error3=df_error3[["WeekNumber", "Username", "ErrorName","Timespent"]].groupby(["WeekNumber", "Username", "ErrorName"]).mean()
df_error3=df_error3.reset_index(drop=False)
#df_error3.head()

In [ ]:
df_error1=pd.merge(df_error1, df_error2, on=["WeekNumber","Username","ErrorName"], how="left")
df_error1=pd.merge(df_error1, df_error3, on=["WeekNumber","Username","ErrorName"], how="left")

In [ ]:
df_error1["failfixct"]=df_error1["error_id"]-df_error1["Solved"]

In [ ]:
df_error1.describe()

In [ ]:
df_error4=df_error[df_error["Solved"]==1]
df_error4["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error4["Timespent"]))
df_error4=df_error4[["WeekNumber", "Username","Timespent"]].groupby(["WeekNumber", "Username"]).sum()
df_error4=df_error4.reset_index(drop=False)
df_error4=df_error4.rename(columns={"Timespent":"DebugTime"})

In [ ]:
Username=list(set(df_error1["Username"])-{"lairuilin","jimgo","laitest","susan","hch"})
WeekNumber=list(set(df_error1["WeekNumber"]))

df_log=pd.DataFrame()
for usr in Username:
    dfx=pd.DataFrame({"Username":[usr]*len(WeekNumber),"WeekNumber":WeekNumber})
    df_log=pd.concat([df_log, dfx], axis=0)
df_log=df_log.reset_index(drop=True)

In [ ]:
li="SyntaxError NameError AttributeError TypeError KeyError ValueError IndentationError IndexError".split()
for i in li:
    df_log[i+"_Ct"]=[0]*len(df_log)
    df_log[i+"_FailfixCt"]=[0]*len(df_log)
    df_log[i+"_Fixtime"]=[np.nan]*len(df_log)

In [ ]:
for er in li:
    print("=============="+er+"===============")
    dfx=df_error1[df_error1["ErrorName"]==er]
    print(len(dfx))
    for wk in WeekNumber:
        print("=="+str(wk)+"==")
        for usr in Username:
            dfx1=dfx[(dfx["WeekNumber"]==wk)&(dfx["Username"]==usr)]
            if len(dfx1)!=0:
                ct=int(dfx1.loc[:,"error_id"])
                fp=float(dfx1.loc[:,"failfixct"])
                ts=float(dfx1.loc[:,"Timespent"])
                ind=df_log[(df_log["WeekNumber"]==wk)&(df_log["Username"]==usr)].index[0]
                df_log.loc[ind, er+"_Ct"]=ct
                df_log.loc[ind, er+"_FailfixCt"]=fp
                df_log.loc[ind, er+"_Fixtime"]=ts
                print(str(usr)+"-->"+str(ct)+", "+str(fp)+", "+str(ts))

In [ ]:
df_log=pd.merge(df_log, df_error4, on=["WeekNumber","Username"], how="left")

In [ ]:
df_log.describe()

#### pattern系列

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
sql='''select `log_id`,`Username`,`notebookName`,`event`,`text`,`CleanCode`,`timestamp`from `edm_backup`.`edm_log`;'''
cursor=conn.cursor()
row=cursor.execute(sql)
lg=cursor.fetchall()
cursor.close()

Log_id=[]
Username=[]
NotebookName=[]
Event=[]
Text=[]
CleanCode=[]
Timestamp=[]

for i in range(len(lg)):
    Log_id.append(lg[i][0])
    Username.append(lg[i][1])
    NotebookName.append(lg[i][2])
    Event.append(lg[i][3])
    Text.append(lg[i][4])
    CleanCode.append(lg[i][5])
    Timestamp.append(lg[i][6])
    
df_logcd=pd.DataFrame({"Log_id":Log_id,"Username":Username,"NotebookName":NotebookName,"Event":Event,"Text":Text,"CleanCode":CleanCode,"Timestamp":Timestamp})

In [ ]:
df_logcd["CleanCode"]=df_logcd["CleanCode"].fillna("")
df_logcd.Timestamp=list(map(lambda x: datetime.strptime(":".join(x.split(":")[:-1]), "%Y-%m-%d-%H:%M:%S"), df_logcd.Timestamp))
df_logcd.head(30)

In [ ]:
s=((1, datetime(2020, 9, 16, 0, 0)),
 (2, datetime(2020, 9, 23, 0, 0)),
 (3, datetime(2020, 9, 30, 0, 0)),
 (4, datetime(2020, 10, 7, 0, 0)),
 (5, datetime(2020, 10, 14, 0, 0)),
 (6, datetime(2020, 10, 21, 0, 0)),
 (7, datetime(2020, 10, 28, 0, 0)),
 (8, datetime(2020, 11, 4, 0, 0)),
 (9, datetime(2020, 11, 11, 0, 0)),
 (11, datetime(2020, 11, 25, 0, 0)),
 (12, datetime(2020, 12, 2, 0, 0)),
 (13, datetime(2020, 12, 9, 0, 0)),
 (14, datetime(2020, 12, 16, 0, 0)),
 (15, datetime(2020, 12, 23, 0, 0)),
 (16, datetime(2020, 12, 30, 0, 0)))

weeknumber=[]
for i in range(len(df_logcd)):
    week=0
    for j,k in s:
        if df_logcd.loc[i,"Timestamp"]>k:
            week=j
    weeknumber.append(week)
df_logcd["WeekNumber"]=weeknumber

In [ ]:
pattern_SimpleCondition_logct=[]
pattern_MultipleCondition_logct=[]
pattern_WhileIteration_logct=[]
pattern_ForInIteration_logct=[]
pattern_NestIteration_logct=[]
pattern_DefFunction_logct=[]

print("total:", len(df_logcd))
for xxx in range(len(df_logcd)):
    print(xxx)
    x=list(df_logcd["CleanCode"])[xxx]
    #MultipleCondition
    target=copy.deepcopy(x)
    mc=0
    for i in list(reversed(range(6))):
        #print("========"+str(i)+"========")
        c="[ ]{%d}if.*:.*\n([ ]{%d}.*\n)+?([ ]{%d}elif.*:.*\n)+?([ ]{%d}.*)([ ]{%d}else:.*\n)+?([ ]{%d}.*)"%(4*i, 4*i+4,4*i, 4*i+4,4*i, 4*i+4)
        c1="[ ]{%d}if.*:.*\n([ ]{%d}.*\n)+?([ ]{%d}elif.*:.*\n)+?([ ]{%d}.*)"%(4*i, 4*i+4,4*i, 4*i+4)
        c2="[ ]{%d}if.*:.*\n([ ]{%d}.*\n)+?([ ]{%d}else:.*\n)+?([ ]{%d}.*)"%(4*i, 4*i+4,4*i, 4*i+4)
        for j in re.finditer(c,target):
            target=target.replace(j.group(),"")
            mc+=1
        for j in re.finditer(c1,target):
            target=target.replace(j.group(),"")
            mc+=1
        for j in re.finditer(c2,target):
            target=target.replace(j.group(),"")
            mc+=1
            
    #SimpleCondition
    c="if .+:.*\n"
    sc=len(list(re.finditer(c,target)))

    #WhileIteration
    c="while .+:.*\n"
    wi=len(list(re.finditer(c,x)))

    #ForInIteration
    c="for .+ in .+:.*\n"
    fi=len(list(re.finditer(c,x)))

    #NestIteration
    ni=0
    for i in range(6):
        #print("======="+str(i)+"=======")
        c="[ ]{%d}((while .+:)|(for .+ in .+:)|(if .+:)).*\n([ ]{%d}.*\n)+"%(4*i, 4*i+4)
        for j in re.finditer(c,x):
            x1=j.group().replace("\n"+" "*(4*i), "\n").strip()
            c1="((while .+:)|(for .+ in .+:|(if .+:))).*\n([ ]{4}.*\n)"
            ni+=len(list(re.finditer(c1,x1)))
                
    #DefFunction
    c="def .+\(.*\):"
    fc=len(list(re.finditer(c,x)))
    
    pattern_SimpleCondition_logct.append(sc)
    pattern_MultipleCondition_logct.append(mc)
    pattern_WhileIteration_logct.append(wi)
    pattern_ForInIteration_logct.append(fi)
    pattern_NestIteration_logct.append(ni)
    pattern_DefFunction_logct.append(fc)

In [ ]:
df_logcd["pattern_SimpleCondition_logct"]=pattern_SimpleCondition_logct
df_logcd["pattern_MultipleCondition_logct"]=pattern_MultipleCondition_logct
df_logcd["pattern_WhileIteration_logct"]=pattern_WhileIteration_logct
df_logcd["pattern_ForInIteration_logct"]=pattern_ForInIteration_logct
df_logcd["pattern_NestIteration_logct"]=pattern_NestIteration_logct
df_logcd["pattern_DefFunction_logct"]=pattern_DefFunction_logct

In [ ]:
df_logcd1=df_logcd.groupby(["Username","WeekNumber"]).sum().reset_index(drop=False)
del df_logcd1["Log_id"]

In [ ]:
df_logcd.head(1)

In [ ]:
df_logcd1.head(1)

In [ ]:
df_log.head(1)

In [ ]:
df_log=pd.merge(df_log, df_logcd1, on=["Username","WeekNumber"], how="left")

#### copy/ paste/ execution


In [ ]:
df_logcd["Text"]=df_logcd["Text"].fillna("")

Username=[]
WeekNumber=[]
Codemix=[]
CopyCt=[]
PasteCt=[]
ExecutionCt=[]
ExternalPasteLength=[]

for usr in list(set(df_logcd["Username"])):
    df_logcdx=df_logcd[(df_logcd["Username"]==usr)]
    print("=====",usr,"=====")
    for wk in list(set(df_logcdx["WeekNumber"])):
        df_logcdxx=df_logcdx[df_logcdx["WeekNumber"]==wk]
        print(wk, len(df_logcdxx))
        df_logcdxx_ex=df_logcdxx[df_logcdxx["Event"]=="code_execution"]
        df_logcdxx_cp=df_logcdxx[df_logcdxx["Event"]=="code_copy"]
        df_logcdxx_ps=df_logcdxx[df_logcdxx["Event"]=="code_paste"]
        
        cdmix="\n".join(list(df_logcdxx_ex["CleanCode"]))
        copy_ct=len(df_logcdxx_cp)
        paste_ct=len(df_logcdxx_ps)
        execution_ct=len(df_logcdxx_ex)
        
        ex_paste=""
        #copycode="".join(list(df_logcdxx_cp["Text"]))
        for ind in df_logcdxx_ps.index:
            tx=df_logcdxx_ps.loc[ind, "Text"]
            previouscode=" ".join(list(df_logcdxx.loc[:ind, "CleanCode"]))
            if tx not in previouscode:
                ex_paste+=tx
        
        Username.append(usr)
        WeekNumber.append(wk)
        Codemix.append(cdmix)
        CopyCt.append(copy_ct)
        PasteCt.append(paste_ct)
        ExecutionCt.append(execution_ct)
        ExternalPasteLength.append(len(ex_paste))

df_cdmix=pd.DataFrame({
    "Username":Username,
    "WeekNumber":WeekNumber,
    "Codemix":Codemix,
    "CopyCt":CopyCt,
    "PasteCt":PasteCt,
    "ExecutionCt":ExecutionCt,
    "ExternalPasteLength":ExternalPasteLength})

#### module/ modulemethod/ basicmethod

In [ ]:
Module=[]
Module_ct=[]
modulemethod_ct=[]
modulemethod_log_usect=[]
modulemethod_log_use=[]
basicmethod_ct=[]
basicmethod_log_usect=[]
basicmethod_log_use=[]

for cd in Codemix:
    m=[]
    mc=0
    mluc=0
    mlu=""
    bmc=0
    bmluc=0
    bmlu=""

    for mth in dir(np):
        if len(list(re.finditer("np\."+mth,cd)))!=0:
            mc+=1
            mlu+=list(re.finditer("np\."+mth,cd))[0].group()+" "
            mluc+=len(list(re.finditer("np\."+mth, cd)))
    for mth in dir(pd):
        if len(list(re.finditer("pd\."+mth,cd)))!=0:
            mc+=1
            mlu+=list(re.finditer("pd\."+mth,cd))[0].group()+" "
            mluc+=len(list(re.finditer("pd\."+mth,cd)))
    for mth in dir(plt):
        if len(list(re.finditer("((plt)|(pt))\."+mth,cd)))!=0:
            mc+=1
            mlu+=list(re.finditer("((plt)|(pt))\."+mth,cd))[0].group()+" "
            mluc+=len(list(re.finditer("((plt)|(pt))\."+mth,cd)))
    for mth in modulemethod:
        if len(list(re.finditer("\."+mth,cd)))!=0:
            mc+=1
            mlu+=list(re.finditer("\."+mth,cd))[0].group()+" "
            mluc+=len(list(re.finditer("\."+mth,cd)))
    modulemethod_ct.append(mc)
    modulemethod_log_usect.append(mluc)
    modulemethod_log_use.append(mlu)
    
    for mth in basicmethod:
        if len(list(re.finditer("\."+mth,cd)))!=0:
            bmc+=1
            bmlu+=list(re.finditer("\."+mth,cd))[0].group()+" "
            bmluc+=len(list(re.finditer("\."+mth,cd)))
    basicmethod_ct.append(bmc)
    basicmethod_log_usect.append(bmluc)
    basicmethod_log_use.append(bmlu)    
    
    r="import .+ as .+"
    r1="import .+"
    for i in re.finditer(r,cd):
        mm=i.group().replace("import ","").split(" as ")[0].split(".")[0]
        if mm not in m:
            m.append(mm)
        cd=cd.replace(i.group(),"")
    for i in re.finditer(r1,cd):
        mm=i.group().replace("import ","")
        if "," in mm:
            mm=list(map(lambda x : x.strip(), mm.split(",")))
            for mmx in mm:
                if mmx not in m:
                    m.append(mmx.split(".")[0])
        else:
            if mm not in m:
                m.append(mm.split(".")[0].strip())
    m=list(set(m))
    Module_ct.append(len(m))
    Module.append(" ".join(m))

    
df_cdmix["modulemethod_ct"]=modulemethod_ct
df_cdmix["modulemethod_log_usect"]=modulemethod_log_usect
df_cdmix["modulemethod_log_use"]=modulemethod_log_use
df_cdmix["Module_ct"]=Module_ct
df_cdmix["Module"]=Module
df_cdmix["basicmethod_ct"]=basicmethod_ct
df_cdmix["basicmethod_log_usect"]=basicmethod_log_usect
df_cdmix["basicmethod_log_use"]=basicmethod_log_use

In [ ]:
df_cdmix.head()

In [ ]:
df_log=pd.merge(df_log, df_cdmix, on=["Username","WeekNumber"], how="left")

In [ ]:
df_log.head()

#### weekly grand total

In [ ]:
df_log=df_log.sort_values(["Username","WeekNumber"]).reset_index(drop=True)

df_log["Module"]=df_log["Module"].fillna("")
df_log["modulemethod_log_use"]=df_log["modulemethod_log_use"].fillna("")
df_log["basicmethod_log_use"]=df_log["basicmethod_log_use"].fillna("")

df_log["Grand_Module_ct"]=[0]*len(df_log)
df_log["Grand_modulemethod_ct"]=[0]*len(df_log)
df_log["Grand_basicmethod_ct"]=[0]*len(df_log)

for usr in list(set(df_log["Username"])):
    print(usr)
    dfx=df_log[df_log["Username"]==usr]
    mm=[]
    m=[]
    rm=0
    bm=[]
    for i in dfx.index:
        m.extend(df_log.loc[i,"Module"].split())
        mm.extend(df_log.loc[i,"modulemethod_log_use"].split())
        bm.extend(df_log.loc[i,"basicmethod_log_use"].split())
        df_log.loc[i,"Grand_Module_ct"]=len(set(m))
        df_log.loc[i,"Grand_modulemethod_ct"]=len(set(mm))
        df_log.loc[i,"Grand_basicmethod_ct"]=len(set(bm))

#### timespent

In [ ]:
conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
select `Username`,`notebookName`,`event`,`kernelId`,`timestamp`from `edm_backup`.`edm_log`;'''
cursor=conn.cursor()
row=cursor.execute(sql)
lg=cursor.fetchall()
cursor.close()

Username=[]
NotebookName=[]
Event=[]
KernelId=[]
Timestamp=[]

for i in range(len(lg)):
    Username.append(lg[i][0])
    NotebookName.append(lg[i][1])
    Event.append(lg[i][2])
    KernelId.append(lg[i][3])
    Timestamp.append(lg[i][4])
    
df_logtm=pd.DataFrame({"Username":Username,
                       "NotebookName":NotebookName,
                       "Event":Event,
                       "KernelId":KernelId,
                       "Timestamp":Timestamp})

df_logtm.Timestamp=list(map(lambda x: datetime.strptime(":".join(x.split(":")[:-1]), "%Y-%m-%d-%H:%M:%S"), df_logtm.Timestamp))

In [ ]:
Username=[]
Kernal=[]
Timestamp=[]
Timespent=[]

usr_list=list(set(df_logtm["Username"]))
df_logtm=df_logtm.sort_values("Timestamp", ascending=True)
for usr in usr_list:
    df_logtmx=df_logtm[df_logtm["Username"]==usr]
    kernal_list=list(set(df_logtmx["KernelId"]))
    for kernal in kernal_list:
        df_logtmxx=df_logtmx[df_logtmx["KernelId"]==kernal].reset_index(drop=True)
        for i in range(1,len(df_logtmxx)):
            Username.append(usr)
            Kernal.append(kernal)
            Timestamp.append(df_logtmxx.loc[i,"Timestamp"])
            Timespent.append((df_logtmxx.loc[i,"Timestamp"]-df_logtmxx.loc[i-1,"Timestamp"]).seconds)
df_tspent=pd.DataFrame({"Username":Username,"KernelId":Kernal,"Timestamp":Timestamp,"Timespent":Timespent})

In [ ]:
# IQRx15=(pd.Series(Timespent).describe()["75%"]-pd.Series(Timespent).describe()["25%"])*1.5
# uplimit=pd.Series(Timespent).describe()["50%"]+IQRx15
uplimit=300
Timespent_ad=list(map(lambda x: uplimit if x>uplimit else x, Timespent))
df_tspent["Timespent_ad"]=Timespent_ad

In [ ]:
s=((1, datetime(2020, 9, 16, 0, 0)),
 (2, datetime(2020, 9, 23, 0, 0)),
 (3, datetime(2020, 9, 30, 0, 0)),
 (4, datetime(2020, 10, 7, 0, 0)),
 (5, datetime(2020, 10, 14, 0, 0)),
 (6, datetime(2020, 10, 21, 0, 0)),
 (7, datetime(2020, 10, 28, 0, 0)),
 (8, datetime(2020, 11, 4, 0, 0)),
 (9, datetime(2020, 11, 11, 0, 0)),
 (11, datetime(2020, 11, 25, 0, 0)),
 (12, datetime(2020, 12, 2, 0, 0)),
 (13, datetime(2020, 12, 9, 0, 0)),
 (14, datetime(2020, 12, 16, 0, 0)),
 (15, datetime(2020, 12, 23, 0, 0)),
 (16, datetime(2020, 12, 30, 0, 0)))

weeknumber=[]
for i in range(len(df_tspent)):
    week=0
    for j,k in s:
        if df_tspent.loc[i,"Timestamp"]>k:
            week=j
    weeknumber.append(week)
df_tspent["WeekNumber"]=weeknumber

In [ ]:
df_tspent.iloc[100:150]

In [ ]:
df_tspent.to_excel("member_log_tmspent.xlsx")

In [ ]:
df_tspent_wk=df_tspent[["Username","WeekNumber","Timespent_ad"]].groupby(["WeekNumber","Username"]).sum().reset_index(drop=False)

In [ ]:
df_log=pd.merge(df_log, df_tspent_wk, on=["Username","WeekNumber"], how="left")

In [ ]:
list(df_log.columns)

In [ ]:
df_log.to_excel("member_weeklog_trainingdata.xlsx")

### member_semester_log

In [ ]:
df_log.head()

In [ ]:
df2=df_log[['Username','Module', 'modulemethod_log_use', 'basicmethod_log_use']]
df2.head()

In [ ]:
Username=[]
Module=[]
Module_ct=[]
Modulemethod_use=[]
Modulemethod_ct=[]
Basicmethod_use=[]
Basicmethod_ct=[]

for usr in list(set(df2["Username"])):
    df1x=df2[df2["Username"]==usr]

    module=list(set(" ".join(list(df1x["Module"])).split()))
    modulemethod_use=list(set(" ".join(list(df1x["modulemethod_log_use"])).split()))
    basicmethod_use=list(set(" ".join(list(df1x["basicmethod_log_use"])).split()))

    Username.append(usr)
    Module.append(" ".join(module))
    Module_ct.append(len(module))
    Modulemethod_use.append(" ".join(modulemethod_use))
    Modulemethod_ct.append(len(modulemethod_use))
    Basicmethod_use.append(" ".join(basicmethod_use))
    Basicmethod_ct.append(len(basicmethod_use))

df2=pd.DataFrame({
    "Username":Username,
    "Module":Module,
    "Module_ct":Module_ct,
    "Modulemethod_use":Modulemethod_use,
    "Modulemethod_ct":Modulemethod_ct,
    "Basicmethod_use":Basicmethod_use,
    "Basicmethod_ct":Basicmethod_ct})

In [ ]:
df_error3=df_error[df_error["Solved"]==1]
df_error3["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error3["Timespent"]))
df_error3=df_error3[["Username", "ErrorName","Timespent"]].groupby(["Username", "ErrorName"]).mean()
df_error3=df_error3.reset_index(drop=False)
df_error3.head()

In [ ]:
df3=df[['Username']].drop_duplicates(keep="last").reset_index(drop=True)
li="SyntaxError NameError AttributeError TypeError KeyError ValueError IndentationError IndexError".split()
for i in li:
    df3[i+"_Fixtime"]=[np.nan]*len(df3)
for er in li:
    print("=============="+er+"===============")
    dfx=df_error3[df_error3["ErrorName"]==er]
    for us in uss:
        dfx1=dfx[dfx["Username"]==str(us)]
        if len(dfx1)!=0:
            ts=float(dfx1.loc[:,"Timespent"])
            ind=df3[df3["Username"]==us].index[0]
            df3.loc[ind, er+"_Fixtime"]=ts

In [ ]:
#.sum()
df4=df_log[['Username',
 'SyntaxError_Ct',
 'NameError_Ct',
 'AttributeError_Ct',
 'TypeError_Ct',
 'KeyError_Ct',
 'ValueError_Ct',
 'IndentationError_Ct',
 'IndexError_Ct',
 'SyntaxError_FailfixCt',
 'NameError_FailfixCt',
 'AttributeError_FailfixCt',
 'TypeError_FailfixCt',
 'KeyError_FailfixCt',
 'ValueError_FailfixCt',
 'IndentationError_FailfixCt',
 'IndexError_FailfixCt',
 'pattern_SimpleCondition_logct',
 'pattern_MultipleCondition_logct',
 'pattern_WhileIteration_logct',
 'pattern_ForInIteration_logct',
 'pattern_NestIteration_logct',
 'pattern_DefFunction_logct',
 'CopyCt',
 'PasteCt',
 'ExecutionCt',
 'ExternalPasteLength',
 'modulemethod_log_usect',
 'basicmethod_log_usect',
 'Timespent_ad',
 'DebugTime'
           ]].groupby(['Username']).sum().reset_index(drop=False)

In [ ]:
df2["Username"]=df2["Username"].astype("int32")
df3["Username"]=df3["Username"].astype("int32")
df4["Username"]=df4["Username"].astype("int32")

df2=pd.merge(df2, df3, on=['Username'], how="left")
df2=pd.merge(df2, df4, on=['Username'], how="left")

In [ ]:
set(list(df_log.columns))-set(list(df2.columns))

In [ ]:
df2=pd.merge(df2, qs, left_on="Username", right_on="id", how="left").drop("id", axis=1)
# df2["Username"]=df2["Username"].astype("int32")
# df2=pd.merge(df2, fnsc, on="Username", how="left")

In [ ]:
df2.shape

In [ ]:
df2.to_excel("member_semesterlog_trainingdata.xlsx")

### member_checkpoint_log

#### before week6

In [ ]:
df_logx=df_log[df_log["WeekNumber"]<=5]

In [ ]:
df2x=df_logx[['Username','Module', 'modulemethod_log_use', 'basicmethod_log_use']]

Username=[]
Module=[]
Module_ct=[]
Modulemethod_use=[]
Modulemethod_ct=[]
Basicmethod_use=[]
Basicmethod_ct=[]

for usr in list(set(df2x["Username"])):
    df1x=df2x[df2x["Username"]==usr]

    module=list(set(" ".join(list(df1x["Module"])).split()))
    modulemethod_use=list(set(" ".join(list(df1x["modulemethod_log_use"])).split()))
    basicmethod_use=list(set(" ".join(list(df1x["basicmethod_log_use"])).split()))

    Username.append(usr)
    Module.append(" ".join(module))
    Module_ct.append(len(module))
    Modulemethod_use.append(" ".join(modulemethod_use))
    Modulemethod_ct.append(len(modulemethod_use))
    Basicmethod_use.append(" ".join(basicmethod_use))
    Basicmethod_ct.append(len(basicmethod_use))

df2x=pd.DataFrame({
    "Username":Username,
    "Module":Module,
    "Module_ct":Module_ct,
    "Modulemethod_use":Modulemethod_use,
    "Modulemethod_ct":Modulemethod_ct,
    "Basicmethod_use":Basicmethod_use,
    "Basicmethod_ct":Basicmethod_ct})

In [ ]:
df_error3=df_error[(df_error["Solved"]==1)&(df_error["WeekNumber"]<=5)]
df_error3["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error3["Timespent"]))
df_error3=df_error3[["Username", "ErrorName","Timespent"]].groupby(["Username", "ErrorName"]).mean()
df_error3=df_error3.reset_index(drop=False)
df_error3.head()

df3x=df[['Username']].drop_duplicates(keep="last").reset_index(drop=True)
li="SyntaxError NameError AttributeError TypeError KeyError ValueError IndentationError IndexError".split()
for i in li:
    df3x[i+"_Fixtime"]=[np.nan]*len(df3x)
for er in li:
    print("=============="+er+"===============")
    dfx=df_error3[df_error3["ErrorName"]==er]
    for us in uss:
        dfx1=dfx[dfx["Username"]==str(us)]
        if len(dfx1)!=0:
            ts=float(dfx1.loc[:,"Timespent"])
            ind=df3x[df3x["Username"]==us].index[0]
            df3x.loc[ind, er+"_Fixtime"]=ts

In [ ]:
df4x=df_logx[['Username',
 'SyntaxError_Ct',
 'NameError_Ct',
 'AttributeError_Ct',
 'TypeError_Ct',
 'KeyError_Ct',
 'ValueError_Ct',
 'IndentationError_Ct',
 'IndexError_Ct',
 'SyntaxError_FailfixCt',
 'NameError_FailfixCt',
 'AttributeError_FailfixCt',
 'TypeError_FailfixCt',
 'KeyError_FailfixCt',
 'ValueError_FailfixCt',
 'IndentationError_FailfixCt',
 'IndexError_FailfixCt',
 'pattern_SimpleCondition_logct',
 'pattern_MultipleCondition_logct',
 'pattern_WhileIteration_logct',
 'pattern_ForInIteration_logct',
 'pattern_NestIteration_logct',
 'pattern_DefFunction_logct',
 'CopyCt',
 'PasteCt',
 'ExecutionCt',
 'ExternalPasteLength',
 'modulemethod_log_usect',
 'basicmethod_log_usect',
 'Timespent_ad',
 'DebugTime']].groupby(['Username']).sum().reset_index(drop=False)

In [ ]:
df2x['Username']=df2x['Username'].astype("int32")
df3x['Username']=df3x['Username'].astype("int32")
df4x['Username']=df4x['Username'].astype("int32")

df2x=pd.merge(df2x, df3x, on=['Username'], how="left")
df2x=pd.merge(df2x, df4x, on=['Username'], how="left")

In [ ]:
df2x=pd.merge(df2x, qs, left_on="Username", right_on="id", how="left").drop("id", axis=1)
# df2x["Username"]=df2x["Username"].astype("int32")
# df2x=pd.merge(df2x, quiz[["Username","Quiz06","Quiz06_group"]], on="Username", how="left")

In [ ]:
df2x.to_excel("member_week06bflog_trainingdata.xlsx")

#### before week12

In [ ]:
df_logx=df_log[df_log["WeekNumber"]<=11]

df2x=df_logx[['Username','Module', 'modulemethod_log_use', 'basicmethod_log_use']]

Username=[]
Module=[]
Module_ct=[]
Modulemethod_use=[]
Modulemethod_ct=[]
Basicmethod_use=[]
Basicmethod_ct=[]

for usr in list(set(df2x["Username"])):
    df1x=df2x[df2x["Username"]==usr]

    module=list(set(" ".join(list(df1x["Module"])).split()))
    modulemethod_use=list(set(" ".join(list(df1x["modulemethod_log_use"])).split()))
    basicmethod_use=list(set(" ".join(list(df1x["basicmethod_log_use"])).split()))

    Username.append(usr)
    Module.append(" ".join(module))
    Module_ct.append(len(module))
    Modulemethod_use.append(" ".join(modulemethod_use))
    Modulemethod_ct.append(len(modulemethod_use))
    Basicmethod_use.append(" ".join(basicmethod_use))
    Basicmethod_ct.append(len(basicmethod_use))

df2x=pd.DataFrame({
    "Username":Username,
    "Module":Module,
    "Module_ct":Module_ct,
    "Modulemethod_use":Modulemethod_use,
    "Modulemethod_ct":Modulemethod_ct,
    "Basicmethod_use":Basicmethod_use,
    "Basicmethod_ct":Basicmethod_ct})

df_error3=df_error[(df_error["Solved"]==1)&(df_error["WeekNumber"]<=11)]
df_error3["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error3["Timespent"]))
df_error3=df_error3[["Username", "ErrorName","Timespent"]].groupby(["Username", "ErrorName"]).mean()
df_error3=df_error3.reset_index(drop=False)
df_error3.head()

df3x=df[['Username']].drop_duplicates(keep="last").reset_index(drop=True)
li="SyntaxError NameError AttributeError TypeError KeyError ValueError IndentationError IndexError".split()
for i in li:
    df3x[i+"_Fixtime"]=[np.nan]*len(df3x)
for er in li:
    print("=============="+er+"===============")
    dfx=df_error3[df_error3["ErrorName"]==er]
    for us in uss:
        dfx1=dfx[dfx["Username"]==str(us)]
        if len(dfx1)!=0:
            ts=float(dfx1.loc[:,"Timespent"])
            ind=df3x[df3x["Username"]==us].index[0]
            df3x.loc[ind, er+"_Fixtime"]=ts

df4x=df_logx[['Username',
 'SyntaxError_Ct', 'NameError_Ct', 'AttributeError_Ct', 'TypeError_Ct',
 'KeyError_Ct', 'ValueError_Ct', 'IndentationError_Ct', 'IndexError_Ct',
 'SyntaxError_FailfixCt', 'NameError_FailfixCt', 'AttributeError_FailfixCt', 'TypeError_FailfixCt',
 'KeyError_FailfixCt', 'ValueError_FailfixCt', 'IndentationError_FailfixCt', 'IndexError_FailfixCt',
 'pattern_SimpleCondition_logct', 'pattern_MultipleCondition_logct', 'pattern_WhileIteration_logct',
 'pattern_ForInIteration_logct', 'pattern_NestIteration_logct', 'pattern_DefFunction_logct',
 'CopyCt',
 'PasteCt',
 'ExecutionCt',
 'ExternalPasteLength',
 'modulemethod_log_usect',
 'basicmethod_log_usect',
 'Timespent_ad',
 'DebugTime']].groupby(['Username']).sum().reset_index(drop=False)

df2x['Username']=df2x['Username'].astype("int32")
df3x['Username']=df3x['Username'].astype("int32")
df4x['Username']=df4x['Username'].astype("int32")
df2x=pd.merge(df2x, df3x, on=['Username'], how="left")
df2x=pd.merge(df2x, df4x, on=['Username'], how="left")

df2x=pd.merge(df2x, qs, left_on="Username", right_on="id", how="left").drop("id", axis=1)
# df2x["Username"]=df2x["Username"].astype("int32")
# df2x=pd.merge(df2x, quiz, on="Username", how="left")

#df2x=df2x.drop(["Quiz06","Quiz06_group"], axis=1)
df2x.to_excel("member_week12bflog_trainingdata.xlsx")

#### before week16

In [ ]:
df_logx=df_log[df_log["WeekNumber"]<=15]

df2x=df_logx[['Username','Module', 'modulemethod_log_use', 'basicmethod_log_use']]

Username=[]
Module=[]
Module_ct=[]
Modulemethod_use=[]
Modulemethod_ct=[]
Basicmethod_use=[]
Basicmethod_ct=[]

for usr in list(set(df2x["Username"])):
    df1x=df2x[df2x["Username"]==usr]

    module=list(set(" ".join(list(df1x["Module"])).split()))
    modulemethod_use=list(set(" ".join(list(df1x["modulemethod_log_use"])).split()))
    basicmethod_use=list(set(" ".join(list(df1x["basicmethod_log_use"])).split()))

    Username.append(usr)
    Module.append(" ".join(module))
    Module_ct.append(len(module))
    Modulemethod_use.append(" ".join(modulemethod_use))
    Modulemethod_ct.append(len(modulemethod_use))
    Basicmethod_use.append(" ".join(basicmethod_use))
    Basicmethod_ct.append(len(basicmethod_use))

df2x=pd.DataFrame({
    "Username":Username,
    "Module":Module,
    "Module_ct":Module_ct,
    "Modulemethod_use":Modulemethod_use,
    "Modulemethod_ct":Modulemethod_ct,
    "Basicmethod_use":Basicmethod_use,
    "Basicmethod_ct":Basicmethod_ct})

df_error3=df_error[(df_error["Solved"]==1)&(df_error["WeekNumber"]<=15)]
df_error3["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error3["Timespent"]))
df_error3=df_error3[["Username", "ErrorName","Timespent"]].groupby(["Username", "ErrorName"]).mean()
df_error3=df_error3.reset_index(drop=False)
df_error3.head()

df3x=df[['Username']].drop_duplicates(keep="last").reset_index(drop=True)
li="SyntaxError NameError AttributeError TypeError KeyError ValueError IndentationError IndexError".split()
for i in li:
    df3x[i+"_Fixtime"]=[np.nan]*len(df3x)
for er in li:
    print("=============="+er+"===============")
    dfx=df_error3[df_error3["ErrorName"]==er]
    for us in uss:
        dfx1=dfx[dfx["Username"]==str(us)]
        if len(dfx1)!=0:
            ts=float(dfx1.loc[:,"Timespent"])
            ind=df3x[df3x["Username"]==us].index[0]
            df3x.loc[ind, er+"_Fixtime"]=ts

df4x=df_logx[['Username',
 'SyntaxError_Ct', 'NameError_Ct', 'AttributeError_Ct', 'TypeError_Ct',
 'KeyError_Ct', 'ValueError_Ct', 'IndentationError_Ct', 'IndexError_Ct',
 'SyntaxError_FailfixCt', 'NameError_FailfixCt', 'AttributeError_FailfixCt', 'TypeError_FailfixCt',
 'KeyError_FailfixCt', 'ValueError_FailfixCt', 'IndentationError_FailfixCt', 'IndexError_FailfixCt',
 'pattern_SimpleCondition_logct', 'pattern_MultipleCondition_logct', 'pattern_WhileIteration_logct',
 'pattern_ForInIteration_logct', 'pattern_NestIteration_logct', 'pattern_DefFunction_logct',
 'CopyCt',
 'PasteCt',
 'ExecutionCt',
 'ExternalPasteLength',
 'modulemethod_log_usect',
 'basicmethod_log_usect',
 'Timespent_ad',
 'DebugTime']].groupby(['Username']).sum().reset_index(drop=False)

df2x['Username']=df2x['Username'].astype("int32")
df3x['Username']=df3x['Username'].astype("int32")
df4x['Username']=df4x['Username'].astype("int32")
df2x=pd.merge(df2x, df3x, on=['Username'], how="left")
df2x=pd.merge(df2x, df4x, on=['Username'], how="left")

df2x=pd.merge(df2x, qs, left_on="Username", right_on="id", how="left").drop("id", axis=1)
# df2x["Username"]=df2x["Username"].astype("int32")
# df2x=pd.merge(df2x, quiz, on="Username", how="left")

#df2x=df2x.drop(["Quiz06","Quiz06_group"], axis=1)
df2x.to_excel("member_week16bflog_trainingdata.xlsx")

#### week6~week11

In [ ]:
df_logx=df_log[(df_log["WeekNumber"]<=11)&(df_log["WeekNumber"]>5)]

df2x=df_logx[['Username','Module', 'modulemethod_log_use', 'basicmethod_log_use']]

Username=[]
Module=[]
Module_ct=[]
Modulemethod_use=[]
Modulemethod_ct=[]
Basicmethod_use=[]
Basicmethod_ct=[]

for usr in list(set(df2x["Username"])):
    df1x=df2x[df2x["Username"]==usr]

    module=list(set(" ".join(list(df1x["Module"])).split()))
    modulemethod_use=list(set(" ".join(list(df1x["modulemethod_log_use"])).split()))
    basicmethod_use=list(set(" ".join(list(df1x["basicmethod_log_use"])).split()))

    Username.append(usr)
    Module.append(" ".join(module))
    Module_ct.append(len(module))
    Modulemethod_use.append(" ".join(modulemethod_use))
    Modulemethod_ct.append(len(modulemethod_use))
    Basicmethod_use.append(" ".join(basicmethod_use))
    Basicmethod_ct.append(len(basicmethod_use))

df2x=pd.DataFrame({
    "Username":Username,
    "Module":Module,
    "Module_ct":Module_ct,
    "Modulemethod_use":Modulemethod_use,
    "Modulemethod_ct":Modulemethod_ct,
    "Basicmethod_use":Basicmethod_use,
    "Basicmethod_ct":Basicmethod_ct})

df_error3=df_error[(df_error["Solved"]==1)&(df_error["WeekNumber"]>5)&(df_error["WeekNumber"]<=11)]
df_error3["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error3["Timespent"]))
df_error3=df_error3[["Username", "ErrorName","Timespent"]].groupby(["Username", "ErrorName"]).mean()
df_error3=df_error3.reset_index(drop=False)
df_error3.head()

df3x=df[['Username']].drop_duplicates(keep="last").reset_index(drop=True)
li="SyntaxError NameError AttributeError TypeError KeyError ValueError IndentationError IndexError".split()
for i in li:
    df3x[i+"_Fixtime"]=[np.nan]*len(df3x)
for er in li:
    print("=============="+er+"===============")
    dfx=df_error3[df_error3["ErrorName"]==er]
    for us in uss:
        dfx1=dfx[dfx["Username"]==str(us)]
        if len(dfx1)!=0:
            ts=float(dfx1.loc[:,"Timespent"])
            ind=df3x[df3x["Username"]==us].index[0]
            df3x.loc[ind, er+"_Fixtime"]=ts

df4x=df_logx[['Username',
 'SyntaxError_Ct', 'NameError_Ct', 'AttributeError_Ct', 'TypeError_Ct',
 'KeyError_Ct', 'ValueError_Ct', 'IndentationError_Ct', 'IndexError_Ct',
 'SyntaxError_FailfixCt', 'NameError_FailfixCt', 'AttributeError_FailfixCt', 'TypeError_FailfixCt',
 'KeyError_FailfixCt', 'ValueError_FailfixCt', 'IndentationError_FailfixCt', 'IndexError_FailfixCt',
 'pattern_SimpleCondition_logct', 'pattern_MultipleCondition_logct', 'pattern_WhileIteration_logct',
 'pattern_ForInIteration_logct', 'pattern_NestIteration_logct', 'pattern_DefFunction_logct',
 'CopyCt',
 'PasteCt',
 'ExecutionCt',
 'ExternalPasteLength',
 'modulemethod_log_usect',
 'basicmethod_log_usect',
 'Timespent_ad',
 'DebugTime']].groupby(['Username']).sum().reset_index(drop=False)

df2x['Username']=df2x['Username'].astype("int32")
df3x['Username']=df3x['Username'].astype("int32")
df4x['Username']=df4x['Username'].astype("int32")
df2x=pd.merge(df2x, df3x, on=['Username'], how="left")
df2x=pd.merge(df2x, df4x, on=['Username'], how="left")

df2x=pd.merge(df2x, qs, left_on="Username", right_on="id", how="left").drop("id", axis=1)
# df2x["Username"]=df2x["Username"].astype("int32")
# df2x=pd.merge(df2x, quiz, on="Username", how="left")

#df2x=df2x.drop(["Quiz06","Quiz06_group"], axis=1)
df2x.to_excel("member_week6to11log_trainingdata.xlsx")

#### week12~week15

In [ ]:
df_logx=df_log[(df_log["WeekNumber"]<=15)&(df_log["WeekNumber"]>11)]

df2x=df_logx[['Username','Module', 'modulemethod_log_use', 'basicmethod_log_use']]

Username=[]
Module=[]
Module_ct=[]
Modulemethod_use=[]
Modulemethod_ct=[]
Basicmethod_use=[]
Basicmethod_ct=[]

for usr in list(set(df2x["Username"])):
    df1x=df2x[df2x["Username"]==usr]

    module=list(set(" ".join(list(df1x["Module"])).split()))
    modulemethod_use=list(set(" ".join(list(df1x["modulemethod_log_use"])).split()))
    basicmethod_use=list(set(" ".join(list(df1x["basicmethod_log_use"])).split()))

    Username.append(usr)
    Module.append(" ".join(module))
    Module_ct.append(len(module))
    Modulemethod_use.append(" ".join(modulemethod_use))
    Modulemethod_ct.append(len(modulemethod_use))
    Basicmethod_use.append(" ".join(basicmethod_use))
    Basicmethod_ct.append(len(basicmethod_use))

df2x=pd.DataFrame({
    "Username":Username,
    "Module":Module,
    "Module_ct":Module_ct,
    "Modulemethod_use":Modulemethod_use,
    "Modulemethod_ct":Modulemethod_ct,
    "Basicmethod_use":Basicmethod_use,
    "Basicmethod_ct":Basicmethod_ct})

df_error3=df_error[(df_error["Solved"]==1)&(df_error["WeekNumber"]>11)&(df_error["WeekNumber"]<=15)]
df_error3["Timespent"]=list(map(lambda x: 7200 if x>7200 else x, df_error3["Timespent"]))
df_error3=df_error3[["Username", "ErrorName","Timespent"]].groupby(["Username", "ErrorName"]).mean()
df_error3=df_error3.reset_index(drop=False)
df_error3.head()

df3x=df[['Username']].drop_duplicates(keep="last").reset_index(drop=True)
li="SyntaxError NameError AttributeError TypeError KeyError ValueError IndentationError IndexError".split()
for i in li:
    df3x[i+"_Fixtime"]=[np.nan]*len(df3x)
for er in li:
    print("=============="+er+"===============")
    dfx=df_error3[df_error3["ErrorName"]==er]
    for us in uss:
        dfx1=dfx[dfx["Username"]==str(us)]
        if len(dfx1)!=0:
            ts=float(dfx1.loc[:,"Timespent"])
            ind=df3x[df3x["Username"]==us].index[0]
            df3x.loc[ind, er+"_Fixtime"]=ts

df4x=df_logx[['Username',
 'SyntaxError_Ct', 'NameError_Ct', 'AttributeError_Ct', 'TypeError_Ct',
 'KeyError_Ct', 'ValueError_Ct', 'IndentationError_Ct', 'IndexError_Ct',
 'SyntaxError_FailfixCt', 'NameError_FailfixCt', 'AttributeError_FailfixCt', 'TypeError_FailfixCt',
 'KeyError_FailfixCt', 'ValueError_FailfixCt', 'IndentationError_FailfixCt', 'IndexError_FailfixCt',
 'pattern_SimpleCondition_logct', 'pattern_MultipleCondition_logct', 'pattern_WhileIteration_logct',
 'pattern_ForInIteration_logct', 'pattern_NestIteration_logct', 'pattern_DefFunction_logct',
 'CopyCt',
 'PasteCt',
 'ExecutionCt',
 'ExternalPasteLength',
 'modulemethod_log_usect',
 'basicmethod_log_usect',
 'Timespent_ad',
 'DebugTime']].groupby(['Username']).sum().reset_index(drop=False)

df2x['Username']=df2x['Username'].astype("int32")
df3x['Username']=df3x['Username'].astype("int32")
df4x['Username']=df4x['Username'].astype("int32")
df2x=pd.merge(df2x, df3x, on=['Username'], how="left")
df2x=pd.merge(df2x, df4x, on=['Username'], how="left")

df2x=pd.merge(df2x, qs, left_on="Username", right_on="id", how="left").drop("id", axis=1)
# df2x["Username"]=df2x["Username"].astype("int32")
# df2x=pd.merge(df2x, quiz, on="Username", how="left")

#df2x=df2x.drop(["Quiz06","Quiz06_group"], axis=1)
df2x.to_excel("member_week12to15log_trainingdata.xlsx")

# data backup

In [ ]:
import time
import datetime
import pymysql
import numpy as np
import pandas as pd
import re
import os
import copy
from datetime import datetime, timedelta

conn=pymysql.connect(host="", port=3306, user="", passwd="", db="", charset="")
cursor=conn.cursor()
cursor.execute("select * from `edm`.`edm_log` ;") 
edm_log=cursor.fetchall()
cursor.close()
x= open("backup_edm_log.txt","w",encoding="utf-8")
x.write(str(edm_log))
x.close

cursor=conn.cursor()
cursor.execute("select * from `edm`.`implementation` ;") 
implementation=cursor.fetchall()
cursor.close()
x= open("backup_implementation.txt","w",encoding="utf-8")
x.write(str(implementation))
x.close

cursor=conn.cursor()
cursor.execute("select * from `edm`.`member_class` ;") 
member_class=cursor.fetchall()
cursor.close()
x= open("backup_member_class.txt","w",encoding="utf-8")
x.write(str(member_class))
x.close

cursor=conn.cursor()
cursor.execute("select * from `edm`.`pre-q` ;") 
preq=cursor.fetchall()
cursor.close()
x= open("backup_pre_q.txt","w",encoding="utf-8")
x.write(str(preq))
x.close

cursor=conn.cursor()
cursor.execute("select * from `edm`.`section` ;") 
section=cursor.fetchall()
cursor.close()
x= open("backup_section.txt","w",encoding="utf-8")
x.write(str(section))
x.close

cursor=conn.cursor()
cursor.execute("select * from `edm`.`member` ;") 
member=cursor.fetchall()
cursor.close()
x= open("backup_member.txt","w",encoding="utf-8")
x.write(str(member))
x.close

conn.close()

# tableau user

In [ ]:
x1=pd.read_excel("活頁簿1.xlsx", sheet_name="工作表1")
x2=pd.read_excel("活頁簿1.xlsx", sheet_name="工作表2")
x3=pd.read_excel("活頁簿1.xlsx", sheet_name="工作表3")

In [ ]:
xx=pd.merge(x1,x3, on="username", how="outer")
xx=pd.merge(xx,x2, on="email", how="outer")
xx=xx[["tableauuser","name","truename_x","username","email"]]
xx.sort_values("tableauuser")

In [ ]:
xx[pd.isnull(xx["tableauuser"])]